<p align="center">
  <img src="header.png" width="100%">
</p>


<div style="text-align: center;">
    <strong style="display: block; margin-bottom: 10px;">Group P</strong> 
    <table style="margin: 0 auto; border-collapse: collapse; border: 1px solid black;">
        <tr>
            <th style="border: 1px solid white; padding: 8px;">Name</th>
            <th style="border: 1px solid white; padding: 8px;">Student ID</th>
        </tr>
        <tr>
            <td style="border: 1px solid white; padding: 8px;">Beatriz Monteiro</td>
            <td style="border: 1px solid white; padding: 8px;">20240591</td>
        </tr>
        <tr>
            <td style="border: 1px solid white; padding: 8px;">Catarina Nunes</td>
            <td style="border: 1px solid white; padding: 8px;">20230083</td>
        </tr>
        <tr>
            <td style="border: 1px solid white; padding: 8px;">Margarida Raposo</td>
            <td style="border: 1px solid white; padding: 8px;">20241020</td>
        </tr>
        <tr>
            <td style="border: 1px solid white; padding: 8px;">Teresa Menezes</td>
            <td style="border: 1px solid white; padding: 8px;">20240333</td>
        </tr>
    </table>
</div>

### 🔗 Table of Contents <a id='table-of-contents'></a>
1. [Introduction](#introduction): Previous Notebook
2. [Business Understanding](#business-understanding): Previous Notebook  
3. [Data Understanding](#data-understanding): Previous Notebook
4. [Data Preparation](#data-preparation): Previous Notebook + In this one we have the preparation pipeline  
5. [Modeling](#modeling): 
6. [Evaluation](#evaluation)
7. [Conclusion](#conclusion) 

---

### <span style="background-color:#000027; padding:5px; border-radius:5px;"> 📌 Introduction <a id='introduction'></a>

This project follows the **CRISP-DM** methodology to conduct a monthly sales forecast of the smart infrastructure business unit of Siemens. 

<b style="background-color:#A9A9A9; padding:5px; border-radius:5px; display: inline-block;">CRISP-DM</b>

<ul style="margin-bottom: 30px;">
    <li><u>Business Understanding</u>: Defining objectives, assessing resources, and project planning.</li>
    <li><u>Data Understanding</u>: Collecting, exploring, and verifying data quality.</li>
    <li><u>Data Preparation</u>: Selecting, cleaning, constructing, integrating, and formatting data to ensure it is ready for analysis.</li>
    <li><u>Modeling</u>: Selecting and applying various modeling techniques while calibrating their parameters to optimal values.</li>
    <li><u>Evaluation</u>: Select the models which are the best performers and evaluate thoroughly if they align with the business objectives. </li>
    <li><u>Deployment</u>: Bridge between data mining goals and the business application of the finalized model.</li>
</ul>

---
<b style="background-color:#A9A9A9; padding:5px; border-radius:5px; display: inline-block;">Montlhy Sales Forecast</b>

<p style="margin-bottom: 50px;"> This case study focuses on ... </p>

---

<b style="background-color:#A9A9A9; padding:5px; border-radius:5px; display: inline-block;">Market Data</b><br>
(https://ec.europa.eu/eurostat/statistics-explained/index.php?title=Thematic_glossaries)<br>
Due to the lack of information about the macroeconomic indicatores used, our analysis was based on the assumption that the following definitions are correct.


- The **Production index** measures changes in value added at factor cost of industry and construction over a month. It does this by measuring changes in the volume of output and activity monthly.
- The **Shipment index** is a measure of the volume of goods shipped by manufacturers monthly. It is an indicator of economic trends, since it reflects production, supply chain dynamics and overall industrial performance. 
- The **World prices of materials** is monthly average global market prices for key industrial inputs, such as energy, heavy materials, etc. These are likely to fluctuate with changes in supply/demand dynamics and economic conditions. 
- The **United States: EUR in LCU** is the monthly exchange rate of the Euro, expressed in local currency units from the United States (US dollar), representing the number of US dollars per euro. It indicates currency fluctuation, which often impacts trade.
- The **Producer prices** measures the monthly change in the trading price of products and related services, from the seller's perspective. It does not only serve as an early indicator of inflationary pressures in the economy before it reaches the consumer, but it can also record the evolution of prices over longer time periods.

<br>

| **Features**                                    | **Feature Description**                                                                                             |
|-------------------------------------------------|---------------------------------------------------------------------------------------------------------------------|
| *MAB_ELE_PRO156*                                | **Production index** for machinery and electrical equipment in China.                                                     |
| *MAB_ELE_SHP156*                                | **Shipments index** for machinery and electrical equipment in China.                                                     |
| *MAB_ELE_PRO250*                                | **Production index** for machinery and electrical equipment in France.                                                    |
| *MAB_ELE_SHP250*                                | **Shipments index** for machinery and electrical equipment in France.                                                    |
| *MAB_ELE_PRO276*                                | **Production index** for machinery and electrical equipment in Germany.                                                  |
| *MAB_ELE_SHP276*                                | **Shipments index** for machinery and electrical equipment in Germany.                                                  |
| *MAB_ELE_PRO380*                                | **Production index** for machinery and electrical equipment in Italy.                                                    |
| *MAB_ELE_SHP380*                                | **Shipments index** for machinery and electrical equipment in Italy.                                                    |
| *MAB_ELE_PRO392*                                | **Production index** for machinery and electrical equipment in Japan.                                                    |
| *MAB_ELE_SHP392*                                | **Shipments index** for machinery and electrical equipment in Japan.                                                    |
| *MAB_ELE_PRO756*                                | **Production index** for machinery and electrical equipment in Switzerland.                                              |
| *MAB_ELE_SHP756*                                | **Shipments index** for machinery and electrical equipment in Switzerland.                                              |
| *MAB_ELE_PRO826*                                | **Production index** for machinery and electrical equipment in United Kingdom.                                           |
| *MAB_ELE_SHP826*                                | **Shipments index** for machinery and electrical equipment in United Kingdom.                                           |
| *MAB_ELE_PRO840*                                | **Production index** for machinery and electrical equipment in the United States.                                        |
| *MAB_ELE_SHP840*                                | **Shipments index** for machinery and electrical equipment in the United States.                                        |
| *MAB_ELE_PRO1100*                               | **Production index** for machinery and electrical equipment in Europe.                                                   |
| *MAB_ELE_SHP1100*                               | **Shipments index** for machinery and electrical equipment in Europe.                                                   |
| *RohiBASEMET1000_org*                           | **World price** of base metals.                                                                                          |
| *RohiENERGY1000_org*                            | **World price** of energy.                                                                                               |
| *RohiMETMIN1000_org*                            | **World price** of metals and minerals.                                                                                   |
| *RohiNATGAS1000_org*                            | **World price** of natural gas index.                                                                                     |
| *RohCRUDE_PETRO1000_org*                        | **World price** of crude oil (average).                                                                                   |
| *RohCOPPER1000_org*                             | **World price** of copper.                                                                                                |
| *WKLWEUR840_org*                                | United States: **Euros in local currency units** (LCU).                                                                    |
| *PRI27840_org*                                  | United States: Electrical equipment (**Producer Prices**).                                                                |
| *PRI27826_org*                                  | United Kingdom: Electrical equipment (**Producer Prices**).                                                               |
| *PRI27380_org*                                  | Italy: Electrical equipment (**Producer Prices**).                                                                        |
| *PRI27250_org*                                  | France: Electrical equipment (**Producer Prices**).                                                                       |
| *PRI27276_org*                                  | Germany: Electrical equipment (**Producer Prices**).                                                                      |
| *PRI27156_org*                                  | China: Electrical equipment (**Producer Prices**).                                                                       |
| *PRO28840_org*                                  | United States: Machinery and equipment n.e.c. (**Production index**).                                                      |
| *PRO281000_org*                                 | World: Machinery and equipment n.e.c. (**Production index**).                                                             |
| *PRO28756_org*                                  | Switzerland: Machinery and equipment n.e.c. (**Production index**).                                                       |
| *PRO28826_org*                                  | United Kingdom: Machinery and equipment n.e.c. (**Production index**).                                                    |
| *PRO28380_org*                                  | Italy: Machinery and equipment n.e.c. (**Production index**).                                                             |
| *PRO28392_org*                                  | Japan: Machinery and equipment n.e.c. (**Production index**).                                                             |
| *PRO28250_org*                                  | France: Machinery and equipment n.e.c. (**Production index**).                                                            |
| *PRO28276_org*                                  | Germany: Machinery and equipment n.e.c. (**Production index**).                                                           |
| *PRO27840_org*                                  | United States: Electrical equipment (**Production index**).                                                               |
| *PRO271000_org*                                 | World: Electrical equipment (**Production index**).                                                                      |
| *PRO27756_org*                                  | Switzerland: Electrical equipment (**Production index**).                                                                 |
| *PRO27826_org*                                  | United Kingdom: Electrical equipment (**Production index**).                                                             |
| *PRO27380_org*                                  | Italy: Electrical equipment (**Production index**).                                                                      |
| *PRO27392_org*                                  | Japan: Electrical equipment (**Production index**).                                                                      |
| *PRO27250_org*                                  | France: Electrical equipment (**Production index**).                                                                     |
| *PRO27276_org*                                  | Germany: Electrical equipment (**Production index**).                                                                    |

<br>

---

<b style="background-color:#A9A9A9; padding:5px; border-radius:5px; display: inline-block;">Sales Data</b>
| Features                                      | Feature Description |
|-----------------------------------------------|---------------------|
| *DATE*                                       | Day at which a sale was made, in the format *dd-mm-yyyy* |
| *Mapped_GCK*                                 | Product group of the product sold |
| *Sales_EUR*                                  | Monetary amount made from the sale, in Euros  |

<br>

---

In [1]:
# pip install prophet == 1.1.6
# pip install keras == 3.9.0 tensorflow == 2.19.0
# pip install neuralprophet == 0.8.0
# pytorch-forecasting-1.3.0
#pip install nixtla == 0.6.6
#pip install pmdarima == 2.0.4
#pip install scikeras == 0.13.0
#pip install keras-tcn == 3.1.2

In [2]:
import os
import pickle
import pandas as pd
import numpy as np

import seaborn as sns

from itertools import combinations
from sklearn.model_selection import PredefinedSplit, GridSearchCV, TimeSeriesSplit
from joblib import Parallel, delayed
from sklearn.preprocessing import StandardScaler


#timesgpt
from nixtla import NixtlaClient
#xgboost
import xgboost as xgb
from xgboost.callback import EarlyStopping
from xgboost import XGBRegressor
#tcn
from scikeras.wrappers import KerasRegressor
from tcn import TCN
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Sequential
#arima
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX
import warnings
#NeuralProphet
from prophet import Prophet
from neuralprophet import NeuralProphet

#evaluation
from sklearn.metrics import (
    mean_squared_error, 
    r2_score, 
    mean_absolute_percentage_error,
    mean_absolute_error,
    root_mean_squared_error)

from scipy.stats import shapiro
from statsmodels.tsa.stattools import adfuller



ERROR:NP.plotly:Importing plotly failed. Interactive plots will not work.
ERROR:NP.plotly:Importing plotly failed. Interactive plots will not work.


In [3]:
from plotly.subplots import make_subplots
from statsmodels.tsa.stattools import acf, pacf
import plotly.graph_objects as go

In [4]:
# Set pandas display options
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', 1000)       # Set display width
pd.set_option('display.max_colwidth', 100) # Show full feature lists
pd.set_option('display.float_format', '{:.4f}'.format)  # 4 decimal places

# If you want to force standard notation (no scientific):
pd.set_option('display.float_format', lambda x: '%.4f' % x if abs(x) > 1e-4 else '%.4e' % x)

In [5]:
from IPython.display import display, HTML

def display_excel_sheets_side_by_side(excel_path, tablespacing=5, max_height="500px"):
    """
    Displays all sheets from an Excel file as tables in a scrollable horizontal div.
    
    Args:
        excel_path (str): Path to the Excel file.
        tablespacing (int): Spacing between tables (in pixels).
        max_height (str): Maximum height of the table container (e.g., "500px").
    """
    # Read all sheets from the Excel file
    sheets_dict = pd.read_excel(excel_path, sheet_name=None)
    
    # Generate HTML for each sheet
    html = """
    <div style='overflow-x: auto; white-space: nowrap; padding-bottom: 10px; border-bottom: 2px solid #ddd;'>
    """
    
    for sheet_name, df in sheets_dict.items():
        # Convert DataFrame to HTML and style it
        table_html = df.to_html(
            index=False, 
            classes='dataframe', 
            border=0,
            max_rows=10  # Limit rows for display (optional)
        )
        
        # Wrap each table in a scrollable div
        html += f"""
        <div style='display:inline-block; margin-right:{tablespacing * 5}px; vertical-align:top;'>
            <h4 style='text-align:center;'>{sheet_name}</h4>
            <div style='max-height:{max_height}; overflow-y:auto; border:1px solid #ccc; padding:5px;'>
                {table_html}
            </div>
        </div>
        """
    
    html += "</div>"
    display(HTML(html))

### <span style="background-color:#000027; padding:5px; border-radius:5px;"> 📌 Business Understanding <a id='business-understanding'></a>

##### Click [here](#table-of-contents) ⬆️ to return to the Index.
---

The **Business Understanding** phase of the project entails the comprehension of the background leading to the project, as well as the business goals and requirements to be achieved. 
<br><br><br>
Siemens is a global technology company that provides multi-industry solutions, with a strong focus on automation, digitalization, and sustainability. 

**Smart Infrastructure** division at Siemens

(https://www.siemens.com/global/en/company/about/businesses/smart-infrastructure.html
https://www.siemens.com/global/en/company/about/history/company/2007-2018.html)

In October 2018, Siemens launched its Smart Infrastructure division, as part of the implementation of an optimized corporate structure. This new division emerged from the merger of former divisions, specifically the power distribution segment of the Energy Management division, the Building Technologies’ business, and the control products business of the Digital Factory division.
Smart Infrastructure helps customers through their digital transformation journey by leveraging cutting-edge technology that enables clients to adapt to evolving environments and to seize new opportunities. It mostly provides B2B services, focusing on energy efficiency and resource optimization, which in turn improves clients’ asset performance, availability, reliability, and operational efficiency. All in all, this division future-proofs infrastructures, fostering collaborative ecosystems.

<br><br>
**Economic Environment 2018**

(Siemens. (2018). Annual report 2018. Siemens AG. https://www.siemens.com/global/en/company/investor-relations/events-publications-ad-hoc/annualreports.html)

`Overall economic conditions`
In 2018, advanced countries saw GDP growth of 2.3%, mainly due to the U.S. economy benefiting from tax cuts. In contrast, emerging countries' GDP growth fell slightly from 4.7% to 4.6%, affected by capital outflows and currency depreciation, notably in Argentina and Turkey. This decline was worsened by U.S. monetary tightening and fears of a global trade war from new tariffs. Political tensions increased in the Middle East, particularly with U.S. sanctions on Iran. In Europe, uncertainties around the U.K.'s exit from the EU and worries about Italy’s budget led to negative investment sentiment. All in all, the economic acceleration at the beginning of 2018 lost steam towards the end of the year due to these adverse effects.

`Energy Management`
The Energy Management Division offers a wide spectrum of software, products, systems, solutions, and services for transmitting, distributing and managing electrical power and for providing intelligent power infrastructure.
This sector faces competition from large multinational companies and fast-growing firms in emerging countries. It generally benefits from major trends, in particular decarbonization, decentralization and digitalization. Although Energy Management reported a double-digit order decline, due to a sharply lower volume of large orders in the transmission solutions business, revenue grew in most Division’s businesses, led by the power distribution businesses. For the latter, on a geographic basis, higher revenue in the region Asia, Australia was partially offset by a decline in the Americas strongly influenced by currency headwinds, while revenue in the region Europe, C. I. S., Africa, Middle East was flat year-over-year.
Under the new organizational structure, transmission solutions will be assigned to the Operating Company Gas and Power, while the businesses for power distribution will be assigned to the Operating Company Smart Infrastructure.


`Building Technologies`
The Building Technologies Division is a leading provider of automation technologies and digital services for safe, secure and efficient buildings and infrastructures. The Division offers products, solutions, services and software for fire safety, security, building automation and energy management.
The Division competes with multinational companies, system integrators, local firms, and IT startups. Strong competition is causing price pressure, especially in the solutions business. Economic changes affect the Division's activities with delay, and particularly in non-residential construction markets, there is a time lag of two to four quarters. Key trends include the demand for energy efficiency and smart space solutions to improve comfort and productivity. In fiscal 2018, the Building Technologies acquired three startups to strengthen its portfolio and expertise. Revenue increased, with the Division’s revenue growing faster than the market on increases in all reporting regions, with strongest market growth contributions came from the Asia, Australia region, particularly including India. Orders grew in the region Europe, C. I. S., Africa, Middle East, driven by double-digit growth in Germany including multiple multi-year service contracts. The Division won major orders in the U. S. in both periods under review. Profit and profitability remained strong, supported by economies of scale and improved productivity. The markets are expected to grow solidly in fiscal 2019, although U.S. growth may slow slightly.


`Digital Factory`
The Digital Factory Division provides a wide range of automation products and solutions for manufacturing, including software, control systems, motors, and integrated systems. It also offers lifecycle services and software for design and testing. Changes in customer demand are strongly driven by macroeconomic cycles, leading to variations in profitability. Competition is mainly from large multinational firms offering a relatively broad portfolio, and companies that are only active in certain geographic or product markets. Key trends include rising interest rates, which will limit investment but at the same time make it more focused on improving efficiency, a shift towards regionalization solutions, either to protect local economies or to better adapt solutions to local needs, and an increased focus on digitalization for competitiveness. The Division reported year-over-year growth in orders and revenue, especially in software. Growth was strongest in the Americas and Asia, notably China, and customer investments grew faster than their production, underlining a clear growth momentum relating to automation and digitalization. While the market is expected to continue growing, slower demand is anticipated for 2019 due to the high investment levels of 2018 and rising global trade tensions.

<br><br>
**Economic Environment 2019**

(Siemens. (2019). Annual report 2019. Siemens AG. https://www.siemens.com/global/en/company/investor-relations/events-publications-ad-hoc/annualreports.html)

`Overall economic conditions`
After GDP growth of 3.2% in 2018, it is expected to drop to 2.6% in 2019, as to the U.S.-China trade dispute and geopolitical tensions in the Middle East contributed to uncertainty, which weighed on capital investments. Global exchange of goods started to decline from October 2018, leading to near-stagnation of industrial production afterwards. The decline affected regions reliant on trade and industry, particularly Germany and Italy in Europe, Mexico and Canada in the Americas, and India in Asia. In addition, European economies suffered from continued uncertainties regarding the U. K. leaving the European Union (Brexit) and the budget clash between the European Commission and Italy’s government, both weighing especially on business investment environment. Countries dependent on commodities and raw material exporting, notably Chile, Brazil and Argentina, saw declines in commodity prices in addition to other adverse factors including domestic political and financial instability. Advanced countries are expected to grow by 1.6%, 0.7 percentage points lower than 2018, while emerging countries will see a decline from 4.6% to 4.1%.

`Smart Infrastructure`
Smart Infrastructure brings together energy supply – from intelligent control across the grid and low- and medium-voltage electrification and control products – with building technology, targeting high-growth areas like energy storage, electric vehicle infrastructure, and microgrids at the grid edge. It reaches customers through multiple channels, including global sales, distributors, OEMs, resellers, installers, and direct sales via regional branch offices. Its diverse customer base encompasses infrastructure developers, construction firms, public and commercial buildings, utilities, and heavy industries. Smart Infrastructure faces competition from large multinationals, smaller manufacturers, local integrators, and facility management firms. Economic changes impact customer demand differently, with discrete manufacturing reacting quickly and strongly with macroeconomic cycles, while other sectors - infrastructure, construction, heavy industries and the utilities – have a slower response to economic cycles. Key trends include rising population and urbanization, the need for safe and sustainable environments, and decarbonization, growing decentralization and prosumers’ influence. Orders and revenue increased across all business areas and regions in fiscal 2019.  

<br><br>
**Economic Environment 2020**

(Siemens. (2020). Annual report 2020. Siemens AG. https://www.siemens.com/global/en/company/investor-relations/events-publications-ad-hoc/annualreports.html)

`Overall economic conditions`
The fiscal year 2020 was heavily affected by the coronavirus pandemic and subsequent recession, the deepest since World War II. Global GDP is expected to fall by 4.5% in 2020, following a growth of 2.6% in 2019. Economic activity was already decelerating due to the U.S.-China trade conflict, but economic sentiment indicators were starting to improve in response to calming of the conflict ( “phase one deal”). Then COVID-19 emerged and started to spread globally, resulting in strict social distancing measures that limited operations, especially in contact-heavy sectors. Many other industries were directly affected by supply chain problems or indirectly by insufficient demand and stopped production.

Governments and central banks responded with strong fiscal and monetary policies to support businesses and households, leading to a brief economic rebound in the summer. However, renewed virus outbreaks hindered full recovery and restrictions in high contact industries made the economy operate only at about 90% of its full potential, with advanced economies projected to see a 5.5% decline in GDP. Siemens experienced mixed impacts across its sectors, benefiting from growth in technology-related areas, while facing challenges in customer access and sales. While the pandemic significantly slowed our sales and service activities, this also resulted in cost reductions such as lower travel and marketing expenses. Furthermore, we were able to keep our production largely stable due to the use of our own technology in our factories and our diversified value chain.


`Smart Infrastructure`
Smart Infrastructure’s businesses are impacted by changes in the overall economic environment to varying degrees, depending on customer segment. Particularly in its solutions and service business, it is affected by changes in the non-residential building construction markets with a time lag of two to four quarters.
Smart Infrastructure is experiencing a decline in orders, particularly in the solutions and services business, which previously saw a higher volume of orders. The products business showed only a slight decrease, while orders in the systems and software business stayed stable. Overall, revenue for the solutions and services business and for the systems and software business remained close to the prior- year levels. The drop in orders and revenue primarily came from Europe, Africa, the Middle East, and parts of Asia and Australia, but the Americas remained stable.
Demand in Smart Infrastructure markets declined moderately due to COVID-19, especially in the automotive, oil and gas, and machine-building sectors. However, energy performance services and the data center market grew due to the increased focus on energy efficiency, digital services and remote work. 
Market development will be affected by lower demand from building construction, leading to a slight decline in volume of markets served by Smart Infrastructure year-over-year. Asia and Australia are expected return to normal growth, while Europe and the U.S. may see declines.

<br><br>
**Economic Environment 2021**

(Siemens. (2021). Annual report 2021. Siemens AG. https://www.siemens.com/global/en/company/investor-relations/events-publications-ad-hoc/annualreports.html)

`Overall economic conditions`
In fiscal 2021, the global economy was heavily influenced by the COVID-19 pandemic and its many repercussions. After a GDP contraction in 2020, a strong recovery is expected in 2021. The economy grew rapidly in the third quarter of 2020 after the first wave of COVID-19. Despite fears of a new recession due to subsequent COVID-19 waves, economic activity had already adapted, supported by significant stimulus measures in Europe and the U.S.
Central banks gave support with expansionary measures, while short-term interest rates were at or near zero. Accordingly, the global economy continued to expand also in the fourth quarter of calendar 2020 and the first quarter of calendar 2021, despite renewed outbreaks and lockdowns. In December 2020, the first countries approved new COVID-19 vaccines.
However, momentum weakened in early 2021 due to rising infections and the more contagious Delta variant. Vaccination efforts lagged, especially in emerging countries, while supply disruptions affected many sectors, leading to logistics bottlenecks that disrupted value chains from raw materials to high-tech goods, especially semiconductors, and caused extraordinary disruptions in global logistics systems. High demand for goods, driven by consumer savings, contributed to increased producer and energy prices,  resulting in elevated rates of inflation. 
The Chinese economy – with the world’s largest manufacturing sector –  particularly benefited from global demand. However, tensions in the property sector and energy shortages weighed on economic activity in the second half of calendar 2021. 
Overall, major economies experienced strong rebounds, and GDP is expected to grow strongly.

`Smart Infrastructure`
Smart Infrastructure is benefitting from several positive trends, including urbanization, demographic changes, climate change, and digitalization. Urbanization and demographic shifts create a demand for smarter buildings, while climate change pushes for decarbonization and the development of flexible energy infrastructures. Digitalization enables the improvement of building efficiency and electricity management with more renewable energy sources.
Orders for Smart Infrastructure improved across all sectors, with notable growth in the products and systems businesses, driven by demand from industrial customers and contracts from U.S. semiconductor manufacturers. Revenue also grew, although the solutions and services business saw a slight decline due to negative currency translation effects. Meanwhile, supply chain challenges were managed effectively, preventing major disruptions.
Overall, markets served by Smart Infrastructure grew moderately in fiscal 2021, experiencing a recovery from COVID-19-related effects that had a strong impact on most customer industries a year earlier.
Smart Infrastructure also experienced a number of supply chain constraints, especially in the areas of base metals (copper, aluminum, steel), plastics, semiconductors and transportation services. Whereas the management of these constraints required additional effort, Smart Infrastructure’s supply chains have proven to be resilient, so that major interruptions could be avoided, and delivery ability was maintained.
Market growth is expected to accelerate, primarily fueled by demand in the pharmaceutical sector, data centers, and utilities. Growth will be strongest in Asia and Australia. Growth in the region Europe, C.I.S., Africa, and the Middle East is expected to accelerate and markets in the region Americas are expected to return to growth.

<br><br>
**Economic Environment 2022**

(Siemens. (2022). Annual report 2022. Siemens AG. https://www.siemens.com/global/en/company/investor-relations/events-publications-ad-hoc/annualreports.html)

`Overall economic conditions`
In fiscal 2022, global economic growth faced major disruptions from the war in Ukraine, the effects of COVID-19, and the economic slowdown in China. After a strong recovery in 2021, GDP growth is expected to decline as the economic rebound ended abruptly. In late 2021, increased vaccinations and reduced restrictions boosted consumer spending, triggering inflationary pressures, especially in the U.S. and Europe. Supply chain issues, labor shortages, and heightened energy prices contributed to this inflation, while large stimulus packages and high household savings fueled it. 
The war in Ukraine further strained economies in early 2022, causing another significant rise in energy prices, which heavily impacted Europe and industrial sectors, especially energy-intensive industries such as chemicals. The war in Ukraine put further pressure on developing economies, especially in the Middle East, Africa, and Turkey, as both Russia and Ukraine were major exporters of grain and fertilizer before the war.
China’s zero-COVID strategy became even more strict with the emergence of the Delta variant and the highly infectious Omicron variant, which hindered economic activity, along with regulations affecting growing sectors and a recession in real estate. Consequently, China's growth was predicted to slow. 
In response to rising inflation, the U.S. Federal Reserve raised interest rates and tightened monetary policy, followed by other central banks. This led to economic disruptions, resulting in slower-than-expected GDP growth for 2022.

`Smart Infrastructure`
In fiscal 2022, Smart Infrastructure acquired Brightly Software Inc., improving its position in software for asset and maintenance management. Smart Infrastructure’s businesses are impacted by changes in the overall economic environment to varying degrees. Especially, demand for service offerings shows only limited influence from macroeconomic cycles.
Orders and revenue grew significantly across all businesses, especially in electrical products and electrification, driven by industrial customers, data centers, and proactive purchasing. Smart Infrastructure faced supply chain challenges but avoided major disruptions. The Americas region saw the strongest growth, while Asia and Australia were hindered by COVID-19 impacts, particularly in China. Both order and revenue development included positive currency translation effects. 
Overall, markets served by Smart Infrastructure grew clearly in fiscal 2022. Market growth is anticipated to be slightly slower due to ongoing inflation and supply chain issues, with solid demand expected in data centers and power distribution despite challenges like price increases and geopolitical tensions.
Market development is expected to continue to be influenced by supply chain constraints and effects from the war in Ukraine, including on energy prices. Further impacts could arise from potential lockdown measures in China and geopolitical tensions.

### <span style="background-color:#000027; padding:5px; border-radius:5px;"> 📌 Modeling <a id='modeling'></a>

[1. Data Loading](#data-loading-and-description)

[2. Usefull Functions](#functions)

[3. Model Selection per Product](#model-selection)
- [3.1. TimeGPT](#timegpt)
- [3.2. XGBoost](#xgboost)
- [3.3. TCN](#tcn) 
- [3.4. Prophet](#prophet) 
- [3.5. ARIMA](#arima)
- [3.6. Results Comparison](#compare) 

[4. Exogenous Variables Prediction](#market)

[5. Final Modeling](#final)


Click [here](#table-of-contents) ⬆️ to return to the Index.

---

#### <span style="background-color:#000027; padding:5px; border-radius:5px;">**1. Data Loading**</span> <a id='data-loading-and-description'></a>  

Click [here](#table-of-contents) ⬆️ to return to the Index.


In [6]:
def load_dfs_from_folder(folder_path):
    """Loads DataFrames from files in a specified folder and returns a dictionary."""
    dfs = {}
    # List all files in the folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith(".pkl"):
            key = file_name.replace(".pkl", "")  # Extract key from the file name
            file_path = os.path.join(folder_path, file_name)
            
            # Load the dataframe from the pickle file
            with open(file_path, 'rb') as f:
                dfs[key] = pickle.load(f)
            print(f"Loaded {key} from {file_path}")
    
    return dfs

# Load both product_dfs and lagged_product_dfs from their respective folders
product_dfs = load_dfs_from_folder("product_dfs_folder")
lagged_product_dfs = load_dfs_from_folder("lagged_product_dfs_folder")

Loaded P1 from product_dfs_folder\P1.pkl
Loaded P11 from product_dfs_folder\P11.pkl
Loaded P12 from product_dfs_folder\P12.pkl
Loaded P13 from product_dfs_folder\P13.pkl
Loaded P14 from product_dfs_folder\P14.pkl
Loaded P16 from product_dfs_folder\P16.pkl
Loaded P20 from product_dfs_folder\P20.pkl
Loaded P3 from product_dfs_folder\P3.pkl
Loaded P36 from product_dfs_folder\P36.pkl
Loaded P4 from product_dfs_folder\P4.pkl
Loaded P5 from product_dfs_folder\P5.pkl
Loaded P6 from product_dfs_folder\P6.pkl
Loaded P8 from product_dfs_folder\P8.pkl
Loaded P9 from product_dfs_folder\P9.pkl
Loaded Sales_CPI from product_dfs_folder\Sales_CPI.pkl
Loaded P1 from lagged_product_dfs_folder\P1.pkl
Loaded P11 from lagged_product_dfs_folder\P11.pkl
Loaded P12 from lagged_product_dfs_folder\P12.pkl
Loaded P13 from lagged_product_dfs_folder\P13.pkl
Loaded P16 from lagged_product_dfs_folder\P16.pkl
Loaded P20 from lagged_product_dfs_folder\P20.pkl
Loaded P3 from lagged_product_dfs_folder\P3.pkl
Loaded P36 

In [7]:
for product_id in product_dfs.keys():
    product_dfs[product_id] = product_dfs[product_id].rename(columns={product_id: "Sales"})

for product_id in lagged_product_dfs.keys():
    lagged_product_dfs[product_id] = lagged_product_dfs[product_id].rename(columns={product_id: "Sales"})

In [8]:
product_dfs['P1'].head()

,Sales,MAB_ELE_SHP840,PRI27276_org,PRO27826_org,MAB_ELE_PRO276,MAB_ELE_SHP1100
month_year,,,,,,
2018-10-01,35774028.5209,127.8088,109.1196,118.6708,124.2279,130.9893
2018-11-01,5063648.6000,117.6759,109.2248,120.4670,127.4041,132.9341
2018-12-01,37321267.9382,123.2801,109.3301,105.3787,120.5186,131.2613
2019-01-01,27090400.9380,111.0438,109.7510,107.1749,104.7763,113.0576
2019-02-01,34132093.4229,116.7369,109.8562,110.6476,109.5970,117.7047


In [9]:
lagged_product_dfs['P8'].head()

,Sales,PRI27840_org,RohCOPPER1000_org,time_idx,year,month,P8_lag_1,P8_lag_2,P8_lag_5,P8_lag_6,P8_lag_10,P8_ma_1,P8_ma_2,P8_ma_5,P8_ma_6,P8_ma_10
month_year,,,,,,,,,,,,,,,,
2019-08-01,398332.2744,110.6561,75.7745,10,2019,8,368081.2008,420287.0223,582419.0346,361474.5342,580778.2653,398332.2744,383206.7376,380139.7221,413852.9409,400303.6854
2019-09-01,1086855.2918,111.0503,76.4355,11,2019,9,398332.2744,368081.2008,416386.5006,582419.0346,518398.3785,1086855.2918,742593.7831,514233.4804,497925.6504,457149.3767
2019-10-01,112014.9227,111.0322,76.4097,12,2019,10,1086855.2918,398332.2744,297611.6126,416386.5006,267418.3494,112014.9227,599435.1073,477114.1424,447197.0541,441609.0341
2019-11-01,540208.8029,111.0354,77.7720,13,2019,11,112014.9227,1086855.2918,420287.0223,297611.6126,372627.9465,540208.8029,326111.8628,501098.4985,487629.9191,458367.1197
2019-12-01,491332.7200,111.2866,80.6535,14,2019,12,540208.8029,112014.9227,368081.2008,420287.0223,361474.5342,491332.7200,515770.7614,525748.8024,499470.8688,471352.9383


> Importing the test template

In [10]:
test = pd.read_csv('Case2_Test Set Template.csv', sep =';')

#some alterations
month_map = {'Mai': 'May', 'Okt': 'Oct', 'Dez': 'Dec'}
test['Month Year'] = test['Month Year'].replace(month_map, regex=True)
test['Month Year'] = pd.to_datetime(test['Month Year'] + ' 1', format='%b %y %d')
test['Mapped_GCK'] = test['Mapped_GCK'].str.replace('#', '')

test.head()

,Month Year,Mapped_GCK,Sales_EUR
0,2022-05-01,3,NaN
1,2022-06-01,3,NaN
2,2022-07-01,3,NaN
3,2022-08-01,3,NaN
4,2022-09-01,3,NaN


#### <span style="background-color:#000027; padding:5px; border-radius:5px;">**2. Usefull Functions**</span> <a id='functions'></a>  

Click [here](#table-of-contents) ⬆️ to return to the Index.


In [11]:
def winsorize_function(df, cols, lower_quantile=0.01, upper_quantile=0.99):
        """Apply Winsorization only to the product sales column"""
        df = df.copy()
        bounds = {}
        for col in cols:
            q1 = df[col].quantile(lower_quantile)
            q3 = df[col].quantile(upper_quantile)
            df[col] = df[col].clip(lower=q1, upper=q3)
            bounds[col] = (q1, q3)  # Store actual percentile values
            print(f"{col}: Winsorized Bounds -> Lower = {q1:.2f}, Upper = {q3:.2f}")

        return df, bounds

def preprocessing_pipeline(train, val=None, test=None, outlier_treatment=True):

    def process_dataset(df, cols, is_train=True, bounds=None):
        df = df.copy() 
        if is_train:
            if outlier_treatment:
                df, bounds = winsorize_function(df, cols)
            else:
                bounds = {}  
        else:
            if outlier_treatment:
                if bounds is None:
                    raise ValueError("Bounds must be provided for validation and test datasets.")
                for col in cols: 
                    if col in bounds:
                        lower, upper = bounds[col]
                        df[col] = df[col].clip(lower, upper)  # Corrected clipping
        return (df, bounds) if is_train else df

    # Process the training dataset
    train, bounds = process_dataset(train, cols = train.columns, is_train=True)

    # Process validation and test datasets with correct bounds
    if val is not None:
        val = process_dataset(val, cols = val.columns, is_train=False, bounds=bounds)

    if test is not None:
        test = process_dataset(test, cols = test.columns, is_train=False, bounds=bounds)
        
    # Return the datasets 
    if test is not None and val is not None:
        return train, val, test
    elif val is not None:
        return train, val
    elif test is not None:
        return train, test
    else:
        return train

def time_series_train_test_split(X, y, test_size=10):
    """Split time series data maintaining temporal order"""
    split_idx = len(X) - test_size
    return (
        X.iloc[:split_idx], X.iloc[split_idx:],
        y.iloc[:split_idx], y.iloc[split_idx:])

def prepare_time_series_data(df_train, feature_set, target_col="Sales", horizon=10, winsorize=False, scaling=False):

    # Extract product data
    data = df_train.copy()
    
    y = data[[target_col]]  
    X = data.drop(columns=[target_col])
    
    # Select features - handle empty feature_set case
    if feature_set:
        try:
            # Convert feature_set to list if it's not already
            if not isinstance(feature_set, list):
                feature_set = [feature_set]
            # Select only columns that exist in X
            available_features = [f for f in feature_set if f in X.columns]
            X = X[available_features]
        except KeyError as e:
            raise ValueError(f"Some features not found in DataFrame: {e}")
    
    # Train/Test Split
    X_train, X_val, y_train, y_val = time_series_train_test_split(X, y, test_size=horizon)

    # Apply preprocessing steps
    y_train, y_val = preprocessing_pipeline(y_train, y_val, test=None, outlier_treatment=winsorize) #just for the sales

    # Scaling
    if scaling: 
        scaler_X = StandardScaler()
        X_train_scaled = scaler_X.fit_transform(X_train)
        X_val_scaled = scaler_X.transform(X_val)
        
        scaler_y = StandardScaler()
        y_train_scaled = scaler_y.fit_transform(y_train)
        y_val_scaled = scaler_y.transform(y_val)
        
        # Convert back to DataFrame for easier handling
        X_train_scaled = pd.DataFrame(X_train_scaled, index=X_train.index, columns=X_train.columns)
        X_val_scaled = pd.DataFrame(X_val_scaled, index=X_val.index, columns=X_val.columns)
        y_train_scaled = pd.DataFrame(y_train_scaled, index=y_train.index, columns=y_train.columns)
        y_val_scaled = pd.DataFrame(y_val_scaled, index=y_val.index, columns=y_val.columns)
        
        return X_train_scaled, X_val_scaled, y_train_scaled, y_val_scaled
    else:
        return X_train, X_val, y_train, y_val

In [12]:
def generate_feature_combinations(features, max_features=None):
    """Generate all possible feature combinations"""
    if max_features is None:
        max_features = len(features)
    
    all_combinations = []
    for r in range(2, max_features + 1):
        all_combinations.extend(combinations(features, r))
    
    return [list(comb) for comb in all_combinations]

In [13]:
def convert_results_to_df(results):
    if results is None or len(results) == 0:
        return pd.DataFrame()

    data = []
    for result in results:
        # Assuming each item in the list is a dictionary
        data.append({
            'product_id': result['product_id'],
            'winsorize': result['winsorize'],
            'features': ', '.join(result['features']) if result['features'] else 'all',
            'RMSE': result['metrics']['RMSE'],
            'MAPE': result['metrics']['MAPE'],
            'Overfit_Score': result['metrics'].get('Overfit Score', np.nan),
            'method': result['method']
        })
    
    return pd.DataFrame(data)

In [14]:
def calculate_metrics(y_true, y_pred, y_train=None, y_train_pred=None, print_metrics=False):
    metrics = {
        'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
        'MAPE': mean_absolute_percentage_error(y_true, y_pred) * 100
    }
    
    if y_train is not None and y_train_pred is not None:
        train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
        metrics['Overfit Score'] = (metrics['RMSE'] - train_rmse) / max(train_rmse, 1e-10)
    
    if print_metrics:
        print("\n=== Metrics ===")
        print(f"RMSE: {metrics['RMSE']:.3f}")
        print(f"MAPE: {metrics['MAPE']:.2f}%")
        if 'Overfit Score' in metrics:
            print(f"Overfit Score: {metrics['Overfit Score']:.3f}")
    
    return metrics

def convert_results_to_df(results):
    """Convert results to DataFrame format"""
    if results is None:
        return pd.DataFrame()
    
    return pd.DataFrame([{
        'product_id': results['product_id'],
        'winsorize': results['winsorize'],
        'features': ', '.join(results['features']) if results['features'] else 'all',
        'RMSE': results['metrics']['RMSE'],
        'MAPE': results['metrics']['MAPE'],
        'Overfit_Score': results['metrics'].get('Overfit Score', np.nan),
        'method': results['method']
    }])

#### <span style="background-color:#000027; padding:5px; border-radius:5px;">**3. Model Selection per Product**</span> <a id='model-selection'></a>  

Click [here](#table-of-contents) ⬆️ to return to the Index.


➡️ Our pipeline is going to use:

- **Walk-Forward Validation:** One-step forecasting where the model predicts the next step, and the validation window moves forward one step at a time.
- In the cases we needed to use **GridSearch**, we use a PredefinedSplit method to ensure no cross validation and prevent messing up with the train/validation timeline.

➡️ For Feature Combinations we will iterate over:

- No macro/lags features (baseline model).
- Single-feature models.
- Multi-feature models.

`IMPORTANT:` For feature selection, we must be extremely careful because our dataset is limited—we only have 43 monthly data points per product. This small sample size restricts our modeling options. For initial validation, we’ll use a 7-month forecast horizon because TimeGPT requires at least 36 data points to perform well in a walk-forward validation setup. To ensure fair comparisons across all models, we’ll maintain this 7-month horizon, even though our final goal is to predict 10 months ahead. Given the scarcity of data, we must also limit the number of features to avoid overfitting. Including too many variables could cause the model to capture noise rather than true patterns, reducing its accuracy on unseen data.

➡️ Models

We will tune hyperparameters for all the models for each product, with the best features.


**Time Series Model Comparison Table**

| Feature               | TimeGPT (Zero-Shot) | XGBoost            | TCN                | ARIMA              | Prophet (Facebook)     |
|-----------------------|---------------------|--------------------|--------------------|--------------------|--------------------|
| **Scaling Needed?**   | ❌ No               | ❌ No              | ✅ Yes             | ❌ No (if stationary) | ❌ No (logistic/linear trends handle scaling internally)            |
| **Handles Trends**    | ✅ Excellent        | ✅ (With features) | ✅ Excellent       | ✅ (With differencing) | ✅ Piecewise linear/logistic     |
| **Handles Seasonality** | ✅ Automatic       | ❌ (Requires manual lags) | ✅ (With large receptive field) | ✅ (SARIMA) | ✅ Fourier terms (fixed but interpretable)      |
| **Multivariate Support** | ✅ Yes            | ✅ Yes             | ✅ Yes             | ❌ Univariate only | Limited            |
| **Training Speed**    | ⚡ Instant (pre-trained) | 🏎️ Fast          | 🐢 Slow (GPU helps) | 🏃 Moderate        |  🏎️ Fast          |
| **Interpretability**  | ❌ Black box        | ✅ Feature importances | ❌ Black box     | ✅ Model coefficients | ✅ Best-in-class (clear trend/seasonality plots) |
| **Best For**          | Zero-shot forecasting | Tabular data with temporal features | Long-range dependencies | Simple univariate series | Business-friendly forecasts with explainability |
| **Weakness**          | Limited control     | Poor with long sequences | Computationally heavy | Linear assumptions only | Struggles with short-term dependencies (no AR terms)     |


1. **TimeGPT** (by Nixtla):
   - Pros: No training needed, handles any frequency, great for quick benchmarks.
   - Cons: Proprietary, limited customization.

2. **XGBoost**:
   - Best when you have:
     - Easy and fast. 

3. **TCN** (Temporal Convolutional Networks):
   - Pros: Captures long-range patterns better than LSTMs.
   - Cons: Needs careful tuning of and is more computationaly expensive.

4. **ARIMA**:
   - Important:
     - Data is stationary (or differenced) 
     - Short-term forecasts (≤12 steps)
   - Warning: Fails miserably with:
     - Non-linear trends
     - High-frequency data, wich is ok, bc we are dealing with months.

5. **Prophet**:
   - Faster and more interpretable than NeuralProphet.
   - No auto-regression (unlike NeuralProphet’s AR-Net).
   -Less flexible seasonality (fixed Fourier terms vs. NN-backed).


➡️ Summary:

- **We will Scale**: TCN, Neural Prophet
- **Optional Scaling**: XGBoost (for extreme values)
- **We will not Scale**: ARIMA (but difference if non-stationary!), TimeGPT

1. We will start with **TimeGPT** for a baseline.
2. Then test **XGBoost** with lag features.
3. Then **TCN** to see he effect for more complex patterns.
4. **Prophet** and **ARIMA** for interpretability.

➡️ Evaluation Metrics: 
- RMSE → Measures error magnitude.
- Overfit Score →  

$$
\text{Overfit Score} = \frac{\text{Test RMSE} - \text{Train RMSE}}{\text{Train RMSE}}
$$



    - < 0.1 (Underfit)
    - 0.1 - 0.5 (Good Fit)
    - > 0.5 (Overfit Warning!)

##### <span style="background-color:#000027; padding:5px; border-radius:5px;">**3.1. TimeGPT**</span> <a id='timegpt'></a>  

Click [here](#table-of-contents) ⬆️ to return to the Index.


In [15]:
nixtla_client = NixtlaClient(
    api_key = "nixak-CIwSKQ0cRLIuFR1TYllLFVakTGx3WCY30YPEKfxG0lDQcE0akGo3GE4aMJO9XXbkKjdFaGDP5x6uSxQ6"
)
nixtla_client.validate_api_key()

INFO:nixtla.nixtla_client:Happy Forecasting! :)


True

In [16]:
def choose_parameters_timegpt(product_id, df_train, feature_set, target_col="Sales", 
                              nixtla_client=None, winsorize=False, 
                              horizon=7):
    
    # --- Input Validation ---
    if nixtla_client is None:
        raise ValueError("NixtlaClient instance is required.")
    if len(df_train) < 12:
        print(f"⚠️ Skipping {product_id} - only {len(df_train)} observations")
        return None
    
    # --- Data Preparation ---
    print('Preparing data...')
    try:
        X_train, X_val, y_train, y_val = prepare_time_series_data(df_train, feature_set, target_col, horizon, winsorize)

    except Exception as e:
        print(f"Data preparation failed: {str(e)}")
        return None
 
    # --- Determine Feature Types ---
    hist_exog = [f for f in feature_set if '_lag_' in f or '_ma_' in f] if feature_set else []
    fut_exog = [f for f in feature_set if f not in hist_exog] if feature_set else []


    # --- Prepare Future Exogenous Features ---
    X_future = None
    if fut_exog:
        X_future = X_val[fut_exog].copy()
        if not X_future.isnull().all().all():  
            X_future.insert(0, 'month_year', X_val.index)
        else:
            X_future = None  # Avoid passing empty DataFrame

    # --- Simple Forecast (12 ≤ obs < 36) ---
    if len(X_train) < 36:
        print(f"Using simple forecast ({len(X_train)} < 36 obs)")
        try:
            history = df_train.copy()
            if feature_set:
                history = history[feature_set + [target_col]]

            forecast = nixtla_client.forecast(df=history.reset_index(),
                                              time_col="month_year", target_col=target_col, h=horizon,
                                              X_df=X_future,  # Always pass X_future if available
                                              hist_exog_list=hist_exog if hist_exog else None)
            
            # Get training predictions
            train_pred = nixtla_client.forecast(df=history.reset_index(),time_col="month_year",
                                                target_col=target_col,h=1,hist_exog_list=hist_exog if hist_exog else None
                                                ).iloc[0]["TimeGPT"]
            
            train_predictions = [train_pred] * len(history)
            
            return {
                'product_id': product_id,'winsorize': winsorize,'features': feature_set,
                'metrics': calculate_metrics(y_true=y_val,y_pred=forecast["TimeGPT"],
                                             y_train=history[target_col],y_train_pred=train_predictions),
                'method': 'simple','validation_predictions': forecast["TimeGPT"]}
        except Exception as e:
            print(f"Simple forecast failed: {str(e)}")
            return None


    # --- Walk-Forward (≥36 obs) ---
    print(f"Using walk-forward ({len(X_train)} ≥ 36 obs)")
    predictions = []
    history = X_train.copy()
    history[target_col] = y_train.values
    history = history.reset_index()
    
    # Get training predictions
    try:
        train_fit = nixtla_client.forecast(df=history,time_col='month_year',target_col=target_col,h=1,
                                           hist_exog_list=hist_exog if hist_exog else None)
        train_predictions = [train_fit["TimeGPT"].iloc[0]] * len(y_train)
    except Exception as e:
        print(f"Training pred failed: {str(e)} - using last value")
        train_predictions = [y_train.iloc[-1]] * len(y_train)

    for i in range(horizon):
        try:
            X_future_step = None
            if fut_exog:
                X_future_step = X_val.iloc[[i]][fut_exog].copy()
                if not X_future_step.isnull().all().all():  
                    X_future_step.insert(0, 'month_year', X_val.index[i])
                else:
                    X_future_step = None  

            forecast = nixtla_client.forecast(df=history,time_col='month_year',target_col=target_col,
                                              h=1, X_df=X_future_step if fut_exog else None, # FUTURE features (macro)
                                              hist_exog_list=hist_exog if hist_exog else None)  # HISTORICAL features (lags, moving averages)
            
            pred = forecast["TimeGPT"].iloc[0]
            predictions.append(pred)
            
            new_row = X_val.iloc[i].copy()
            new_row['month_year'] = X_val.index[i]
            new_row[target_col] = pred
            history = pd.concat([history, pd.DataFrame([new_row])])
        
        except Exception as e:
            print(f"Iteration {i+1} failed: {str(e)}")
            raise
    
    metrics = calculate_metrics(y_true=y_val,y_pred=predictions,y_train=y_train,y_train_pred=train_predictions)
    
    return {'product_id': product_id,'winsorize': winsorize,'features': feature_set,'metrics': metrics,
            'method': 'walkforward','validation_predictions': predictions}


def find_best_timegpt_config(product_id, df_train, nixtla_client, lagged_df=None, target_col="Sales"):

    # Validate input data
    if not isinstance(df_train.index, pd.DatetimeIndex):
        raise TypeError("❌ df_train index is not a DatetimeIndex!")
    if df_train.index.isnull().any():
        raise ValueError("❌ Some index values could not be parsed into datetime!")
    
    results = []
    data = df_train.copy()
    
    # Feature groups for original data
    normal_features = [col for col in data.columns if col != target_col]
    normal_feature_combinations = generate_feature_combinations(normal_features, max_features=6)  
    normal_feature_combinations = [[]] + normal_feature_combinations
    print(normal_feature_combinations)
    
    # Test original data configurations
    for winsorize in [True, False]:
        for features in normal_feature_combinations:
            try:
                print(f"\nTesting - Winsorize: {winsorize}, Features: {features}")

                result = choose_parameters_timegpt(
                    product_id=product_id,df_train=data, feature_set=features, target_col=target_col,
                    nixtla_client=nixtla_client,winsorize=winsorize)
                
                if result:
                    print(f"✅ Success - RMSE: {result['metrics']['RMSE']:.2f}")
                    results.append(result)
            except Exception as e:
                print(f"⚠️ Configuration failed: {str(e)}")
                continue
    
    # Test lagged data configurations if available
    if lagged_df is not None:
        lagged_data = lagged_df.copy()
        lag_features = [col for col in lagged_data.columns if 'lag' in col]
        ma_features = [col for col in lagged_data.columns if '_ma_' in col]
        macro_features = [col for col in data.columns if col != target_col]
        
        # Generate lag feature combinations
        lag_feature_combinations = generate_feature_combinations(lag_features, max_features=5)
        
        # Create combined feature sets
        full_combinations = []
        for lag_combo in lag_feature_combinations:
            full_combinations.append(lag_combo)  # Just lags
            full_combinations.append(lag_combo + macro_features)  # Lags + macros
            full_combinations.append(lag_combo + ma_features)  # Lags + MAs
            full_combinations.append(lag_combo + macro_features + ma_features)  # All
        
        # Remove duplicates
        seen = set()
        unique_combinations = []
        for combo in full_combinations:
            combo_tuple = tuple(sorted(combo))
            if combo_tuple not in seen:
                seen.add(combo_tuple)
                unique_combinations.append(combo)
        
        # Test all unique combinations
        for winsorize in [True, False]:
            for features in unique_combinations:
                try:
                    print(f"\nTesting - Lagged Features: {features}")

                    result = choose_parameters_timegpt(product_id=product_id,df_train=lagged_data,feature_set=features,
                                                       target_col=target_col,nixtla_client=nixtla_client,winsorize=winsorize)
                    
                    if result:
                        print(f"✅ Success - RMSE: {result['metrics']['RMSE']:.2f}")
                        results.append(result)
                except Exception as e:
                    print(f"⚠️ Lagged configuration failed: {str(e)}")
                    continue
    
    if not results:
        raise ValueError(f"No valid configurations for {product_id}")
    
    # Select best configuration by RMSE
    best_config = min(results, key=lambda x: x['metrics']['RMSE'])
    
    print(f"\n✅ Best configuration for {product_id}:")
    print(f"- Winsorize: {best_config['winsorize']}")
    print(f"- Features: {best_config['features']}")
    print(f"- RMSE: {best_config['metrics']['RMSE']:.2f}")
    print(f"- Method: {best_config.get('method', 'walkforward')}")
    
    return best_config, results

def process_product_parallel(product_id):
    try:
        df_train = product_dfs[product_id].rename(columns={product_id: "Sales"})
        
        # Ensure DatetimeIndex
        if not isinstance(df_train.index, pd.DatetimeIndex):
            df_train.index = pd.to_datetime(df_train.index, errors="coerce")
        
        if df_train.index.isnull().any():
            raise ValueError("❌ Some index values could not be parsed into datetime!")

        lagged_df = lagged_product_dfs.get(product_id, None)

        best_config, results = find_best_timegpt_config(product_id=product_id,df_train=df_train,lagged_df=lagged_df,
            nixtla_client=nixtla_client,target_col="Sales")
        
        return best_config, results
    except Exception as e:
        print(f"⚠️ Failed to process {product_id}: {str(e)}")
        return None

##### <span style="background-color:#000027; padding:5px; border-radius:5px;">**3.2. XGBoost**</span> <a id='xgboost'></a>  


Click [here](#table-of-contents) ⬆️ to return to the Index.


In [17]:
def choose_parameters_xgboost(product_id, df_train, feature_set, target_col="Sales", 
                            param_grid=None, n_jobs=-1, winsorize=False, horizon=7):
    """Optimized XGBoost forecasting with proper walk-forward validation"""
    
    # --- Input Validation ---
    if len(df_train) < 12:
        print(f"⚠️ Skipping {product_id} - only {len(df_train)} observations")
        return None
    
    # --- Data Preparation ---
    print('Preparing data...')
    try:
        X_train, X_val, y_train, y_val = prepare_time_series_data(
            df_train, feature_set, target_col, horizon, winsorize)
        # Verify temporal ordering
        assert X_train.index[-1] < X_val.index[0], "Validation data must be after training data"
    except Exception as e:
        print(f"Data preparation failed: {str(e)}")
        return None

    # --- Optimized Parameter Grid for Small Data ---
    if param_grid is None:
        param_grid = {
            'n_estimators': [50, 100],
            'max_depth': [2, 3],
            'learning_rate': [0.01, 0.05, 0.1],
            'subsample': [0.9, 1.0],
            'colsample_bytree': [0.9, 1.0],
            'min_child_weight': [1, 3]}

    # --- Efficient Model Setup ---
    base_model = xgb.XGBRegressor(
        objective='reg:squarederror',random_state=42,n_jobs=1,tree_method='hist',  # Faster training
        enable_categorical=False,gamma=0,reg_alpha=0,reg_lambda=1)

    # --- Strict Temporal Validation ---
    test_fold = np.array([-1] * len(X_train) + [0] * len(X_val))
    ps = PredefinedSplit(test_fold)

    # --- Focused Grid Search ---
    grid_search = GridSearchCV(
        estimator=base_model,param_grid=param_grid,scoring='neg_root_mean_squared_error',cv=ps,n_jobs=n_jobs,verbose=1,refit=True)

    # --- Training with Validation ---
    X_combined = pd.concat([X_train, X_val])
    y_combined = pd.concat([y_train, y_val])
    
    print(f"Starting grid search for {product_id}...")
    try:
        grid_search.fit(X_combined, y_combined)
        best_model = grid_search.best_estimator_
    except Exception as e:
        print(f"Grid search failed: {str(e)}")
        return None

    # --- Robust Walk-forward Validation ---
    predictions = []
    print(f"Using walk-forward (n_train={len(X_train)}, horizon={horizon})")
    
    for i in range(horizon):
        try:
            # Create expanding window
            window_X = pd.concat([X_train, X_val.iloc[:i]])
            window_y = pd.concat([y_train, y_val.iloc[:i]])
            
            # Correct XGBoost 2.1.3+ fitting
            best_model.fit(window_X, window_y, eval_set=[(X_val.iloc[[i]], y_val.iloc[[i]])], verbose=0)
    
            pred = best_model.predict(X_val.iloc[[i]]).item()
            predictions.append(pred)
            
        except Exception as e:
            print(f"Iteration {i+1} failed: {str(e)} - using fallback")
            fallback = (predictions[-1] if len(predictions) > 0 else 
                       y_train.iloc[-1] if len(y_train) > 0 else np.nan)
            predictions.append(fallback)

    # --- Training Metrics ---
    try:
        train_predictions = best_model.predict(X_train).tolist()
    except Exception as e:
        print(f"Training predictions failed: {str(e)} - using last value")
        train_predictions = [y_train.iloc[-1]] * len(y_train)

    # --- Comprehensive Metrics ---
    metrics = calculate_metrics(y_true=y_val,y_pred=predictions,y_train=y_train,y_train_pred=train_predictions)
    
    return {
        'product_id': product_id,
        'winsorize': winsorize,
        'features': feature_set,
        'metrics': metrics,
        'method': 'walkforward',
        'best_params': grid_search.best_params_,
        'validation_predictions': predictions,
        'feature_importances': dict(zip(feature_set, best_model.feature_importances_))}


def find_best_xgboost_config(product_id, df_train, lagged_df=None, target_col="Sales", 
                            n_jobs=-1, custom_param_grids=None):
    # Validate input data
    if not isinstance(df_train.index, pd.DatetimeIndex):
        raise TypeError("❌ df_train index is not a DatetimeIndex!")
    if df_train.index.isnull().any():
        raise ValueError("❌ Some index values could not be parsed into datetime!")
    
    results = []
    data = df_train.copy()
    
    # Feature groups for original data
    normal_features = [col for col in data.columns if col != target_col]
    normal_feature_combinations = generate_feature_combinations(normal_features, max_features=6)  
    normal_feature_combinations = [[]] + normal_feature_combinations
    
    # Test original data configurations
    for winsorize in [True, False]:
        for features in normal_feature_combinations:
            try:
                print(f"\nTesting - Winsorize: {winsorize}, Features: {features}")

                # Use custom param grid if provided for this feature set
                param_grid = None
                if custom_param_grids and tuple(features) in custom_param_grids:
                    param_grid = custom_param_grids[tuple(features)]

                result = choose_parameters_xgboost(product_id=product_id,df_train=data,feature_set=features,
                    target_col=target_col,winsorize=winsorize,n_jobs=n_jobs,param_grid=param_grid)
                
                if result:
                    print(f"✅ Success - RMSE: {result['metrics']['RMSE']:.2f}")
                    results.append(result)
            except Exception as e:
                print(f"⚠️ Configuration failed: {str(e)}")
                continue
    
    # Test lagged data configurations if available
    if lagged_df is not None:
        lagged_data = lagged_df.copy()
        lag_features = [col for col in lagged_data.columns if 'lag' in col]
        ma_features = [col for col in lagged_data.columns if '_ma_' in col]
        macro_features = [col for col in data.columns if col != target_col]
        
        # Generate lag feature combinations
        lag_feature_combinations = generate_feature_combinations(lag_features, max_features=5)
        
        # Create combined feature sets
        full_combinations = []
        for lag_combo in lag_feature_combinations:
            full_combinations.append(lag_combo)  # Just lags
            full_combinations.append(lag_combo + macro_features)  # Lags + macros
            full_combinations.append(lag_combo + ma_features)  # Lags + MAs
            full_combinations.append(lag_combo + macro_features + ma_features)  # All
        
        # Remove duplicates
        seen = set()
        unique_combinations = []
        for combo in full_combinations:
            combo_tuple = tuple(sorted(combo))
            if combo_tuple not in seen:
                seen.add(combo_tuple)
                unique_combinations.append(combo)
        
        # Test all unique combinations
        for winsorize in [True, False]:
            for features in unique_combinations:
                try:
                    print(f"\nTesting - Lagged Features: {features}")

                    # Use custom param grid if provided for this feature set
                    param_grid = None
                    if custom_param_grids and tuple(features) in custom_param_grids:
                        param_grid = custom_param_grids[tuple(features)]

                    result = choose_parameters_xgboost(product_id=product_id,df_train=lagged_data,feature_set=features,
                                                       target_col=target_col,winsorize=winsorize,n_jobs=n_jobs,param_grid=param_grid)
                    
                    if result:
                        print(f"✅ Success - RMSE: {result['metrics']['RMSE']:.2f}")
                        results.append(result)
                except Exception as e:
                    print(f"⚠️ Lagged configuration failed: {str(e)}")
                    continue
    
    if not results:
        raise ValueError(f"No valid configurations for {product_id}")
    
    # Select best configuration by RMSE
    best_config = min(results, key=lambda x: x['metrics']['RMSE'])
    
    print(f"\n✅ Best configuration for {product_id}:")
    print(f"- Winsorize: {best_config['winsorize']}")
    print(f"- Features: {best_config['features']}")
    print(f"- Best params: {best_config['best_params']}")
    print(f"- RMSE: {best_config['metrics']['RMSE']:.2f}")
    print(f"- Method: {best_config.get('method', 'walkforward')}")
    
    return best_config, results

def process_product_parallel(product_id, custom_param_grids=None):
    try:
        df_train = product_dfs[product_id].rename(columns={product_id: "Sales"})
        
        # Ensure DatetimeIndex
        if not isinstance(df_train.index, pd.DatetimeIndex):
            df_train.index = pd.to_datetime(df_train.index, errors="coerce")
        
        if df_train.index.isnull().any():
            raise ValueError("❌ Some index values could not be parsed into datetime!")

        lagged_df = lagged_product_dfs.get(product_id, None)

        best_config, results = find_best_xgboost_config(product_id=product_id,df_train=df_train,lagged_df=lagged_df,target_col="Sales", custom_param_grids=custom_param_grids)
        
        return best_config, results
    except Exception as e:
        print(f"⚠️ Failed to process {product_id}: {str(e)}")
        return None

##### <span style="background-color:#000027; padding:5px; border-radius:5px;">**3.3. TCN**</span> <a id='tcn'></a>  

Click [here](#table-of-contents) ⬆️ to return to the Index.


In [18]:
def choose_parameters_tcn(product_id, df_train, feature_set, target_col="Sales", 
                         param_grid=None, n_jobs=-1, winsorize=False, horizon=7):
    """Optimized TCN forecasting with walk-forward validation"""
    
    if len(df_train) < 12:
        print(f"⚠️ Skipping {product_id} - only {len(df_train)} observations")
        return None
    
    print('Preparing data...')
    try:
        X_train, X_val, y_train, y_val = prepare_time_series_data(
            df_train, feature_set, target_col, horizon, winsorize, scaling=True)
        
        assert X_train.index[-1] < X_val.index[0], "Validation data must be after training data"
    except Exception as e:
        print(f"Data preparation failed: {str(e)}")
        return None

    # Reshape data for TCN input (samples, timesteps, features)
    X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
    X_val = X_val.values.reshape((X_val.shape[0], 1, X_val.shape[1]))
    y_train = y_train.values.reshape(-1, 1)
    y_val = y_val.values.reshape(-1, 1)

    if param_grid is None:
        param_grid = {
            'nb_filters': [16, 32],
            'kernel_size': [2, 3],
            'dilations': [[1, 2, 4], [1, 2, 4, 8]],
            'dropout_rate': [0.1, 0.2],
            'return_sequences': [False]  # single-step prediction
}
    
    def build_tcn(nb_filters=32, kernel_size=2, dilations=[1, 2, 4], 
                 dropout_rate=0.1, return_sequences=False):
        inputs = Input(shape=(1, X_train.shape[2]))
        x = TCN(nb_filters=nb_filters, kernel_size=kernel_size, 
               dilations=dilations, dropout_rate=dropout_rate,
               return_sequences=return_sequences)(inputs)
        outputs = Dense(1)(x)
        model = Model(inputs, outputs)
        model.compile(optimizer="adam", loss="mse")
        return model
    
    best_params = None
    best_rmse = float('inf')

    for filters in param_grid['nb_filters']:
        for kernel in param_grid['kernel_size']:
            for dilations in param_grid['dilations']:
                for dropout in param_grid['dropout_rate']:
                    for return_seq in param_grid['return_sequences']:
                        model = build_tcn(filters, kernel, dilations, dropout, return_seq)
                        try:
                            model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=0)
                            
                            # Walk-forward validation (predict one step at a time)
                            predictions = []
                            history_X = X_train.copy()
                            history_y = y_train.copy()
                            
                            for i in range(len(X_val)):
                                # Predict next step
                                X_next = X_val[i].reshape(1, 1, -1)
                                pred = model.predict(X_next, verbose=0).flatten()[0]
                                predictions.append(pred)
                                
                            # Calculate RMSE
                            if len(y_val) == len(predictions):
                                rmse = np.sqrt(np.mean((y_val.flatten() - predictions) ** 2))
                            else:
                                print(f"Shape mismatch: y_val ({y_val.shape}), preds ({len(predictions)})")
                                continue
                            
                            if rmse < best_rmse:
                                best_rmse = rmse
                                best_params = {
                                    'nb_filters': filters,
                                    'kernel_size': kernel,
                                    'dilations': dilations,
                                    'dropout_rate': dropout,
                                    'return_sequences': return_seq
                                }
                        except Exception as e:
                            print(f"Hyperparameter set failed: {str(e)}")

    if best_params is None:
        print(f"❌ No valid models for {product_id}")
        return None

    # Final model training and walk-forward prediction
    final_model = build_tcn(**best_params)
    final_model.fit(X_train, y_train, epochs=50, batch_size=16, verbose=0)
    predictions = []

    # FIXED: Create DataFrame with both features and target
    history_data = np.column_stack([X_train[:, 0, :], y_train.flatten()])
    history = pd.DataFrame(history_data, columns=feature_set + [target_col])
    history.index = range(len(history))
    
    for i in range(horizon):
        try:
            X_future_step = X_val[i].reshape(1, 1, -1)
            pred = final_model.predict(X_future_step, verbose=0).flatten()[0]
            predictions.append(pred)
            
            # Update training data
            new_row = X_val[i, 0, :].tolist()
            new_row.append(pred)
            new_df = pd.DataFrame([new_row], columns=feature_set + [target_col])
            history = pd.concat([history, new_df]).reset_index(drop=True)
            
            # Retrain on expanded window
            rolling_window = 36 if len(history) > 36 else len(history)
            train_subset = history.iloc[-rolling_window:]
            
            X_train_update = train_subset[feature_set].values.reshape(-1, 1, len(feature_set))
            y_train_update = train_subset[target_col].values.reshape(-1, 1)
            
            final_model.fit(X_train_update, y_train_update, epochs=5, batch_size=16, verbose=0)
        
        except Exception as e:
            print(f"Iteration {i+1} failed: {str(e)}")
            predictions.append(predictions[-1] if predictions else y_train[-1][0])
    
    metrics = calculate_metrics(y_true=y_val.flatten(), y_pred=predictions, y_train=y_train.flatten())
    
    return {
        'product_id': product_id,
        'winsorize': winsorize,
        'features': feature_set,
        'metrics': metrics,
        'method': 'walkforward',
        'best_params': best_params,
        'validation_predictions': predictions
    }

def find_best_tcn_config(product_id, df_train, lagged_df=None, target_col="Sales", 
                            n_jobs=-1, custom_param_grids=None):
    # Validate input data
    if not isinstance(df_train.index, pd.DatetimeIndex):
        raise TypeError("❌ df_train index is not a DatetimeIndex!")
    if df_train.index.isnull().any():
        raise ValueError("❌ Some index values could not be parsed into datetime!")
    
    results = []
    data = df_train.copy()
    
    # Feature groups for original data
    normal_features = [col for col in data.columns if col != target_col]
    normal_feature_combinations = generate_feature_combinations(normal_features, max_features=6)  
    
    # Test original data configurations
    for winsorize in [True, False]: 
        for features in normal_feature_combinations:
            try:
                print(f"\nTesting - Winsorize: {winsorize}, Features: {features}")

                # Use custom param grid if provided for this feature set
                param_grid = None
                if custom_param_grids and tuple(features) in custom_param_grids:
                    param_grid = custom_param_grids[tuple(features)]

                result = choose_parameters_tcn(product_id=product_id,df_train=data,feature_set=features,
                    target_col=target_col,winsorize=winsorize,n_jobs=n_jobs,param_grid=param_grid)
                
                if result:
                    print(f"✅ Success - RMSE: {result['metrics']['RMSE']:.2f}")
                    results.append(result)
            except Exception as e:
                print(f"⚠️ Configuration failed: {str(e)}")
                continue
    
    # Test lagged data configurations if available
    if lagged_df is not None:
        lagged_data = lagged_df.copy()
        lag_features = [col for col in lagged_data.columns if 'lag' in col]
        ma_features = [col for col in lagged_data.columns if '_ma_' in col]
        macro_features = [col for col in data.columns if col != target_col]
        
        # Generate lag feature combinations
        lag_feature_combinations = generate_feature_combinations(lag_features, max_features=5)
        
        # Create combined feature sets
        full_combinations = []
        for lag_combo in lag_feature_combinations:
            full_combinations.append(lag_combo)  # Just lags
            full_combinations.append(lag_combo + macro_features)  # Lags + macros
            full_combinations.append(lag_combo + ma_features)  # Lags + MAs
            full_combinations.append(lag_combo + macro_features + ma_features)  # All
        
        # Remove duplicates
        seen = set()
        unique_combinations = []
        for combo in full_combinations:
            combo_tuple = tuple(sorted(combo))
            if combo_tuple not in seen:
                seen.add(combo_tuple)
                unique_combinations.append(combo)
        
        # Test all unique combinations
        for winsorize in [True, False]:
            for features in unique_combinations:
                try:
                    print(f"\nTesting - Lagged Features: {features}")

                    # Use custom param grid if provided for this feature set
                    param_grid = None
                    if custom_param_grids and tuple(features) in custom_param_grids:
                        param_grid = custom_param_grids[tuple(features)]

                    result = choose_parameters_tcn(product_id=product_id,df_train=lagged_data,feature_set=features,
                                                       target_col=target_col,winsorize=winsorize,n_jobs=n_jobs,param_grid=param_grid)
                    
                    if result:
                        print(f"✅ Success - RMSE: {result['metrics']['RMSE']:.2f}")
                except Exception as e:
                    print(f"⚠️ Lagged configuration failed: {str(e)}")
                    continue
    
    if not results:
        raise ValueError(f"No valid configurations for {product_id}")
    
    # Select best configuration by RMSE
    best_config = min(results, key=lambda x: x['metrics']['RMSE'])
    
    print(f"\n✅ Best configuration for {product_id}:")
    print(f"- Winsorize: {best_config['winsorize']}")
    print(f"- Features: {best_config['features']}")
    print(f"- Best params: {best_config['best_params']}")
    print(f"- RMSE: {best_config['metrics']['RMSE']:.2f}")
    print(f"- Method: {best_config.get('method', 'walkforward')}")
    
    return best_config, results

def process_product_parallel(product_id, custom_param_grids=None):
    try:
        df_train = product_dfs[product_id].rename(columns={product_id: "Sales"})
        
        # Ensure DatetimeIndex
        if not isinstance(df_train.index, pd.DatetimeIndex):
            df_train.index = pd.to_datetime(df_train.index, errors="coerce")
        
        if df_train.index.isnull().any():
            raise ValueError("❌ Some index values could not be parsed into datetime!")

        lagged_df = lagged_product_dfs.get(product_id, None)
        best_config, results = find_best_tcn_config(product_id=product_id,df_train=df_train,lagged_df=lagged_df,target_col="Sales", custom_param_grids=custom_param_grids)
        
        return best_config, results
    except Exception as e:
        print(f"⚠️ Failed to process {product_id}: {str(e)}")
        return None

##### <span style="background-color:#000027; padding:5px; border-radius:5px;">**3.4. Prophet**</span> <a id='prophet'></a>  

Click [here](#table-of-contents) ⬆️ to return to the Index.

Remember - Summary to test in modeling:

| Product | Stationary? (Yes/No) | Notes | 
|---------|----------------------|--------------------|
| P1      |        ✅ Yes       |  Tails off gradually  →   AR(1) |        
| P3      |          ✅ Yes       |   Product 3 (P3), the presence of spikes at lag 12 in both ACF and PACF (with a stronger signal in PACF) strongly suggests seasonal behavior with a 12-period cycle (e.g., yearly patterns in monthly data)→  SARIMA(1, 0, 0)(1, 0, 0, 12)  |             
| P4      |          ✅ Yes         |      strong negative spike at lag 6 in the PACF but a moderate negative spike in the ACF (within confidence intervals) →  SARIMA(0, 0, 0)(1, 0, 0, 6)  # Seasonal AR(1) with period 6 |             
| P5      |          ✅ Yes          |     SARIMA(0,0,0)(1,0,1,6) →  Combines: Seasonal AR(1) (P=1) to capture the PACF spike at lag 6 & Seasonal MA(1) (Q=1) to account for ACF decay at lag 6; Ignores non-seasonal terms (p=q=0) since oscillations are weak|           
| P6      |          ✅ Yes           |    Tails off gradually →   AR(1) |     
| P8      |          ❌ No           |     ACF/PACF Spike at Lag 3: A dominant spike at lag 3 in both ACF and PACF suggests a strong seasonal or cyclical pattern with a 3-period cycle (e.g., quarterly data) → SARIMA(3, 0, 3)(0, 0, 0, 0)  # Pure ARMA(3,3)|            
| P9      |          ✅ Yes            |   ACF: negative spike at lag 12 (-0.37, exceeding confidence bounds) →  yearly anti-correlation →  SARIMA(0, 0, 0)(0, 0, 1, 12)                |             
| P11     |          ✅ Yes            |   PACF: Strong spike at lag 12 (exceeding confidence bounds) →  yearly dependency →   SARIMA(1, 0, 0)(1, 0, 0, 12)              |                     
| P12     |          ✅ Yes           |  Strong Lags 1-3 →   ARIMA(3, 0, 0)                |                    
| P13     |          ✅ Yes           |  The significant PACF spike at lag 17 (with insignificant ACF spike) indicates: A pure seasonal AR(1) effect at lag 17 (uncommon, but possible for long cycles) →  test standard and SARIMA(0, 0, 0)(1, 0, 0, 17) |                      
| P14     |          ✅ Yes           |     Negative Spike at Lag 6 in PACF (Significant) and Near-CI Drop in ACF →  SARIMA(0, 0, 0)(1, 0, 0, 6)  |                      
| P16     |          ✅ Yes          |       Both ACF and PACF show sharp cutoffs after lag 3 → Mixed ARMA(3,3) signature →    ARIMA(3, 0, 3)         |                       
| P20     |          ✅ Yes          |    Tails off gradually  →  PACF spike at lag 13; Seasonal AR(1) → SARIMA(0, 0, 0)(1, 0, 0, 13)|                       
| P36     |          ✅ Yes          |    BOTH ACF and PACF spike at Lag 10   →  SARIMA(0,0,0)(1,0,1,10)           |  

In [19]:
def get_product_specific_config(product_id):
    """Returns Prophet configuration tailored to each product's time series characteristics"""
    product_configs = {
        'P1': {
            'seasonality_mode': 'additive',
            'yearly_seasonality': False,
            'weekly_seasonality': False,
            'daily_seasonality': False,
            'changepoint_prior_scale': 0.05, # → Tails off gradually → AR(1) → Requires smooth trend changes.
            'seasonality_prior_scale': 10.0, # → No strong seasonal effect detected, keeps seasonality influence moderate.
            'extra_regressors': []},
        'P3': {
            'seasonality_mode': 'multiplicative',
            'yearly_seasonality': True, # → Spikes at lag 12 (ACF & PACF) indicate yearly seasonality.
            'weekly_seasonality': False,
            'daily_seasonality': False,
            'changepoint_prior_scale': 0.1, # → Allows flexibility for potential trend shifts in yearly patterns.
            'seasonality_prior_scale': 15.0, # → Strong yearly dependency → Needs a higher seasonal influence.
            'extra_regressors': []},
        'P4': {
            'seasonality_mode': 'additive',
            'yearly_seasonality': False,
            'weekly_seasonality': False,
            'daily_seasonality': False,
            'changepoint_prior_scale': 0.05, # → Trend is relatively stable, no abrupt changes expected.
            'seasonality_prior_scale': 10.0,  # → Moderate seasonal influence, not overly strong.
            'extra_seasonalities': [{
                'name': 'semiannual',
                'period': 6,  # → Strong negative spike at lag 6 (PACF), indicating a semiannual pattern.
                'fourier_order': 3}]},  #→ Simpler seasonal patterns (smoother curves). Balances capturing structure vs. overfitting
        'P5': {
            'seasonality_mode': 'multiplicative',
            'yearly_seasonality': False,
            'weekly_seasonality': False,
            'daily_seasonality': False,
            'changepoint_prior_scale': 0.05,  # → Seasonal AR(1) at lag 6 → Trend should remain stable.
            'seasonality_prior_scale': 10.0, # → Moderate seasonal effect since oscillations are weak.
            'extra_seasonalities': [{
                'name': 'semiannual',
                'period': 6, # → PACF spike at lag 6 → Semiannual pattern.
                'fourier_order': 3}]}, #→ Simpler seasonal patterns (smoother curves).
        'P6': {
            'seasonality_mode': 'additive',
            'yearly_seasonality': False,
            'weekly_seasonality': False,
            'daily_seasonality': False,
            'changepoint_prior_scale': 0.05,  # → Tails off gradually → AR(1) → Requires smooth trend changes.
            'seasonality_prior_scale': 10.0,  # → No strong seasonal effect detected, keeps seasonality influence moderate.
            'extra_regressors': []},
        'P8': {
            'seasonality_mode': 'additive',
            'yearly_seasonality': False,
            'weekly_seasonality': False,
            'daily_seasonality': False,
            'changepoint_prior_scale': 0.2, # → Strong ARMA(3,3) pattern → Needs more flexibility for short-term dependencies.
            'seasonality_prior_scale': 15.0, # → Quarterly seasonality (lag 3) is dominant → Higher seasonal weight.
            'extra_seasonalities': [{
                'name': 'quarterly',
                'period': 3, # → ACF/PACF spike at lag 3 → Strong quarterly pattern.
                'fourier_order': 3}]}, # → Balances capturing structure vs. overfitting
        'P9': {
            'seasonality_mode': 'multiplicative',
            'yearly_seasonality': True, # → Annual pattern with anti-correlation at lag 12
            'weekly_seasonality': False,
            'daily_seasonality': False,
            'changepoint_prior_scale': 0.1, # → Allows some trend flexibility but not too much
            'seasonality_prior_scale': 15.0,  # → Strong yearly anti-correlation (ACF spike at lag 12)
            'extra_regressors': []},
        'P11': {
            'seasonality_mode': 'multiplicative',
            'yearly_seasonality': True, # → Yearly seasonality detected (PACF spike at lag 12)
            'weekly_seasonality': False,
            'daily_seasonality': False,
            'changepoint_prior_scale': 0.15, # → More trend flexibility than P9 (to capture variations)
            'seasonality_prior_scale': 20.0,  # → Very strong yearly dependency (PACF spike at lag 12)
            'extra_regressors': []},
        'P12': {
            'seasonality_mode': 'additive',
            'yearly_seasonality': False,
            'weekly_seasonality': False,
            'daily_seasonality': False,
            'changepoint_prior_scale': 0.3,  # → Short-term dependency detected (lags 1-3 in PACF) → needs a high changepoint_prior_scale (0.3) to adapt quickly
            'seasonality_prior_scale': 5.0, # → Lower seasonality weight since pattern isn't dominant
            'extra_regressors': []},
        'P13': {
            'seasonality_mode': 'additive',
            'yearly_seasonality': False,
            'weekly_seasonality': False,
            'daily_seasonality': False,
            'changepoint_prior_scale': 0.1, # → Moderate flexibility to capture gradual changes
            'seasonality_prior_scale': 15.0, # → Needed for seasonal AR(1) effect at lag 17
            'extra_seasonalities': [{
                'name': 'custom_17',
                'period': 17, # →  PACF spike at lag 17 (capturing uncommon long cycle)
                'fourier_order': 3}]},  # → Balances capturing structure vs. overfitting
        'P14': {
            'seasonality_mode': 'additive',
            'yearly_seasonality': False,
            'weekly_seasonality': False,
            'daily_seasonality': False,
            'changepoint_prior_scale': 0.05, # → Minimal trend flexibility (stable seasonality)
            'seasonality_prior_scale': 10.0, # → Moderate seasonality strength for lag 6 pattern
            'extra_seasonalities': [{ 
                'name': 'semiannual',
                'period': 6, # → Captures PACF negative spike at lag 6
                'fourier_order': 3}]}, # → Balances capturing structure vs. overfitting
        'P16': {
            'seasonality_mode': 'additive',
            'yearly_seasonality': False,
            'weekly_seasonality': False,
            'daily_seasonality': False,
            'changepoint_prior_scale': 0.3,  # → High flexibility to adapt to short-term ARMA(3,3) behavior 
            'seasonality_prior_scale': 5.0, # → Seasonality isn't dominant, keeping it low
            'extra_regressors': []},
        'P20': {
            'seasonality_mode': 'additive',
            'yearly_seasonality': False,
            'weekly_seasonality': False,
            'daily_seasonality': False,
            'changepoint_prior_scale': 0.1, # → Allows mild trend changes 
            'seasonality_prior_scale': 15.0, # → Strong seasonal AR(1) at lag 13
            'extra_seasonalities': [{
                'name': 'custom_13',
                'period': 13, # → Captures PACF spike at lag 13
                'fourier_order': 3}]}, # → Balances capturing structure vs. overfitting
        'P36': {
            'seasonality_mode': 'multiplicative',
            'yearly_seasonality': False,
            'weekly_seasonality': False,
            'daily_seasonality': False,
            'changepoint_prior_scale': 0.1,  # → Some flexibility in trend but not too much
            'seasonality_prior_scale': 15.0, # → Strong seasonality to match ACF/PACF spikes at lag 10
            'extra_seasonalities': [{
                'name': 'custom_10',
                'period': 10, # → Captures both ACF & PACF spikes at lag 10
                'fourier_order': 3}]}}  # → Balances capturing structure vs. overfitting
    
    # Default configuration 
    default_config = {
        'seasonality_mode': 'additive',
        'yearly_seasonality': 'auto',
        'weekly_seasonality': False,
        'daily_seasonality': False,
        'changepoint_prior_scale': 0.05, # → Minimal trend flexibility (stable seasonality)
        'seasonality_prior_scale': 10.0, # → Moderate seasonality strength for lag 6 pattern 
        'extra_regressors': []}
    
    return product_configs.get(product_id, default_config)

In [20]:
def choose_parameters_prophet(product_id, df_train, feature_set, target_col="Sales", 
                             winsorize=False, horizon=7):
    
    # --- Input Validation ---
    if len(df_train) < 12:
        print(f"⚠️ Skipping {product_id} - only {len(df_train)} observations")
        return None
    
    # --- Data Preparation ---
    print('Preparing data...')
    try:
        X_train, X_val, y_train, y_val = prepare_time_series_data(
            df_train, feature_set, target_col, horizon, winsorize
        )

    except Exception as e:
        print(f"Data preparation failed: {str(e)}")
        return None
    
    # Get product-specific configuration
    config = get_product_specific_config(product_id)
    
    # Prepare Prophet-compatible DataFrames
    train_df = pd.DataFrame({
        'ds': X_train.index.to_numpy(),
        'y': y_train.values.flatten()
    })
    
    # Add regressors if any
    regressors = []
    if feature_set:
        for feature in feature_set:
            if feature in X_train.columns:
                train_df[feature] = X_train[feature].values.reshape(-1)
                regressors.append(feature)
    
    # --- Model Configuration Function ---
    def create_prophet_model():
        """Helper to create a fresh Prophet instance with current config"""
        model = Prophet(seasonality_mode=config['seasonality_mode'], yearly_seasonality=config['yearly_seasonality'], weekly_seasonality=config['weekly_seasonality'],
            daily_seasonality=config['daily_seasonality'], changepoint_prior_scale=config['changepoint_prior_scale'], seasonality_prior_scale=config['seasonality_prior_scale'])
        
        # custom seasonalities
        if 'extra_seasonalities' in config:
            for seasonality in config['extra_seasonalities']:
                model.add_seasonality(name=seasonality['name'], period=seasonality['period'], fourier_order=seasonality['fourier_order'])
        
        # Add regressors
        for reg in regressors:
            model.add_regressor(reg)
        
        return model
    
    # --- Simple Forecast (12 ≤ obs < 36) ---
    if len(X_train) < 36:
        print(f"Using simple forecast ({len(X_train)} < 36 obs)")
        try:
            model = create_prophet_model()
            model.fit(train_df)
            
            future = model.make_future_dataframe(periods=horizon, freq='M')
            
            if regressors and len(X_val) >= horizon:
                for reg in regressors:
                    future[reg] = X_val[reg].iloc[:horizon].values.reshape(-1)
            
            forecast = model.predict(future)
            val_predictions = forecast['yhat'].values[-horizon:].flatten()
            train_predictions = forecast['yhat'].values[:-horizon].flatten() if horizon > 0 else forecast['yhat'].values.flatten()
            
            return {
                'product_id': product_id,
                'winsorize': winsorize,
                'features': feature_set,
                'metrics': calculate_metrics(y_true=y_val.values.flatten(), y_pred=val_predictions, y_train=y_train.values.flatten(), y_train_pred=train_predictions),
                'method': 'simple',
                'validation_predictions': val_predictions,
                'model_config': config}
        except Exception as e:
            print(f"Simple forecast failed: {str(e)}")
            return None
    
    # --- Walk-Forward (≥36 obs) ---
    print(f"Using walk-forward ({len(X_train)} ≥ 36 obs)")
    predictions = []
    actuals = []
    history = train_df.copy()
    history = history.convert_dtypes()

    # Get training predictions
    try:
        train_model = create_prophet_model()
        train_model.fit(history)
        future_train = train_model.make_future_dataframe(periods=0, freq='M')
        if regressors:
            for reg in regressors:
                future_train[reg] = history[reg].values.astype(float)  # Explicit float conversion
        train_forecast = train_model.predict(future_train)
        train_predictions = train_forecast['yhat'].values.flatten()
    except Exception as e:
        print(f"Training pred failed: {str(e)} - using last value")
        train_predictions = np.full(len(y_train), y_train.iloc[-1])

    # Walk-forward validation
    for i in range(horizon):
        try:
            # Create fresh model
            step_model = create_prophet_model()
            
            # Ensure numeric types
            history = history.infer_objects()
            
            # Fit model
            step_model.fit(history)
            
            # Prepare one-step forecast
            future_step = step_model.make_future_dataframe(periods=1, include_history=False, freq='M')
            
            # Add regressors with type checking
            if regressors and i < len(X_val):
                for reg in regressors:
                    if not np.issubdtype(X_val[reg].dtype, np.number):
                        raise TypeError(f"Regressor {reg} contains non-numeric values")
                    future_step[reg] = float(X_val[reg].iloc[i])  # Explicit float conversion
            
            # Make prediction
            forecast_step = step_model.predict(future_step)
            pred = float(forecast_step['yhat'].iloc[0])  # Explicit float conversion
            predictions.append(pred)
            actual = float(y_val.iloc[i]) if i < len(y_val) else np.nan
            actuals.append(actual)
            
            # Update history 

            new_row = {'ds': X_val.index[i], 'y': actual}
            if regressors and i < len(X_val):
                for reg in regressors:
                    new_row[reg] = float(X_val[reg].iloc[i])
            
            history = pd.concat([
                history, 
                pd.DataFrame([new_row]).infer_objects()], ignore_index=True).convert_dtypes()
            
        except Exception as e:
            print(f"Iteration {i+1} failed: {str(e)}")
            predictions.append(float(y_train.iloc[-1]))
            actuals.append(float(y_val.iloc[i]) if i < len(y_val) else np.nan)
    
    # Calculate metrics using actual observed values
    metrics = calculate_metrics(
        y_true=np.array(actuals).flatten(),
        y_pred=np.array(predictions).flatten(),
        y_train=y_train.values.flatten(),
        y_train_pred=train_predictions)
    
    return {
        'product_id': product_id, 'winsorize': winsorize, 'features': feature_set, 'metrics': metrics, 'method': 'walkforward', 'validation_predictions': predictions, 'model_config': config}

def find_best_prophet_config(product_id, df_train, lagged_df=None, target_col="Sales"):
    """Find best Prophet configuration with improved error handling"""
    
    # Validate input data
    if not isinstance(df_train.index, pd.DatetimeIndex):
        raise TypeError("❌ df_train index is not a DatetimeIndex!")
    if df_train.index.isnull().any():
        raise ValueError("❌ Some index values could not be parsed into datetime!")
    
    results = []
    data = df_train.copy()
    
    # Feature groups for original data
    normal_features = [col for col in data.columns if col != target_col]
    normal_feature_combinations = generate_feature_combinations(normal_features, max_features=6)  
    
    # Test original data configurations
    for winsorize in [True, False]:
        for features in normal_feature_combinations:
            try:
                print(f"\nTesting - Winsorize: {winsorize}, Features: {features}")

                result = choose_parameters_prophet(product_id=product_id, df_train=data, feature_set=features, target_col=target_col, winsorize=winsorize)
                
                if result:
                    print(f"✅ Success - RMSE: {result['metrics']['RMSE']:.2f}")
                    results.append(result)
            except Exception as e:
                print(f"⚠️ Configuration failed: {str(e)}")
                continue
    
    # Test lagged data configurations if available
    if lagged_df is not None:
        lagged_data = lagged_df.copy()
        lag_features = [col for col in lagged_data.columns if 'lag' in col]
        ma_features = [col for col in lagged_data.columns if '_ma_' in col]
        macro_features = [col for col in data.columns if col != target_col]
        
        # Generate lag feature combinations
        lag_feature_combinations = generate_feature_combinations(lag_features, max_features=5)
        
        # Create combined feature sets
        full_combinations = []
        for lag_combo in lag_feature_combinations:
            full_combinations.append(lag_combo)
            full_combinations.append(lag_combo + macro_features)
            full_combinations.append(lag_combo + ma_features)
            full_combinations.append(lag_combo + macro_features + ma_features)
        
        # Remove duplicates
        seen = set()
        unique_combinations = []
        for combo in full_combinations:
            combo_tuple = tuple(sorted(combo))
            if combo_tuple not in seen:
                seen.add(combo_tuple)
                unique_combinations.append(combo)
        
        # Test all unique combinations
        for winsorize in [True, False]:
            for features in unique_combinations:
                try:
                    print(f"\nTesting - Lagged Features: {features}")

                    result = choose_parameters_prophet(product_id=product_id, df_train=lagged_data, feature_set=features, target_col=target_col, winsorize=winsorize)
                    
                    if result:
                        print(f"✅ Success - RMSE: {result['metrics']['RMSE']:.2f}")
                        results.append(result)
                except Exception as e:
                    print(f"⚠️ Lagged configuration failed: {str(e)}")
                    continue
    
    if not results:
        raise ValueError(f"No valid configurations for {product_id}")
    
    # Select best configuration by RMSE
    best_config = min(results, key=lambda x: x['metrics']['RMSE'])
    
    print(f"\n✅ Best configuration for {product_id}:")
    print(f"- Winsorize: {best_config['winsorize']}")
    print(f"- Features: {best_config['features']}")
    print(f"- RMSE: {best_config['metrics']['RMSE']:.2f}")
    print(f"- Method: {best_config.get('method', 'walkforward')}")
    
    return best_config, results

def process_product_prophet(product_id):
    """Parallel processing wrapper for NeuralProphet"""
    try:
        df_train = product_dfs[product_id].rename(columns={product_id: "Sales"})
        
        # Ensure DatetimeIndex
        if not isinstance(df_train.index, pd.DatetimeIndex):
            df_train.index = pd.to_datetime(df_train.index, errors="coerce")
        
        if df_train.index.isnull().any():
            raise ValueError("❌ Some index values could not be parsed into datetime!")

        lagged_df = lagged_product_dfs.get(product_id, None)
        
        best_config, results = find_best_prophet_config(
            product_id=product_id,
            df_train=df_train,
            lagged_df=lagged_df,
            target_col="Sales")
        
        return best_config, results
    except Exception as e:
        print(f"⚠️ Failed to process {product_id}: {str(e)}")
        return None

##### <span style="background-color:#000027; padding:5px; border-radius:5px;">**3.5. ARIMA**</span> <a id='arima'></a>  

Click [here](#table-of-contents) ⬆️ to return to the Index.

Remember - Summary to test in modeling:

| Product | Stationary? (Yes/No) | Notes | 
|---------|----------------------|--------------------|
| P1      |        ✅ Yes       |  Tails off gradually  →   AR(1) |        
| P3      |          ✅ Yes       |   Product 3 (P3), the presence of spikes at lag 12 in both ACF and PACF (with a stronger signal in PACF) strongly suggests seasonal behavior with a 12-period cycle (e.g., yearly patterns in monthly data)→  SARIMA(1, 0, 0)(1, 0, 0, 12)  |             
| P4      |          ✅ Yes         |      strong negative spike at lag 6 in the PACF but a moderate negative spike in the ACF (within confidence intervals) →  SARIMA(0, 0, 0)(1, 0, 0, 6)  # Seasonal AR(1) with period 6 |             
| P5      |          ✅ Yes          |     SARIMA(0,0,0)(1,0,1,6) →  Combines: Seasonal AR(1) (P=1) to capture the PACF spike at lag 6 & Seasonal MA(1) (Q=1) to account for ACF decay at lag 6; Ignores non-seasonal terms (p=q=0) since oscillations are weak|           
| P6      |          ✅ Yes           |    Tails off gradually →   AR(1) |     
| P8      |          ❌ No           |     ACF/PACF Spike at Lag 3: A dominant spike at lag 3 in both ACF and PACF suggests a strong seasonal or cyclical pattern with a 3-period cycle (e.g., quarterly data) → SARIMA(3, 0, 3)(0, 0, 0, 0)  # Pure ARMA(3,3)|            
| P9      |          ✅ Yes            |   ACF: negative spike at lag 12 (-0.37, exceeding confidence bounds) →  yearly anti-correlation →  SARIMA(0, 0, 0)(0, 0, 1, 12)                |             
| P11     |          ✅ Yes            |   PACF: Strong spike at lag 12 (exceeding confidence bounds) →  yearly dependency →   SARIMA(1, 0, 0)(1, 0, 0, 12)              |                     
| P12     |          ✅ Yes           |  Strong Lags 1-3 →   ARIMA(3, 0, 0)                |                    
| P13     |          ✅ Yes           |  The significant PACF spike at lag 17 (with insignificant ACF spike) indicates: A pure seasonal AR(1) effect at lag 17 (uncommon, but possible for long cycles) →  test standard and SARIMA(0, 0, 0)(1, 0, 0, 17) |                      
| P14     |          ✅ Yes           |     Negative Spike at Lag 6 in PACF (Significant) and Near-CI Drop in ACF →  SARIMA(0, 0, 0)(1, 0, 0, 6)  |                      
| P16     |          ✅ Yes          |       Both ACF and PACF show sharp cutoffs after lag 3 → Mixed ARMA(3,3) signature →    ARIMA(3, 0, 3)         |                       
| P20     |          ✅ Yes          |    Tails off gradually  →  PACF spike at lag 13; Seasonal AR(1) → SARIMA(0, 0, 0)(1, 0, 0, 13)|                       
| P36     |          ✅ Yes          |    BOTH ACF and PACF spike at Lag 10   →  SARIMA(0,0,0)(1,0,1,10)           |  

In [21]:
def choose_parameters_arima(product_id, df_train, feature_set, target_col="Sales", 
                          winsorize=False, horizon=7, seasonal_period=None,
                          manual_order=None, manual_seasonal_order=None):
    """
    ARIMA/SARIMAX with:
    - Support for both manual and auto-ARIMA
    - Proper non-stationary handling
    - Fixed dimensionality issues
    """
    # --- Input Validation ---
    if len(df_train) < 12:
        print(f"⚠️ Skipping {product_id} - only {len(df_train)} observations")
        return None
    
    # --- Data Preparation ---
    print('Preparing data...')
    try:
        X_train, X_val, y_train, y_val = prepare_time_series_data(df_train, feature_set, target_col, horizon, winsorize)
        
        # Convert to numpy arrays and ensure correct shapes
        y_train = np.array(y_train).ravel()
        y_val = np.array(y_val).ravel()
        
    except Exception as e:
        print(f"Data preparation failed: {str(e)}")
        return None
    
    # --- Prepare Exogenous Data ---
    exog_train = np.array(X_train[feature_set]) if feature_set else None
    exog_val = np.array(X_val[feature_set]) if feature_set else None
    
    # --- Walk-Forward Validation ---
    print(f"Using walk-forward validation ({len(X_train)} observations)")
    predictions = []
    history = y_train.copy()
    exog_history = exog_train.copy() if exog_train is not None else None
    
    # Get training predictions
    try:
        if manual_order:  # Manual configuration path
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")

                model = SARIMAX(history, exog=exog_history, order=manual_order,
                              seasonal_order=manual_seasonal_order if manual_seasonal_order else (0,0,0,0),
                              enforce_stationarity=False, enforce_invertibility=False).fit(disp=False)
                
                train_predictions = model.get_prediction().predicted_mean

        else:  # Auto-ARIMA path
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")

                train_model = auto_arima(history, X=exog_history, seasonal=seasonal_period is not None,
                                      m=seasonal_period if seasonal_period else 1, 
                                      suppress_warnings=True, error_action='ignore',
                                      test='adf') # Auto-detect differencing needs
                
                train_predictions = train_model.predict_in_sample(X=exog_history)

    except Exception as e:
        print(f"Training pred failed: {str(e)} - using last value")
        train_predictions = [history[-1]] * len(history)
    
    for i in range(horizon):
        try:
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                current_exog = exog_val[[i]] if exog_val is not None else None
                
                if manual_order:  # Manual fit
                    model = SARIMAX(history, exog=exog_history, order=manual_order,
                                  seasonal_order=manual_seasonal_order if manual_seasonal_order else (0,0,0,0),
                                  enforce_stationarity=False).fit(disp=False)
                    
                    pred = model.forecast(steps=1, exog=current_exog)[0]

                else:  # Auto-ARIMA
                    model = auto_arima(history, X=exog_history, seasonal=seasonal_period is not None,
                                     m=seasonal_period if seasonal_period else 1, 
                                     suppress_warnings=True, error_action='ignore')
                    
                    pred = model.predict(n_periods=1, X=current_exog)[0]
                
                predictions.append(pred)
                # Update history maintaining 1D array
                history = np.concatenate([history, [pred]])
                if exog_history is not None:
                    exog_history = np.vstack([exog_history, exog_val[i]])
                    
        except Exception as e:
            print(f"Iteration {i+1} failed: {str(e)}")
            predictions.append(history[-1])
            continue
    
    metrics = calculate_metrics(y_true=y_val, y_pred=predictions, y_train=y_train, y_train_pred=train_predictions)
    
    return {
        'product_id': product_id, 'winsorize': winsorize, 'features': feature_set,  
        'metrics': metrics, 'method': 'manual' if manual_order else 'auto',
        'validation_predictions': predictions, 'model_order': manual_order if manual_order else model.order,
        'seasonal_order': manual_seasonal_order if manual_seasonal_order else (model.seasonal_order if hasattr(model, 'seasonal_order') else (0,0,0,0))}

def find_best_arima_config(product_id, df_train, target_col="Sales"):
    """
    Tests both manual (from ACF/PACF) and auto-ARIMA configurations
    """
        
    #  DatetimeIndex
    if not isinstance(df_train.index, pd.DatetimeIndex):
        df_train.index = pd.to_datetime(df_train.index, errors="coerce")

    # Validate input
    if not isinstance(df_train.index, pd.DatetimeIndex):
        raise TypeError("❌ df_train index is not a DatetimeIndex!")
    
    results = []
    data = df_train.copy()
    
    # --- Manual Configurations from ACF/PACF Analysis ---
    manual_configs = {
        'P1': [{'order': (1,0,0)}], # AR(1)
        'P3': [{'order': (1,0,0), 'seasonal_order': (1,0,0,12)}], # SARIMA(1,0,0)(1,0,0,12)
        'P4': [{'order': (0,0,0), 'seasonal_order': (1,0,0,6)}], # SARIMA(0,0,0)(1,0,0,6)
        'P5': [{'order': (0,0,0), 'seasonal_order': (1,0,1,6)}], # SARIMA(0,0,0)(1,0,1,6)
        'P6': [{'order': (1,0,0)}], # AR(1)
        'P8': [  # Non-stationary series
            {'order': (3,1,3)},  # ARIMA(3,1,3) - with differencing
            {'order': (3,0,3)}], # ARIMA(3,0,3) - without differencing for comparison
        'P9': [{'order': (0,0,0), 'seasonal_order': (0,0,1,12)}], # SARIMA(0,0,0)(0,0,1,12)
        'P11': [{'order': (1,0,0), 'seasonal_order': (1,0,0,12)}], # SARIMA(1,0,0)(1,0,0,12)
        'P12': [{'order': (3,0,0)}],  # AR(3)
        'P13': [
            {'order': (0,0,0), 'seasonal_order': (1,0,0,17)},  # SARIMA(0,0,0)(1,0,0,17)
            {'order': (1,0,0)}], # Also test standard AR(1) as fallback
        'P14': [{'order': (0,0,0), 'seasonal_order': (1,0,0,6)}], # SARIMA(0,0,0)(1,0,0,6)
        'P16': [{'order': (3,0,3)}],  # ARMA(3,3)
        'P20': [{'order': (0,0,0), 'seasonal_order': (1,0,0,13)}], # SARIMA(0,0,0)(1,0,0,13)
        'P36': [{'order': (0,0,0), 'seasonal_order': (1,0,1,10)}]} # SARIMA(0,0,0)(1,0,1,10)
    
    # --- Test Manual Configurations ---
    if product_id in manual_configs:
        for config in manual_configs[product_id]:
            for winsorize in [True, False]:
                try:
                    result = choose_parameters_arima(product_id=product_id, df_train=data,
                                                     feature_set=[f for f in data.columns if f != target_col],
                                                     target_col=target_col,
                                                     winsorize=winsorize,
                                                     manual_order=config['order'],
                                                     manual_seasonal_order=config.get('seasonal_order'))
                    if result:
                        print(f"✅ Manual Success - Order: {result['model_order']} "
                              f"Seasonal: {result['seasonal_order']} "
                              f"RMSE: {result['metrics']['RMSE']:.2f}")
                        results.append(result)
                except Exception as e:
                    print(f"⚠️ Manual config failed: {str(e)}")
    
    # --- Test Auto-ARIMA Configurations ---
    seasonal_periods = {
        'P3': 12, 'P4': 6, 'P5': 6, 'P9': 12,
        'P11': 12, 'P13': 17, 'P14': 6, 'P20': 13, 'P36': 10}
    
    seasonal_period = seasonal_periods.get(product_id)
    
    normal_features = [col for col in data.columns if col != target_col]
    feature_combinations = generate_feature_combinations(normal_features, max_features=6)
    feature_combinations = [[]] + feature_combinations  # Include no-features option
    
    for winsorize in [True, False]:
        for features in feature_combinations:
            try:
                result = choose_parameters_arima(product_id=product_id, df_train=data, feature_set=features,
                                                 target_col=target_col, winsorize=winsorize, seasonal_period=seasonal_period)
                if result:
                    print(f"✅ AutoARIMA Success - Features: {len(features)} "
                          f"RMSE: {result['metrics']['RMSE']:.2f}")
                    results.append(result)
            except Exception as e:
                print(f"⚠️ AutoARIMA failed: {str(e)}")
    
    if not results:
        raise ValueError(f"No valid ARIMA configurations for {product_id}")
    
    # Select best configuration
    best_config = min(results, key=lambda x: x['metrics']['RMSE'])
    
    print(f"\n✅ Best ARIMA configuration for {product_id}:")
    print(f"- Method: {best_config['method']}")
    print(f"- Winsorize: {best_config['winsorize']}")
    print(f"- Features: {best_config['features']}")
    print(f"- Order: {best_config['model_order']}")
    print(f"- Seasonal Order: {best_config['seasonal_order']}")
    print(f"- RMSE: {best_config['metrics']['RMSE']:.2f}")
    
    return best_config, results

##### <span style="background-color:#000027; padding:5px; border-radius:5px;">**3.6. Results Comparison**</span> <a id='compare'></a>  

Click [here](#table-of-contents) ⬆️ to return to the Index.

In [22]:
display_excel_sheets_side_by_side("Modeling_results.xlsx")

Model,product_id,winsorize,features,RMSE,Overfit_Score,Parameters,Feature Importance
TimeGPT,P1,True,"PRO27826_org, MAB_ELE_SHP1100",3475393.6312,-0.3474,NaN,NaN
Prophet,P1,True,"['MAB_ELE_SHP840', 'PRO27826_org', 'MAB_ELE_SHP1100']",2164903.6986,-0.5316,NaN,NaN
ARIMA,P1,True,"['PRI27276_org', 'PRO27826_org', 'MAB_ELE_PRO276', 'MAB_ELE_SHP1100']",2077977.6128,-0.9453,"arima: auto; model_order: (1, 0, 0); seasonal_order: (0, 0, 0, 0)",NaN
XGBoost,P1,True,"['P1_lag_6', 'P1_lag_18', 'P1_ma_6', 'P1_ma_18']",1756694.2558,22.3548,"{'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 0.9}","{'P1_lag_6': 0.30282173, 'P1_lag_18': 0.18046999, 'P1_ma_6': 0.30943987, 'P1_ma_18': 0.20726843}"
Model,product_id,winsorize,features,RMSE,Overfit_Score,Parameters,Feature Importance
TimeGPT,P3,True,PRO27826_org,1400586.7625,-0.5273,NaN,NaN
Prophet,P3,True,['PRI27840_org'],4038685.9300,5.8188,NaN,NaN
ARIMA,P3,True,['PRI27840_org'],2177873.1620,-0.3089,"arima: auto; model_order: (1, 0, 1); seasonal_order: (0, 1, 1, 12)",NaN
XGBoost,P3,True,"['P3_lag_12', 'P3_ma_12']",1142677.4785,-0.1396,"{'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 2, 'min_child_weight': 3, 'n_estimators': 50, 'subsample': 0.9}","{'P3_lag_12': 0.5673661, 'P3_ma_12': 0.43263388}"
Model,product_id,winsorize,features,RMSE,Overfit_Score,Parameters,Feature Importance


Final Decisions:

| Product | Model | Justification |
|---------|-------|---------------|
|    P1      |  ARIMA  ['PRI27276_org', 'PRO27826_org', 'MAB_ELE_PRO276', 'MAB_ELE_SHP1100']  |               |
|    P3    |  XGBoost ['P3_lag_12', 'P3_ma_12'] |   Lowest RMSE and Overfit_Score   |
|    P4    |  ARIMA  ['PRO27380_org', 'WKLWEUR840_org', 'PRO27276_org', 'MAB_ELE_SHP380']  |  XGBoost has the best RMSE but only 1 feature, could be risky, so we will opt for the  ARIMA since it captured the sesonality 6 we saw before - has the second lowest RMSE and also a relatively stable Overfit         |
|    P5    |   XGBoost ['PRO27826_org'] |  P3-FALSE-PRO27826_org, PRO271000_org-3 190 655.46 Bad option Similiar case to P4 but ARIMA would be safer if the RMSE was close (like in P4), but here, the performance gap is too big to ignore here.     |         
|    P6    |  ARIMA 	['PRO27840_org', 'RohCRUDE_PETRO1000_org']   |   Second lowest, but XGBoost has an Overfit Score of 1.0149, meaning it's unstable            |
|    P8 !!   |  XGBoost ['P8_lag_10', 'PRI27840_org', 'RohCOPPER1000_org', 'P8_ma_1', 'P8_ma_2', 'P8_ma_5', 'P8_ma_6', 'P8_ma_10']   |   We have a lot of features so it can lead to overfit neverthless since P8 has a complex nature its best to capture that trhough lags. Remember  → The lag model is highly statistically significant.  try wiyhou some mas             |
|    P9    |   XGBoost ['P9_lag_12', 'PRO27826_org', 'PRO271000_org', 'PRO28250_org', 'MAB_ELE_PRO156']  |   Clear winner            |
|    P11   |  XGBoost  ['MAB_ELE_SHP276']   |  Best RMSE with a good Overfit Score   maybe add ['MAB_ELE_SHP392']  -> even if adding another feature doesn't improve RMSE, it helps the model generalize better        |
|    P12   |   ARIMA ['MAB_ELE_PRO156']  |               |
|    P13      |  XGBoost  ['P13_lag_8', 'P13_ma_17', 'P13_ma_5', 'P13_ma_8']   |               |
|    P14      |   Prophet ['PRO27392_org', 'PRO28380_org']  |               |
|    P16      |   XGBoost  ['P16_lag_3', 'P16_lag_8', 'P16_lag_10', 'P16_ma_3', 'P16_ma_7', 'P16_ma_8', 'P16_ma_9', 'P16_ma_10'] try to remove the mas of the ones that dont have lags  |               |
|    P20      |  XGBoost  ['PRO27250_org', 'PRO28392_org']  |   lowest RMSE and a solid negative Overfit Score       |
|    P36      |  XGBoost  ['PRO27756_org']	  |    Not a very stable result for this product           |
|    Sales Total      |  XGBoost ['PRI27276_org', 'PRO27826_org']   |               |


> Funtions

> XGBoost

In [23]:
def forecast_xgboost(df, test_dataset, product_id, feature_set, best_params, target_col='Sales', horizon=10, winsorize=False):
    """Forecast using XGBoost, taking values directly from test_dataset."""

    # Base model configuration
    base_config = {
        'objective': 'reg:squarederror',
        'random_state': 42,
        'n_jobs': 1,
        'tree_method': 'hist',
        'enable_categorical': False,
        'gamma': 0,
        'reg_alpha': 0,
        'reg_lambda': 1
    }
    final_params = {**base_config, **best_params}

    print(f"Processing XGBoost for {product_id} with params: {final_params}")
    
    try:
        df.index = pd.to_datetime(df.index)
        test_dataset.index = pd.to_datetime(test_dataset.index)

        if winsorize:
            X, _ = winsorize_function(df, cols=[target_col], lower_quantile=0.01, upper_quantile=0.99)
        else:
            X = df.copy()

        X_train = X[feature_set + [target_col]].copy()
        y_train = X[target_col]

        model = xgb.XGBRegressor(**final_params)
        model.fit(X_train[feature_set], y_train)

        predictions = []
        
        for i in range(horizon):
            try:
                # 🚀 Take values directly from test_dataset
                input_features = test_dataset.iloc[[i]][feature_set].copy()

                # Check for NaNs in inputs
                if input_features.isna().sum().sum() > 0:
                    print(f"⚠️ NaNs detected in iteration {i+1}.")
                
                # Make prediction
                pred = model.predict(input_features)[0]
                predictions.append(pred)

                print(f"Iteration {i+1}: Prediction={pred}, Inputs={input_features.values}")

            except Exception as e:
                print(f"Iteration {i+1} failed: {str(e)}")
                predictions.append(np.nan)

        # Plot result
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=df.index, y=df[target_col], mode='lines', name='Actual'))
        fig.add_trace(go.Scatter(x=test_dataset.index[:horizon], y=predictions, mode='lines', name='Forecast'))
        fig.update_layout(title=f'XGBoost Forecast for {product_id}', xaxis_title='Date', yaxis_title=target_col)
        fig.show()

        return predictions

    except Exception as e:
        print(f"Forecast failed for {product_id}: {str(e)}")
        return None

> Prophet

In [24]:
def forecast_prophet(df, test_dataset, product_id, feature_set, target_col='Sales', horizon=10,
                    winsorize=False):
    """Forecast using Prophet without lag features."""
    print(f"Processing Prophet for {product_id}...")
    
    try:
        # Prepare the data
        if winsorize:
            X, _ = winsorize_function(df, cols=[target_col], lower_quantile=0.01, upper_quantile=0.99)
            X = X.reset_index()  # Ensure month_year is a column
        else:
            X = df.reset_index()
        
        # Get product-specific configuration
        config = get_product_specific_config(product_id)
        
        # Initialize Prophet with the configuration
        model = Prophet(
            seasonality_mode=config['seasonality_mode'],
            yearly_seasonality=config['yearly_seasonality'],
            weekly_seasonality=config['weekly_seasonality'],
            daily_seasonality=config['daily_seasonality'],
            changepoint_prior_scale=config['changepoint_prior_scale'],
            seasonality_prior_scale=config['seasonality_prior_scale'])
        
        # Add extra seasonalities if specified
        if 'extra_seasonalities' in config:
            for seasonality in config['extra_seasonalities']:
                model.add_seasonality(name=seasonality['name'],period=seasonality['period'],fourier_order=seasonality['fourier_order'])
        
        # Identify regressors (exclude date and target columns)
        regressors = [f for f in feature_set if f not in ['month_year', target_col]]
        
        # Add regressors to the model
        for reg in regressors:
            model.add_regressor(reg)
        
        # Prepare Prophet-compatible dataframe
        prophet_df = X[['month_year', target_col] + regressors].copy()
        prophet_df = prophet_df.rename(columns={'month_year': 'ds', target_col: 'y'})
        
        # Fit the model
        model.fit(prophet_df)
        
        # Create future dataframe for all steps
        future = pd.DataFrame({'ds': test_dataset.index[:horizon]})
        
        # Add regressor values from test data
        for reg in regressors:
            future[reg] = test_dataset[reg].values[:horizon]
        
        # Make predictions
        forecast = model.predict(future)
        predictions = forecast['yhat'].values
        
        # Plotting
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=df.index, y=df[target_col], mode='lines', name='Actual'))
        fig.add_trace(go.Scatter(x=test_dataset.index[:horizon], y=predictions, mode='lines', name='Forecast'))
        fig.update_layout(title=f'Prophet Forecast for {product_id}',xaxis_title='Date',yaxis_title='Sales')
        fig.show()
        
        return predictions
        
    except Exception as e:
        print(f"Forecasting failed for {product_id}: {str(e)}")
        return None

> ARIMA

In [25]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
import pandas as pd
import numpy as np
import plotly.graph_objects as go

def forecast_sarimax(df, test_dataset, product_id, feature_set, target_col='Sales', horizon=10,
                    order=(0,0,0), seasonal_order=(0,0,0,0), winsorize=False):
    """Forecast using SARIMAX without lag features."""
    print(f"Processing SARIMAX for {product_id}...")
    
    try:
        # Prepare the data
        if winsorize:
            X, _ = winsorize_function(df, cols=[target_col], lower_quantile=0.01, upper_quantile=0.99)
            X = X.reset_index(drop=True)
        else:
            X = df.reset_index(drop=True)
        
        # Identify regressors (exclude target column)
        regressors = [f for f in feature_set if f != target_col]
        
        # Prepare history data
        history = X[target_col]
        exog_history = X[regressors] if regressors else None
        
        # Fit SARIMAX model
        model = SARIMAX(history, 
                       exog=exog_history, 
                       order=order,
                       seasonal_order=seasonal_order,
                       enforce_stationarity=False).fit(disp=False)
        
        # Prepare future exog data
        exog_future = test_dataset[regressors].iloc[:horizon] if regressors else None
        
        # Make predictions
        predictions = model.get_forecast(steps=horizon, exog=exog_future).predicted_mean
        
        # Plotting
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=df.index, y=df[target_col], mode='lines', name='Actual'))
        fig.add_trace(go.Scatter(x=test_dataset.index[:horizon], y=predictions, mode='lines', name='Forecast'))
        fig.update_layout(title=f'SARIMAX Forecast for {product_id}',
                         xaxis_title='Date',
                         yaxis_title='Sales')
        fig.show()
        
        return predictions
        
    except Exception as e:
        print(f"Forecasting failed for {product_id}: {str(e)}")
        return None

#### <span style="background-color:#000027; padding:5px; border-radius:5px;">**4. Exogenous Variables Prediction**</span> <a id='market'></a>  

Click [here](#table-of-contents) ⬆️ to return to the Index.


So noe that we have the models and the features for each product we will predict every macro feature a product needs, but first we need to see the ones the products have in common so that we can reuse the predictions if we need:

In [26]:
products = list(product_dfs.keys())

# Dictionary to store macros and their corresponding products
macro_products = {}
for product_key, df in product_dfs.items():
    # Iterate through columns (excluding 'Sales')
    for col in df.columns:
        if col != 'Sales' and col not in products:  # Ignores 'Sales' and product names in columns
            if col not in macro_products:
                macro_products[col] = []
            macro_products[col].append(product_key)

macro_df = pd.DataFrame(list(macro_products.items()), columns=['Macro', 'Products'])
macro_df['Products'] = macro_df['Products'].apply(lambda x: ', '.join(sorted(x)))  
macro_df

,Macro,Products
0,MAB_ELE_SHP840,"P1, P11, P5, Sales_CPI"
1,PRI27276_org,"P1, P16, Sales_CPI"
2,PRO27826_org,"P1, P11, P3, P5, P9, Sales_CPI"
3,MAB_ELE_PRO276,"P1, P13"
4,MAB_ELE_SHP1100,P1
5,MAB_ELE_SHP392,P11
6,MAB_ELE_SHP276,P11
7,PRI27840_org,"P12, P13, P3, P8"
8,RohCOPPER1000_org,"P12, P8"
9,MAB_ELE_PRO156,"P12, P36, P5, P9"


Some products have macros in common wich is good for us in this next phase. 

In [27]:
# Step 1: Combine all dataframes while keeping index
for key in product_dfs:
    product_dfs[key].index = pd.to_datetime(product_dfs[key].index).strftime('%Y-%m-%d')

combined_macros = pd.DataFrame(index=next(iter(product_dfs.values())).index)
# Iterate through each product DataFrame
for product_name, df in product_dfs.items():
    # Get only macro columns (non-Sales)
    macros = df.drop(columns=[col for col in df.columns if 'sales' in col.lower()])
    
    # Add product prefix to avoid duplicate column names
    macros = macros.add_prefix(f'{product_name}_')
    
    # Merge with the combined DataFrame
    combined_macros = combined_macros.join(macros, how='left')

#combined_macros = combined_macros.dropna(axis=1, how='all')
combined_macros.columns = combined_macros.columns.str.replace(r'^(P\d+_|Sales_CPI_)', '', regex=True)
combined_macros = combined_macros.loc[:, ~combined_macros.columns.duplicated(keep='first')]

print(f"Final shape: {combined_macros.shape}")
combined_macros.head()

Final shape: (43, 27)


,MAB_ELE_SHP840,PRI27276_org,PRO27826_org,MAB_ELE_PRO276,MAB_ELE_SHP1100,MAB_ELE_SHP392,MAB_ELE_SHP276,PRI27840_org,RohCOPPER1000_org,MAB_ELE_PRO156,MAB_ELE_PRO756,PRO27756_org,PRO27392_org,PRO28380_org,PRO28276_org,PRO28826_org,MAB_ELE_SHP250,PRO271000_org,PRO27250_org,PRO28392_org,PRO27380_org,WKLWEUR840_org,PRO27276_org,MAB_ELE_SHP380,PRO27840_org,RohCRUDE_PETRO1000_org,PRO28250_org
month_year,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-10-01,127.8088,109.1196,118.6708,124.2279,130.9893,124.7933,137.7420,110.7004,82.5451,211.9558,109.0778,112.9708,112.3768,129.3892,127.4611,110.8935,122.4517,129.3258,97.8495,131.5038,93.0015,1.1484,118.2982,122.4569,112.8533,97.0723,114.7208
2018-11-01,117.6759,109.2248,120.4670,127.4041,132.9341,123.2899,142.7322,110.9940,82.2309,220.5197,112.2415,114.7360,111.9075,117.9902,132.9879,112.1199,115.9582,128.2362,91.1560,127.8808,84.1334,1.1367,117.1637,120.1320,113.1453,78.8412,104.8731
2018-12-01,123.2801,109.3301,105.3787,120.5186,131.2613,124.5084,141.4077,111.1622,80.6304,241.8469,115.4052,116.5012,112.5242,99.1917,137.3633,99.4464,115.1285,117.0435,78.0330,128.1257,64.8812,1.1384,89.6261,131.9361,111.8236,68.2686,104.9746
2019-01-01,111.0438,109.7510,107.1749,104.7763,113.0576,106.3039,118.7038,111.3861,78.8225,175.6681,109.6218,110.3546,95.9574,106.9911,104.4329,98.8331,108.3252,111.5222,81.9083,110.2009,81.5563,1.1416,105.4061,92.4188,109.4997,71.5875,95.2284
2019-02-01,116.7369,109.8562,110.6476,109.5970,117.7047,114.9963,122.6870,111.5611,83.6188,175.6681,103.8384,104.2080,107.3100,113.7905,112.8382,99.5486,108.9447,112.6667,80.3229,117.9765,83.4512,1.1351,103.6528,103.2142,110.8357,77.3440,100.1015


##### <span style="background-color:#000027; padding:5px; border-radius:5px;font-size:13pt">**4.1. Dickey-Fuller & Shapiro Test For Macro Variables**</span> <a id='Dickey-Fuller'></a> 

- Null Hypothesis: Time series is non-stationary
- Alternative Hypothesis: Time series is stationary.

A p-value below 0.05 indicates that we reject the null hypothesis, suggesting the time series is stationary.

In [28]:
def classify_variable(series):
    """Classifies variable based on normality and stationarity tests."""
    clean_series = series.dropna()
    
    # Initialize results (default: not normal, not stationary)
    is_normal = False
    is_stationary = False
    if len(clean_series) > 3:
        # Normality test (Shapiro-Wilk)
        _, shapiro_p = shapiro(clean_series)
        is_normal = shapiro_p > 0.05  # p > 0.05 → normal
        
        # Stationarity test (ADF)
        adf_result = adfuller(clean_series)
        adf_p = adf_result[1]
        is_stationary = adf_p < 0.05  # p < 0.05 → stationary
    
    return is_normal, is_stationary

results = []
for col in combined_macros.columns:
    is_normal, is_stationary = classify_variable(combined_macros[col])
    results.append({
        "Variable": col,
        "Is_Normal": is_normal,
        "Is_Stationary": is_stationary})
classification_results = pd.DataFrame(results).set_index("Variable")
classification_results

,Is_Normal,Is_Stationary
Variable,,
MAB_ELE_SHP840,False,False
PRI27276_org,False,False
PRO27826_org,False,True
MAB_ELE_PRO276,True,False
MAB_ELE_SHP1100,True,False
MAB_ELE_SHP392,True,False
MAB_ELE_SHP276,True,False
PRI27840_org,False,False
RohCOPPER1000_org,False,False


##### <span style="background-color:#000027; padding:5px; border-radius:5px;font-size:13pt">**4.2. Autocorrelation Function (ACF)**</span> <a id='Autocorrelation'></a> 

- The Autocorrelation Function (ACF) measures how correlated a time series is with itself over different time lags.
- This helps determine the q parameter in Moving Average (MA) models.

- If ACF values are near zero → The time series has no correlation (random).
- If ACF values drop slowly → There’s some correlation over time.
- If some lags have high ACF outside the confidence interval (blue region) → Suggests a pattern at that lag.

`Since our data is random noise, we expect the ACF to be close to zero for all lags except lag = 0 (which is always 1 because a series is perfectly correlated with itself).`

In [29]:
def plot_correlation_for_products(data, lags=30, plot_type='acf'):
    """
    Plots ACF or PACF with stem-like bars and constant confidence intervals
    
    Parameters:
    -----------
    lags : int
        Number of lags to compute
    plot_type : str ('acf' or 'pacf')
        Type of correlation plot to display
    """
    # Validate plot_type
    plot_type = plot_type.lower()
    if plot_type not in ['acf', 'pacf']:
        raise ValueError("plot_type must be either 'acf' or 'pacf'")
    
    fig = make_subplots(
        rows=1,
        cols=len(data.columns[:15]),
        subplot_titles=[f"{plot_type.upper()}: {product}" for product in data.columns[:15]],
        horizontal_spacing=0.01)

    for i, product in enumerate(data.columns[:15]):
        clean_data = data[product].fillna(0)
        
        if plot_type == 'acf':
            corr_values = acf(clean_data, nlags=lags, fft=True)
            # Standard error for ACF (Bartlett's formula)
            se = 2.24 / np.sqrt(len(clean_data)) # A 97.5% confidence interval means that (1 - 0.975) / 2 = 0.0125 (or 1.25%) of the probability is in each tail. The corresponding Z-score for 97.5% confidence is 2.24 (approximately).
        else:
            corr_values = pacf(clean_data, nlags=lags)
            # Standard error for PACF (1/sqrt(n))
            se = 2.24 / np.sqrt(len(clean_data))
            
        lags_range = list(range(lags+1))
        
        # Create stem-like effect (line + marker at top)
        fig.add_trace(
            go.Scatter(
                x=lags_range,y=corr_values,mode='lines+markers',line=dict(color='blue', width=1),
                marker=dict(color='blue', size=5),name=product,
                hovertemplate=f"Lag %{{x}}<br>{plot_type.upper()}: %{{y:.2f}}<extra></extra>"),
            row=1, col=i+1)
        
        # Add thin vertical lines from 0 to correlation value (stem effect)
        for lag, val in zip(lags_range, corr_values):
            fig.add_trace(
                go.Scatter(
                    x=[lag, lag],y=[0, val],mode='lines',line=dict(color='blue', width=1),
                    showlegend=False,hoverinfo='skip'),row=1, col=i+1)
        
        # Add constant confidence intervals
        fig.add_hline(y=se, line=dict(color='red', width=1, dash='dash'), 
                     row=1, col=i+1, opacity=0.7)
        fig.add_hline(y=-se, line=dict(color='red', width=1, dash='dash'), 
                     row=1, col=i+1, opacity=0.7)
        
        # Shaded confidence interval area
        fig.add_hrect(y0=-se, y1=se, 
                     line_width=0, fillcolor='rgba(100,100,100,0.2)', 
                     row=1, col=i+1)

    fig.update_layout(
        title_text=f"{plot_type.upper()} by Product (97.5% Confidence)",
        height=500,
        width=500*len(data.columns[:15]),
        showlegend=False,
        margin=dict(l=20, r=20))
    
    # Uniform axes settings
    fig.update_yaxes(range=[-1.1, 1.1], title_text=plot_type.upper())
    fig.update_xaxes(title_text="Lag")
    
    fig.show()

In [30]:
plot_correlation_for_products(combined_macros, plot_type='acf')

##### <span style="background-color:#000027; padding:5px; border-radius:5px;font-size:13pt">**4.3. Partial Autocorrelation Function (PACF)**</span> <a id='PACF'></a> 

- PACF at Lag 0 = 1. This is always 1 because a time series is perfectly correlated with itself.
- The first significant spike in PACF suggests the order (p) of the AR model. If PACF cuts off after lag k, then p = k. The rest of the lags should ideally fall within the confidence interval (blue shaded area).

Comparison with ACF:

    - ACF shows how a value is correlated with past values directly and indirectly.
    - PACF shows how a value is correlated only with its direct past, ignoring indirect effects.

- If all PACF values are within the blue region, then there is no AR structure in the data, and an AR model may not be needed.
- Slow decay in ACF? → Data is non-stationary

In [31]:
plot_correlation_for_products(combined_macros, lags=21, plot_type='pacf')

We checked for **normality** and **stationarity** because many traditional time-series models, such as **ARIMA and AutoARIMA**, assume that the residuals follow a normal distribution and that the data is stationary. If these assumptions are violated, the model’s accuracy and reliability decrease significantly.  

- Many variables exhibited **non-normal distributions**, meaning their error terms may not follow a Gaussian pattern. This is problematic for models like ARIMA, which rely on normally distributed residuals for optimal performance.  
- A lot of variables were also **non-stationary**, meaning they had trends, seasonality, or structural changes that could not be directly modeled without transformations like differencing.  
- The **Partial Autocorrelation Function (PACF) plots** were complex and difficult to interpret, making it challenging to manually determine the optimal ARIMA parameters.  

Given these challenges, we decided to use **TimeGPT**:  

1. **Handles Non-Normality & Non-Stationarity** – Unlike ARIMA, which requires data transformations (e.g., differencing and Box-Cox transformations), TimeGPT can work directly with raw data.  
2. **No Need for Manual Parameter Tuning** – ARIMA requires selecting (p, d, q) values, which is difficult when dealing with complex PACF/ACF plots. TimeGPT operates in a **zero-shot** manner, meaning it **does not require manual tuning** to produce accurate forecasts.  
3. **Better at Handling Volatility** – Many of the variables exhibit fluctuations that ARIMA struggles with, while TimeGPT is trained on **100B+ time-series points**, making it inherently more robust to volatility.  

imeGPT is the **most practical and scientifically reliable choice** for this forecasting task. While traditional statistical models may work in controlled cases, the complexity of my dataset demands a more advanced AI-based approach.  

##### <span style="background-color:#000027; padding:5px; border-radius:5px;font-size:13pt">**4.4. Macro Prediction**</span> <a id='macro-prediction'></a> 

In [32]:
# Get unique dates from test set
test_dates = test['Month Year'].unique()

# Create DataFrame with test dates as index (named "Month Year")
test_macros = pd.DataFrame(index=pd.Index(test_dates, name="Month Year"),columns=combined_macros.columns)
test_macros = test_macros.fillna(np.nan)
test_macros.index = pd.to_datetime(test_macros.index)

# Sort chronologically 
test_macros = test_macros.sort_index()

print(f"Test macros shape: {test_macros.shape}")
test_macros

WARNING - (py.warnings._showwarnmsg) - C:\Users\catar\AppData\Local\Temp\ipykernel_524\2288804386.py:6: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`




Test macros shape: (10, 27)


,MAB_ELE_SHP840,PRI27276_org,PRO27826_org,MAB_ELE_PRO276,MAB_ELE_SHP1100,MAB_ELE_SHP392,MAB_ELE_SHP276,PRI27840_org,RohCOPPER1000_org,MAB_ELE_PRO156,MAB_ELE_PRO756,PRO27756_org,PRO27392_org,PRO28380_org,PRO28276_org,PRO28826_org,MAB_ELE_SHP250,PRO271000_org,PRO27250_org,PRO28392_org,PRO27380_org,WKLWEUR840_org,PRO27276_org,MAB_ELE_SHP380,PRO27840_org,RohCRUDE_PETRO1000_org,PRO28250_org
Month Year,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-07-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
def forecast_macros(combined_macros, forecast_horizon=10, nixtla_client=None):

    if nixtla_client is None:
        raise ValueError("Initialized NixtlaClient is required")
    
    # Prepare output DataFrame with future dates
    last_date = pd.to_datetime(combined_macros.index[-1])
    forecast_dates = pd.date_range(start=last_date + pd.offsets.MonthBegin(1), periods=forecast_horizon, freq='MS')
    predictions = pd.DataFrame(index=forecast_dates, columns=combined_macros.columns)
    
    # Forecast each macro variable
    for macro in combined_macros.columns:
        print(f"Forecasting {macro}...")
        
        try:
            # Prepare data (TimeGPT expects 'ds' and 'y' columns)
            history = combined_macros[[macro]].reset_index()
            history.columns = ['ds', 'y']

            # forecast
            forecast = nixtla_client.forecast(df=history, h=forecast_horizon, freq='MS', time_col='ds', target_col='y')
            predictions[macro] = forecast['TimeGPT'].values
            
        except Exception as e:
            print(f"⚠️ Failed on {macro}: {str(e)}")
            predictions[macro] = np.nan
    
    predictions.index.name = 'Month Year'
    return predictions

In [34]:
#macro_predictions.to_csv('macro_predictions.csv')
macro_preds = pd.read_csv('macro_predictions.csv', index_col='Month Year', parse_dates=True)

In [35]:
macro_preds

,MAB_ELE_SHP840,PRI27276_org,PRO27826_org,MAB_ELE_PRO276,MAB_ELE_SHP1100,MAB_ELE_SHP392,MAB_ELE_SHP276,PRI27840_org,RohCOPPER1000_org,MAB_ELE_PRO156,MAB_ELE_PRO756,PRO27756_org,PRO27392_org,PRO28380_org,PRO28276_org,PRO28826_org,MAB_ELE_SHP250,PRO271000_org,PRO27250_org,PRO28392_org,PRO27380_org,WKLWEUR840_org,PRO27276_org,MAB_ELE_SHP380,PRO27840_org,RohCRUDE_PETRO1000_org,PRO28250_org
Month Year,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-05-01,159.8134,122.3341,115.9040,101.2446,134.5055,114.4035,129.8432,139.9015,140.0734,283.1652,96.9175,93.8987,99.8816,112.4203,101.3997,88.7580,110.5841,132.7095,77.7203,118.6531,86.0224,1.0736,102.3751,129.5264,112.9188,135.8693,92.2586
2022-06-01,173.6270,122.9719,125.3562,113.2584,153.1603,132.9693,145.7785,141.4168,140.2763,307.8518,96.2652,90.7787,115.7264,121.0243,116.7264,98.7261,136.3268,142.1336,91.9900,136.6734,92.8159,1.0716,107.6265,147.4014,112.9100,141.2402,111.3546
2022-07-01,159.8994,123.6769,123.8094,109.8021,141.9189,127.0304,140.1388,142.8848,141.9527,269.5245,99.1147,95.5829,110.5113,121.1212,111.4507,93.4788,113.9221,138.8142,84.7953,130.1510,91.9484,1.0690,107.0694,144.1310,113.3063,145.0944,102.3235
2022-08-01,166.3691,124.1967,122.5133,101.2170,126.8985,117.5288,129.8402,143.8159,141.9732,257.5743,102.3551,101.4852,95.5346,75.0991,101.0217,89.5251,104.7270,132.1480,66.3808,120.9310,57.2111,1.0685,103.4843,104.9816,114.0220,146.4789,72.9638
2022-09-01,171.0225,124.5127,130.8887,113.9225,152.2086,132.5392,147.7511,145.0316,142.4949,265.2198,104.7845,106.8750,112.9201,122.8645,115.4725,98.7456,125.4079,145.8849,91.9670,136.9477,97.7606,1.0653,110.3740,146.2103,113.4420,147.0982,108.4830
2022-10-01,165.1973,124.9870,125.7549,111.7053,149.1204,127.1631,142.2516,146.3284,144.2400,278.9898,106.9761,111.6346,108.5995,122.8734,112.5937,94.5848,125.1931,144.6288,91.7211,133.1175,96.1748,1.0608,109.3358,147.5842,115.5943,150.0592,107.0753
2022-11-01,158.8748,125.1497,130.2161,116.2251,155.0913,131.9138,148.9787,147.4518,144.7244,291.4109,108.7037,114.7831,110.3707,120.8869,119.3199,98.8176,128.0164,146.1588,89.9914,138.3113,92.9822,1.0576,110.6498,149.3881,115.5804,151.3860,104.1748
2022-12-01,167.0725,125.5513,124.3347,116.4987,159.5794,136.6096,152.8668,148.1660,146.2719,301.0567,109.8227,118.0444,114.6863,110.1409,124.2850,96.0815,133.9264,139.9869,84.2375,142.4281,81.2518,1.0583,101.9211,158.6928,114.2411,151.8687,108.9752
2023-01-01,151.9106,125.8922,121.0189,98.9604,130.6706,122.2128,128.0293,148.8286,147.2713,237.1610,107.0766,113.0140,103.1821,102.6021,96.4502,90.3143,112.2637,129.8708,81.0084,127.9608,83.8557,1.0591,102.5711,115.7837,115.5560,158.1927,91.7178


In [36]:
print(f"Shape: {macro_preds.shape}")
print(f"Total NaN values: {macro_preds.isna().sum().sum()}")

Shape: (10, 27)
Total NaN values: 0


#### <span style="background-color:#000027; padding:5px; border-radius:5px;">**5. Final Modeling**</span> <a id='final'></a>  

Click [here](#table-of-contents) ⬆️ to return to the Index.


In [37]:
def forecast_ma_columns(df, forecast_horizon=10, nixtla_client=nixtla_client):
    if nixtla_client is None:
        raise ValueError("Initialized NixtlaClient is required")

    # Ensure datetime index
    df.index = pd.to_datetime(df.index)

    # Filter only columns that contain '_ma_'
    ma_columns = [col for col in df.columns if '_ma_' in col]
    if not ma_columns:
        raise ValueError("No moving average (_ma_) columns found in the dataset")

    # Prepare output DataFrame
    last_date = df.index[-1]
    forecast_dates = pd.date_range(start=last_date + pd.offsets.MonthBegin(1), periods=forecast_horizon, freq='MS')
    predictions = pd.DataFrame(index=forecast_dates)

    for col in ma_columns:
        print(f"Forecasting {col}...")

        try:
            # Prepare data for TimeGPT
            history = df[[col]].reset_index()
            history.columns = ['ds', 'y']

            # Forecast
            forecast = nixtla_client.forecast(df=history, h=forecast_horizon, freq='MS', time_col='ds', target_col='y')
            predictions[col] = forecast['TimeGPT'].values

        except Exception as e:
            print(f"⚠️ Failed to forecast {col}: {str(e)}")
            predictions[col] = np.nan

    return predictions

##### <span style="background-color:#000027; padding:5px; border-radius:5px;font-size:13pt">**5.1. P1: ARIMA  ['PRI27276_org', 'PRO27826_org', 'MAB_ELE_PRO276', 'MAB_ELE_SHP1100']**</span> <a id='p1'></a> 

In [38]:
test_1 = pd.DataFrame(index=test[test['Mapped_GCK'] == '1']['Month Year'], columns=lagged_product_dfs['P1'][['Sales','PRI27276_org', 'PRO27826_org', 'MAB_ELE_PRO276', 'MAB_ELE_SHP1100']].columns) #we need the lags
test_1

,Sales,PRI27276_org,PRO27826_org,MAB_ELE_PRO276,MAB_ELE_SHP1100
Month Year,,,,,
2022-05-01,NaN,NaN,NaN,NaN,NaN
2022-06-01,NaN,NaN,NaN,NaN,NaN
2022-07-01,NaN,NaN,NaN,NaN,NaN
2022-08-01,NaN,NaN,NaN,NaN,NaN
2022-09-01,NaN,NaN,NaN,NaN,NaN
2022-10-01,NaN,NaN,NaN,NaN,NaN
2022-11-01,NaN,NaN,NaN,NaN,NaN
2022-12-01,NaN,NaN,NaN,NaN,NaN
2023-01-01,NaN,NaN,NaN,NaN,NaN


In [39]:
macro_preds[['PRI27276_org', 'PRO27826_org', 'MAB_ELE_PRO276', 'MAB_ELE_SHP1100']]

,PRI27276_org,PRO27826_org,MAB_ELE_PRO276,MAB_ELE_SHP1100
Month Year,,,,
2022-05-01,122.3341,115.9040,101.2446,134.5055
2022-06-01,122.9719,125.3562,113.2584,153.1603
2022-07-01,123.6769,123.8094,109.8021,141.9189
2022-08-01,124.1967,122.5133,101.2170,126.8985
2022-09-01,124.5127,130.8887,113.9225,152.2086
2022-10-01,124.9870,125.7549,111.7053,149.1204
2022-11-01,125.1497,130.2161,116.2251,155.0913
2022-12-01,125.5513,124.3347,116.4987,159.5794
2023-01-01,125.8922,121.0189,98.9604,130.6706


In [40]:
test_1 = test_1.merge(macro_preds[['PRI27276_org', 'PRO27826_org', 'MAB_ELE_PRO276', 'MAB_ELE_SHP1100']], 
                       left_index=True, 
                       right_index=True, 
                       how='left')
test_1 = test_1[['Sales', 'PRI27276_org_y', 'PRO27826_org_y', 'MAB_ELE_PRO276_y', 'MAB_ELE_SHP1100_y']]
test_1.columns = ['Sales','PRI27276_org', 'PRO27826_org', 'MAB_ELE_PRO276', 'MAB_ELE_SHP1100'] 

In [41]:
df_1 = product_dfs['P1'][['Sales', 'PRI27276_org', 'PRO27826_org', 'MAB_ELE_PRO276', 'MAB_ELE_SHP1100']].copy()

In [42]:
predictions1 = forecast_sarimax(
    df=df_1,
    test_dataset=test_1,
    product_id='P1',
    feature_set=['PRI27276_org', 'PRO27826_org', 'MAB_ELE_PRO276', 'MAB_ELE_SHP1100'],
    target_col='Sales',
    horizon=10,
    order=(1, 0, 0),  # Example order (p,d,q)
    seasonal_order=(0, 0, 0, 0), # Example seasonal order (P,D,Q,s)
    winsorize=True
    )

Processing SARIMAX for P1...
Sales: Winsorized Bounds -> Lower = 14314884.58, Upper = 45669226.10


In [43]:
test_1['Sales'] = predictions1.values

In [44]:
final_1 = pd.concat([df_1, test_1])

##### <span style="background-color:#000027; padding:5px; border-radius:5px;font-size:13pt">**5.2. P3: XGBoost ['P3_lag_12', 'P3_ma_12']**</span> <a id='p3'></a> 

In [45]:
test_3 = pd.DataFrame(index=test[test['Mapped_GCK'] == '3']['Month Year'], columns=lagged_product_dfs['P3'][['Sales','P3_lag_12', 'P3_ma_12']].columns) #we need the lags
test_3

,Sales,P3_lag_12,P3_ma_12
Month Year,,,
2022-05-01,NaN,NaN,NaN
2022-06-01,NaN,NaN,NaN
2022-07-01,NaN,NaN,NaN
2022-08-01,NaN,NaN,NaN
2022-09-01,NaN,NaN,NaN
2022-10-01,NaN,NaN,NaN
2022-11-01,NaN,NaN,NaN
2022-12-01,NaN,NaN,NaN
2023-01-01,NaN,NaN,NaN


In [46]:
df_3 = lagged_product_dfs['P3'][['Sales', 'P3_lag_12', 'P3_ma_12']].copy()
test_index = pd.date_range(start="2022-05-01", periods=10, freq="MS")
df_extended = pd.concat([df_3, test_3])

WARNING - (py.warnings._showwarnmsg) - C:\Users\catar\AppData\Local\Temp\ipykernel_524\1790518331.py:3: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.




In [47]:
df_extended['P3_lag_12'] = df_extended['Sales'].shift(12)

In [48]:
# Extract only the last 10 rows (test set) with recalculated lags
test_3 = df_extended.loc[test_index, ['Sales', 'P3_lag_12', 'P3_ma_12']]

In [49]:
forecast_results = forecast_ma_columns(df_3, forecast_horizon=10)

INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Querying model metadata...


Forecasting P3_ma_12...


INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


In [50]:
for col in forecast_results.columns:
    if col in test_3.columns:
        test_3.loc[test_3.index.isin(forecast_results.index), col] = forecast_results[col]

In [51]:
test_3

,Sales,P3_lag_12,P3_ma_12
2022-05-01,NaN,10565829.4747,14631796.0000
2022-06-01,NaN,14046254.8601,14700870.0000
2022-07-01,NaN,13739993.8782,14800271.0000
2022-08-01,NaN,12819257.1081,14838973.0000
2022-09-01,NaN,16477951.3259,14888258.0000
2022-10-01,NaN,14390316.8711,14994135.0000
2022-11-01,NaN,19527726.7433,15072822.0000
2022-12-01,NaN,12958106.0653,15121014.0000
2023-01-01,NaN,14469514.4686,15084094.0000
2023-02-01,NaN,16605130.5182,15158293.0000


In [52]:
forecast_results = forecast_xgboost(
    df=df_3,  # Full dataset with target (Sales)
    test_dataset=test_3,  # Future exogenous variables only
    product_id='P3',
    feature_set=['P3_lag_12', 'P3_ma_12'],  # Features available in test
    best_params={'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 2, 'min_child_weight': 3, 'n_estimators': 50, 'subsample': 0.9},
    target_col='Sales',
    horizon=10,
    winsorize=True)

Processing XGBoost for P3 with params: {'objective': 'reg:squarederror', 'random_state': 42, 'n_jobs': 1, 'tree_method': 'hist', 'enable_categorical': False, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1, 'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 2, 'min_child_weight': 3, 'n_estimators': 50, 'subsample': 0.9}
Sales: Winsorized Bounds -> Lower = 5610461.53, Upper = 18650947.88
Iteration 1: Prediction=13512219.0, Inputs=[[10565829.47466 14631796.     ]]
Iteration 2: Prediction=14214265.0, Inputs=[[14046254.86005 14700870.     ]]
Iteration 3: Prediction=13830102.0, Inputs=[[13739993.87816 14800271.     ]]
Iteration 4: Prediction=13647657.0, Inputs=[[12819257.1081 14838973.    ]]
Iteration 5: Prediction=16661674.0, Inputs=[[16477951.32588 14888258.     ]]
Iteration 6: Prediction=14214265.0, Inputs=[[14390316.87114 14994135.     ]]
Iteration 7: Prediction=16661674.0, Inputs=[[19527726.74325 15072822.     ]]
Iteration 8: Prediction=13647657.0, Inputs=[[12958106.06526 15121014

In [53]:
test_3.loc[test_3.index[:len(forecast_results)], "Sales"] = forecast_results

In [54]:
test_3

,Sales,P3_lag_12,P3_ma_12
2022-05-01,13512219.0000,10565829.4747,14631796.0000
2022-06-01,14214265.0000,14046254.8601,14700870.0000
2022-07-01,13830102.0000,13739993.8782,14800271.0000
2022-08-01,13647657.0000,12819257.1081,14838973.0000
2022-09-01,16661674.0000,16477951.3259,14888258.0000
2022-10-01,14214265.0000,14390316.8711,14994135.0000
2022-11-01,16661674.0000,19527726.7433,15072822.0000
2022-12-01,13647657.0000,12958106.0653,15121014.0000
2023-01-01,15536638.0000,14469514.4686,15084094.0000
2023-02-01,16661674.0000,16605130.5182,15158293.0000


In [55]:
final_3 = pd.concat([df_3, test_3])

##### <span style="background-color:#000027; padding:5px; border-radius:5px;font-size:13pt">**5.23. P4: ARIMA ['PRO27380_org', 'WKLWEUR840_org', 'PRO27276_org', 'MAB_ELE_SHP380']**</span> <a id='p3'></a> 

In [56]:
test_4 = pd.DataFrame(index=test[test['Mapped_GCK'] == '4']['Month Year'], columns=product_dfs['P4'][['Sales','PRO27380_org', 'WKLWEUR840_org', 'PRO27276_org', 'MAB_ELE_SHP380']].columns)
test_4

,Sales,PRO27380_org,WKLWEUR840_org,PRO27276_org,MAB_ELE_SHP380
Month Year,,,,,
2022-05-01,NaN,NaN,NaN,NaN,NaN
2022-06-01,NaN,NaN,NaN,NaN,NaN
2022-07-01,NaN,NaN,NaN,NaN,NaN
2022-08-01,NaN,NaN,NaN,NaN,NaN
2022-09-01,NaN,NaN,NaN,NaN,NaN
2022-10-01,NaN,NaN,NaN,NaN,NaN
2022-11-01,NaN,NaN,NaN,NaN,NaN
2022-12-01,NaN,NaN,NaN,NaN,NaN
2023-01-01,NaN,NaN,NaN,NaN,NaN


In [57]:
macro_preds[['PRO27380_org', 'WKLWEUR840_org', 'PRO27276_org', 'MAB_ELE_SHP380']]

test_4 = test_4.merge(
    macro_preds[['PRO27380_org', 'WKLWEUR840_org', 'PRO27276_org', 'MAB_ELE_SHP380']],
    left_index=True,
    right_index=True,
    how='left'
)

test_4 = test_4[['Sales', 'PRO27380_org_y', 'WKLWEUR840_org_y', 'PRO27276_org_y', 'MAB_ELE_SHP380_y']]
test_4.columns = ['Sales','PRO27380_org', 'WKLWEUR840_org', 'PRO27276_org', 'MAB_ELE_SHP380'] 

In [58]:
df_4 = product_dfs['P4'][['Sales','PRO27380_org', 'WKLWEUR840_org', 'PRO27276_org', 'MAB_ELE_SHP380']].copy()

In [59]:
predictions4 = forecast_sarimax(
    df=df_4,
    test_dataset=test_4,
    product_id='P4',
    feature_set=['PRO27380_org', 'WKLWEUR840_org', 'PRO27276_org', 'MAB_ELE_SHP380'],
    target_col='Sales',
    horizon=10,
    order=(0, 0, 0),  # Example order (p,d,q)
    seasonal_order=(1, 0, 0, 6), # Example seasonal order (P,D,Q,s)
    winsorize=False
    )

Processing SARIMAX for P4...


In [60]:
test_4['Sales'] = predictions4.values

In [61]:
final_4 = pd.concat([df_4, test_4])

##### <span style="background-color:#000027; padding:5px; border-radius:5px;font-size:13pt">**5.4. P5: XGBoost ['PRO27826_org']**</span> <a id='p5'></a> 

In [62]:
test_5 = pd.DataFrame(index=test[test['Mapped_GCK'] == '5']['Month Year'], columns=product_dfs['P5'][['Sales','PRO27826_org']].columns)
test_5

,Sales,PRO27826_org
Month Year,,
2022-05-01,NaN,NaN
2022-06-01,NaN,NaN
2022-07-01,NaN,NaN
2022-08-01,NaN,NaN
2022-09-01,NaN,NaN
2022-10-01,NaN,NaN
2022-11-01,NaN,NaN
2022-12-01,NaN,NaN
2023-01-01,NaN,NaN


In [63]:
macro_preds[['PRO27826_org']]

,PRO27826_org
Month Year,
2022-05-01,115.9040
2022-06-01,125.3562
2022-07-01,123.8094
2022-08-01,122.5133
2022-09-01,130.8887
2022-10-01,125.7549
2022-11-01,130.2161
2022-12-01,124.3347
2023-01-01,121.0189


In [64]:
test_5 = test_5.merge(macro_preds[['PRO27826_org']], 
                       left_index=True, 
                       right_index=True, 
                       how='left')
test_5 = test_5[['Sales', 'PRO27826_org_y']]
test_5.columns = ['Sales','PRO27826_org'] 

In [65]:
df_5 = product_dfs['P5'][['Sales', 'PRO27826_org']].copy()
df_extended = pd.concat([df_5, test_5])

WARNING - (py.warnings._showwarnmsg) - C:\Users\catar\AppData\Local\Temp\ipykernel_524\742315741.py:2: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.




In [67]:
forecast_results_p5 = forecast_xgboost(
    df=df_5,  # Full dataset with target (Sales)
    test_dataset=test_5,  # Future exogenous variables only
    product_id='P5',
    feature_set=['PRO27826_org'],  # Features available in test
    best_params={'colsample_bytree': 0.9, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 50, 'subsample': 0.9}	,
    target_col='Sales',
    horizon=10,
    winsorize=False)

Processing XGBoost for P5 with params: {'objective': 'reg:squarederror', 'random_state': 42, 'n_jobs': 1, 'tree_method': 'hist', 'enable_categorical': False, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1, 'colsample_bytree': 0.9, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 50, 'subsample': 0.9}
Iteration 1: Prediction=9232623.0, Inputs=[[115.904015]]
Iteration 2: Prediction=9480693.0, Inputs=[[125.3562]]
Iteration 3: Prediction=11648594.0, Inputs=[[123.80945]]
Iteration 4: Prediction=12159371.0, Inputs=[[122.513275]]
Iteration 5: Prediction=14009545.0, Inputs=[[130.88869]]
Iteration 6: Prediction=9258901.0, Inputs=[[125.75492]]
Iteration 7: Prediction=12469747.0, Inputs=[[130.21606]]
Iteration 8: Prediction=9480693.0, Inputs=[[124.33473]]
Iteration 9: Prediction=8445892.0, Inputs=[[121.01893]]
Iteration 10: Prediction=12159371.0, Inputs=[[122.64957]]


In [68]:
test_5.loc[test_5.index[:len(forecast_results_p5)], "Sales"] = forecast_results_p5

In [69]:
test_5

,Sales,PRO27826_org
Month Year,,
2022-05-01,9232623.0000,115.9040
2022-06-01,9480693.0000,125.3562
2022-07-01,11648594.0000,123.8094
2022-08-01,12159371.0000,122.5133
2022-09-01,14009545.0000,130.8887
2022-10-01,9258901.0000,125.7549
2022-11-01,12469747.0000,130.2161
2022-12-01,9480693.0000,124.3347
2023-01-01,8445892.0000,121.0189


In [70]:
final_5 = pd.concat([df_5, test_5])

##### <span style="background-color:#000027; padding:5px; border-radius:5px;font-size:13pt">**5.5. P6: ARIMA ['PRO27840_org', 'RohCRUDE_PETRO1000_org']**</span> <a id='p6'></a> 

In [71]:
test_6 = pd.DataFrame(index=test[test['Mapped_GCK'] == '6']['Month Year'], columns=product_dfs['P6'][['Sales', 'PRO27840_org', 'RohCRUDE_PETRO1000_org']].columns)
test_6

,Sales,PRO27840_org,RohCRUDE_PETRO1000_org
Month Year,,,
2022-05-01,NaN,NaN,NaN
2022-06-01,NaN,NaN,NaN
2022-07-01,NaN,NaN,NaN
2022-08-01,NaN,NaN,NaN
2022-09-01,NaN,NaN,NaN
2022-10-01,NaN,NaN,NaN
2022-11-01,NaN,NaN,NaN
2022-12-01,NaN,NaN,NaN
2023-01-01,NaN,NaN,NaN


In [72]:
macro_preds[[ 'PRO27840_org', 'RohCRUDE_PETRO1000_org']]

,PRO27840_org,RohCRUDE_PETRO1000_org
Month Year,,
2022-05-01,112.9188,135.8693
2022-06-01,112.9100,141.2402
2022-07-01,113.3063,145.0944
2022-08-01,114.0220,146.4789
2022-09-01,113.4420,147.0982
2022-10-01,115.5943,150.0592
2022-11-01,115.5804,151.3860
2022-12-01,114.2411,151.8687
2023-01-01,115.5560,158.1927


In [73]:
test_6 = test_6.merge(macro_preds[['PRO27840_org', 'RohCRUDE_PETRO1000_org']], 
                       left_index=True, 
                       right_index=True, 
                       how='left')
test_6 = test_6[['Sales', 'PRO27840_org_y', 'RohCRUDE_PETRO1000_org_y']]
test_6.columns = ['Sales','PRO27840_org', 'RohCRUDE_PETRO1000_org'] 

In [74]:
df_6 = product_dfs['P6'][['Sales','PRO27840_org', 'RohCRUDE_PETRO1000_org']].copy()

In [75]:
predictions6 = forecast_sarimax(
    df=df_6,
    test_dataset=test_6,
    product_id='P6',
    feature_set=['PRO27840_org', 'RohCRUDE_PETRO1000_org'],
    target_col='Sales',
    horizon=10,
    order=(1, 0, 0),  # Example order (p,d,q)
    seasonal_order=(0, 0, 0, 0), # Example seasonal order (P,D,Q,s)
    winsorize=False
    )

Processing SARIMAX for P6...


In [76]:
test_6['Sales'] = predictions6.values
final_6 = pd.concat([df_6, test_6])

##### <span style="background-color:#000027; padding:5px; border-radius:5px;font-size:13pt">**5.6. P8: XGBoost ['P8_lag_10', 'PRI27840_org', 'RohCOPPER1000_org', 'P8_ma_1', 'P8_ma_2', 'P8_ma_5', 'P8_ma_6', 'P8_ma_10']**</span> <a id='p8'></a> 

In [77]:
test_8 = pd.DataFrame(index=test[test['Mapped_GCK'] == '8']['Month Year'], columns=lagged_product_dfs['P8'][['Sales','P8_lag_10', 'PRI27840_org', 'RohCOPPER1000_org', 'P8_ma_1', 'P8_ma_2', 'P8_ma_5', 'P8_ma_6', 'P8_ma_10']].columns) #we need the lags
test_8

,Sales,P8_lag_10,PRI27840_org,RohCOPPER1000_org,P8_ma_1,P8_ma_2,P8_ma_5,P8_ma_6,P8_ma_10
Month Year,,,,,,,,,
2022-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-07-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
macro_preds[['PRI27840_org', 'RohCOPPER1000_org']]

,PRI27840_org,RohCOPPER1000_org
Month Year,,
2022-05-01,139.9015,140.0734
2022-06-01,141.4168,140.2763
2022-07-01,142.8848,141.9527
2022-08-01,143.8159,141.9732
2022-09-01,145.0316,142.4949
2022-10-01,146.3284,144.2400
2022-11-01,147.4518,144.7244
2022-12-01,148.1660,146.2719
2023-01-01,148.8286,147.2713


In [79]:
test_8 = test_8.merge(macro_preds[['PRI27840_org', 'RohCOPPER1000_org']], 
                       left_index=True, 
                       right_index=True, 
                       how='left')

test_8 = test_8[['Sales', 'P8_lag_10','PRI27840_org_y',	'RohCOPPER1000_org_y','P8_ma_1','P8_ma_2', 'P8_ma_5',	'P8_ma_6','P8_ma_10']]
test_8.columns = ['Sales', 'P8_lag_10','PRI27840_org',	'RohCOPPER1000_org','P8_ma_1','P8_ma_2', 'P8_ma_5',	'P8_ma_6','P8_ma_10'] 

In [80]:
df_8 = lagged_product_dfs['P8'][['Sales', 'P8_lag_10','PRI27840_org','RohCOPPER1000_org','P8_ma_1','P8_ma_2', 'P8_ma_5','P8_ma_6','P8_ma_10']].copy()

In [81]:
test_index = pd.date_range(start="2022-05-01", periods=10, freq="MS")
df_extended = pd.concat([df_8, test_8])

WARNING - (py.warnings._showwarnmsg) - C:\Users\catar\AppData\Local\Temp\ipykernel_524\3888404637.py:2: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.




In [82]:
df_extended['P8_lag_10'] = df_extended['Sales'].shift(10) # we have these lags

In [83]:
# Extract only the last 10 rows (test set) with recalculated lags
test_8 = df_extended.loc[test_index, ['Sales', 'P8_lag_10', 'PRI27840_org', 'RohCOPPER1000_org', 'P8_ma_1', 'P8_ma_2', 'P8_ma_5', 'P8_ma_6', 'P8_ma_10']]

In [84]:
forecast_results_ma_8 = forecast_ma_columns(df_8, forecast_horizon=10)

INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


Forecasting P8_ma_1...


INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


Forecasting P8_ma_2...


INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


Forecasting P8_ma_5...


INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


Forecasting P8_ma_6...


INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


Forecasting P8_ma_10...


In [85]:
for col in forecast_results_ma_8.columns:
    if col in test_8.columns:
        test_8.loc[test_8.index.isin(forecast_results_ma_8.index), col] = forecast_results_ma_8[col]

In [86]:
forecast_results_p8 = forecast_xgboost(
    df=df_8,  # Full dataset with target (Sales)
    test_dataset=test_8,  # Future exogenous variables only
    product_id='P8',
    feature_set=['P8_lag_10', 'PRI27840_org', 'RohCOPPER1000_org', 'P8_ma_1', 'P8_ma_2', 'P8_ma_5', 'P8_ma_6', 'P8_ma_10'],  # Features available in test
    best_params={'colsample_bytree': 1.0, 'learning_rate': 0.05, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 0.9},
    target_col='Sales',
    horizon=10,
    winsorize=True)

Processing XGBoost for P8 with params: {'objective': 'reg:squarederror', 'random_state': 42, 'n_jobs': 1, 'tree_method': 'hist', 'enable_categorical': False, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1, 'colsample_bytree': 1.0, 'learning_rate': 0.05, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 0.9}
Sales: Winsorized Bounds -> Lower = 125360.43, Upper = 2566675.67
Iteration 1: Prediction=1332805.0, Inputs=[[1.24861137e+06 1.39901520e+02 1.40073400e+02 1.45111410e+06
  1.55729790e+06 1.74616200e+06 1.68652150e+06 1.55598120e+06]]
Iteration 2: Prediction=1428448.875, Inputs=[[1.13164829e+06 1.41416800e+02 1.40276340e+02 1.63009690e+06
  1.67830750e+06 1.87361800e+06 1.74023020e+06 1.60496780e+06]]
Iteration 3: Prediction=1742106.125, Inputs=[[2.1137365e+06 1.4288481e+02 1.4195267e+02 1.6548449e+06 1.6650968e+06
  1.8689952e+06 1.8037981e+06 1.6206019e+06]]
Iteration 4: Prediction=1625787.5, Inputs=[[3.7164442e+05 1.4381592e+02 1.4197324e+02 1.6610072e+06 1.745

In [87]:
test_8.loc[test_8.index[:len(forecast_results_p8)], "Sales"] = forecast_results_p8

In [88]:
test_8

,Sales,P8_lag_10,PRI27840_org,RohCOPPER1000_org,P8_ma_1,P8_ma_2,P8_ma_5,P8_ma_6,P8_ma_10
2022-05-01,1332805.0000,1248611.3737,139.9015,140.0734,1451114.1000,1557297.9000,1746162.0000,1686521.5000,1555981.2000
2022-06-01,1428448.8750,1131648.2896,141.4168,140.2763,1630096.9000,1678307.5000,1873618.0000,1740230.2000,1604967.8000
2022-07-01,1742106.1250,2113736.5004,142.8848,141.9527,1654844.9000,1665096.8000,1868995.2000,1803798.1000,1620601.9000
2022-08-01,1625787.5000,371644.4196,143.8159,141.9732,1661007.2000,1745315.8000,1893104.9000,1818436.6000,1685825.9000
2022-09-01,1628165.7500,1192052.9434,145.0316,142.4949,1943098.4000,1927054.2000,2037475.1000,1915258.8000,1754554.2000
2022-10-01,1228419.8750,2779823.5087,146.3284,144.2400,1247144.8000,1650774.8000,1968863.2000,1895365.2000,1756134.9000
2022-11-01,1628165.7500,1270910.9851,147.4518,144.7244,1660361.2000,1391248.1000,2021325.8000,1951649.8000,1825914.8000
2022-12-01,1628165.7500,1313521.6946,148.1660,146.2719,2005162.1000,1869363.8000,2157544.0000,2067021.4000,1905558.0000
2023-01-01,1361951.0000,2025275.5308,148.8286,147.2713,1451628.0000,1801201.0000,2124943.0000,2042916.6000,1912425.5000
2023-02-01,1721602.1250,1453808.4045,149.3251,148.5520,1818319.8000,1723008.0000,2096474.1000,2109418.5000,1962208.6000


In [89]:
final_8 = pd.concat([df_8, test_8])

##### <span style="background-color:#000027; padding:5px; border-radius:5px;font-size:13pt">**5.7. P9: XGBoost ['P9_lag_12', 'PRO27826_org', 'PRO271000_org', 'PRO28250_org', 'MAB_ELE_PRO156']**</span> <a id='p9'></a> 

In [90]:
test_9 = pd.DataFrame(index=test[test['Mapped_GCK'] == '9']['Month Year'], columns=lagged_product_dfs['P9'][['Sales','P9_lag_12', 'PRO27826_org', 'PRO271000_org', 'PRO28250_org', 'MAB_ELE_PRO156']].columns) #we need the lags
test_9

,Sales,P9_lag_12,PRO27826_org,PRO271000_org,PRO28250_org,MAB_ELE_PRO156
Month Year,,,,,,
2022-05-01,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-01,NaN,NaN,NaN,NaN,NaN,NaN
2022-07-01,NaN,NaN,NaN,NaN,NaN,NaN
2022-08-01,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-01,NaN,NaN,NaN,NaN,NaN,NaN
2022-10-01,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-01,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-01,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-01,NaN,NaN,NaN,NaN,NaN,NaN


In [91]:
macro_preds[['PRO27826_org', 'PRO271000_org', 'PRO28250_org', 'MAB_ELE_PRO156']]

,PRO27826_org,PRO271000_org,PRO28250_org,MAB_ELE_PRO156
Month Year,,,,
2022-05-01,115.9040,132.7095,92.2586,283.1652
2022-06-01,125.3562,142.1336,111.3546,307.8518
2022-07-01,123.8094,138.8142,102.3235,269.5245
2022-08-01,122.5133,132.1480,72.9638,257.5743
2022-09-01,130.8887,145.8849,108.4830,265.2198
2022-10-01,125.7549,144.6288,107.0753,278.9898
2022-11-01,130.2161,146.1588,104.1748,291.4109
2022-12-01,124.3347,139.9869,108.9752,301.0567
2023-01-01,121.0189,129.8708,91.7178,237.1610


In [92]:
test_9 = test_9.merge(macro_preds[['PRO27826_org', 'PRO271000_org', 'PRO28250_org', 'MAB_ELE_PRO156']], 
                       left_index=True, 
                       right_index=True, 
                       how='left')

test_9 = test_9[['Sales','P9_lag_12', 'PRO27826_org_y', 'PRO271000_org_y', 'PRO28250_org_y', 'MAB_ELE_PRO156_y']]
test_9.columns = ['Sales','P9_lag_12', 'PRO27826_org', 'PRO271000_org', 'PRO28250_org', 'MAB_ELE_PRO156'] 

In [93]:
df_9 = lagged_product_dfs['P9'][['Sales','P9_lag_12', 'PRO27826_org', 'PRO271000_org', 'PRO28250_org', 'MAB_ELE_PRO156']].copy()

In [94]:
df_extended = pd.concat([df_9, test_9])

WARNING - (py.warnings._showwarnmsg) - C:\Users\catar\AppData\Local\Temp\ipykernel_524\1228142121.py:1: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.




In [95]:
df_extended['P9_lag_12'] = df_extended['Sales'].shift(12) # we have these lags

In [96]:
# Extract only the last 10 rows (test set) with recalculated lags
test_9 = df_extended.loc[test_index, ['Sales','P9_lag_12', 'PRO27826_org', 'PRO271000_org', 'PRO28250_org', 'MAB_ELE_PRO156']]

In [97]:
forecast_results_p9 = forecast_xgboost(
    df=df_9,  # Full dataset with target (Sales)
    test_dataset=test_9,  # Future exogenous variables only
    product_id='P9',
    feature_set=['P9_lag_12', 'PRO27826_org', 'PRO271000_org', 'PRO28250_org', 'MAB_ELE_PRO156'],  # Features available in test
    best_params={'colsample_bytree': 0.9, 'learning_rate': 0.05, 'max_depth': 2, 'min_child_weight': 1, 'n_estimators': 50, 'subsample': 0.9},
    target_col='Sales',
    horizon=10,
    winsorize=False)

Processing XGBoost for P9 with params: {'objective': 'reg:squarederror', 'random_state': 42, 'n_jobs': 1, 'tree_method': 'hist', 'enable_categorical': False, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1, 'colsample_bytree': 0.9, 'learning_rate': 0.05, 'max_depth': 2, 'min_child_weight': 1, 'n_estimators': 50, 'subsample': 0.9}
Iteration 1: Prediction=6779.60107421875, Inputs=[[8234.676     115.904015  132.70953    92.25861   283.16522 ]]
Iteration 2: Prediction=12996.3046875, Inputs=[[7450.82436  125.3562   142.13358  111.35458  307.85178]]
Iteration 3: Prediction=3312.931640625, Inputs=[[11679.8055    123.80945   138.8142    102.32352   269.52448]]
Iteration 4: Prediction=10847.2431640625, Inputs=[[447.12     122.513275 132.14804   72.963776 257.57428 ]]
Iteration 5: Prediction=8744.712890625, Inputs=[[9611.0496   130.88869  145.88486  108.48302  265.21982]]
Iteration 6: Prediction=25118.373046875, Inputs=[[977.33272 125.75492 144.62878 107.07527 278.98978]]
Iteration 7: Prediction=588

In [98]:
test_9.loc[test_9.index[:len(forecast_results_p9)], "Sales"] = forecast_results_p9

In [99]:
test_9

,Sales,P9_lag_12,PRO27826_org,PRO271000_org,PRO28250_org,MAB_ELE_PRO156
2022-05-01,6779.6011,8234.6760,115.9040,132.7095,92.2586,283.1652
2022-06-01,12996.3047,7450.8244,125.3562,142.1336,111.3546,307.8518
2022-07-01,3312.9316,11679.8055,123.8094,138.8142,102.3235,269.5245
2022-08-01,10847.2432,447.1200,122.5133,132.1480,72.9638,257.5743
2022-09-01,8744.7129,9611.0496,130.8887,145.8849,108.4830,265.2198
2022-10-01,25118.3730,977.3327,125.7549,144.6288,107.0753,278.9898
2022-11-01,5885.3511,14667.1811,130.2161,146.1588,104.1748,291.4109
2022-12-01,12343.3174,15637.1544,124.3347,139.9869,108.9752,301.0567
2023-01-01,11074.1562,2172.0644,121.0189,129.8708,91.7178,237.1610
2023-02-01,10669.5508,955.5900,122.6496,131.3341,93.2339,231.0398


In [100]:
final_9 = pd.concat([df_9, test_9])

##### <span style="background-color:#000027; padding:5px; border-radius:5px;font-size:13pt">**5.8. P11: XGBoost  ['MAB_ELE_SHP276']**</span> <a id='p11'></a> 

In [101]:
test_11 = pd.DataFrame(index=test[test['Mapped_GCK'] == '11']['Month Year'], columns=product_dfs['P11'][['Sales','MAB_ELE_SHP276']].columns)
test_11

,Sales,MAB_ELE_SHP276
Month Year,,
2022-05-01,NaN,NaN
2022-06-01,NaN,NaN
2022-07-01,NaN,NaN
2022-08-01,NaN,NaN
2022-09-01,NaN,NaN
2022-10-01,NaN,NaN
2022-11-01,NaN,NaN
2022-12-01,NaN,NaN
2023-01-01,NaN,NaN


In [102]:
macro_preds[['MAB_ELE_SHP276']]

,MAB_ELE_SHP276
Month Year,
2022-05-01,129.8432
2022-06-01,145.7785
2022-07-01,140.1388
2022-08-01,129.8402
2022-09-01,147.7511
2022-10-01,142.2516
2022-11-01,148.9787
2022-12-01,152.8668
2023-01-01,128.0293


In [103]:
test_11 = test_11.merge(macro_preds[['MAB_ELE_SHP276']], 
                       left_index=True, 
                       right_index=True, 
                       how='left')
test_11 = test_11[['Sales', 'MAB_ELE_SHP276_y']]
test_11.columns = ['Sales','MAB_ELE_SHP276'] 

In [104]:
df_11 = product_dfs['P11'][['Sales', 'MAB_ELE_SHP276']].copy()

In [105]:
forecast_results_p11 = forecast_xgboost(
    df=df_11,  # Full dataset with target (Sales)
    test_dataset=test_11,  # Future exogenous variables only
    product_id='P11',
    feature_set=['MAB_ELE_SHP276'],  # Features available in test
    best_params={'colsample_bytree': 0.9, 'learning_rate': 0.05, 'max_depth': 2, 'min_child_weight': 3, 'n_estimators': 100, 'subsample': 0.9}    ,
    target_col='Sales',
    horizon=10,
    winsorize=True)

Processing XGBoost for P11 with params: {'objective': 'reg:squarederror', 'random_state': 42, 'n_jobs': 1, 'tree_method': 'hist', 'enable_categorical': False, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1, 'colsample_bytree': 0.9, 'learning_rate': 0.05, 'max_depth': 2, 'min_child_weight': 3, 'n_estimators': 100, 'subsample': 0.9}
Sales: Winsorized Bounds -> Lower = 86784.03, Upper = 3914481.34
Iteration 1: Prediction=944714.5625, Inputs=[[129.84322]]
Iteration 2: Prediction=2275831.0, Inputs=[[145.77846]]
Iteration 3: Prediction=1436210.25, Inputs=[[140.13882]]
Iteration 4: Prediction=944714.5625, Inputs=[[129.84024]]
Iteration 5: Prediction=2289027.5, Inputs=[[147.7511]]
Iteration 6: Prediction=1895736.875, Inputs=[[142.25162]]
Iteration 7: Prediction=2289027.5, Inputs=[[148.97873]]
Iteration 8: Prediction=2289027.5, Inputs=[[152.86679]]
Iteration 9: Prediction=1513383.375, Inputs=[[128.02931]]
Iteration 10: Prediction=1342389.125, Inputs=[[136.07765]]


In [106]:
test_11.loc[test_11.index[:len(forecast_results_p11)], "Sales"] = forecast_results_p11
final_11 = pd.concat([df_11, test_11])

In [107]:
test_11

,Sales,MAB_ELE_SHP276
Month Year,,
2022-05-01,944714.5625,129.8432
2022-06-01,2275831.0000,145.7785
2022-07-01,1436210.2500,140.1388
2022-08-01,944714.5625,129.8402
2022-09-01,2289027.5000,147.7511
2022-10-01,1895736.8750,142.2516
2022-11-01,2289027.5000,148.9787
2022-12-01,2289027.5000,152.8668
2023-01-01,1513383.3750,128.0293


##### <span style="background-color:#000027; padding:5px; border-radius:5px;font-size:13pt">**5.10. P12: ARIMA	['MAB_ELE_PRO156']**</span> <a id='p12'></a> 

In [108]:
test_12 = pd.DataFrame(index=test[test['Mapped_GCK'] == '12']['Month Year'], columns=product_dfs['P12'][['Sales','MAB_ELE_PRO156']].columns)
test_12

,Sales,MAB_ELE_PRO156
Month Year,,
2022-05-01,NaN,NaN
2022-06-01,NaN,NaN
2022-07-01,NaN,NaN
2022-08-01,NaN,NaN
2022-09-01,NaN,NaN
2022-10-01,NaN,NaN
2022-11-01,NaN,NaN
2022-12-01,NaN,NaN
2023-01-01,NaN,NaN


In [109]:
macro_preds[['MAB_ELE_PRO156']]

test_12 = test_12.merge(
    macro_preds[['MAB_ELE_PRO156']],
    left_index=True,
    right_index=True,
    how='left'
)

test_12 = test_12[['Sales', 'MAB_ELE_PRO156_y']]
test_12.columns = ['Sales','MAB_ELE_PRO156'] 

In [110]:
df_12 = product_dfs['P12'][['Sales','MAB_ELE_PRO156']].copy()

In [111]:
predictions12 = forecast_sarimax(
    df=df_12,
    test_dataset=test_12,
    product_id='P12',
    feature_set=['MAB_ELE_PRO156'],
    target_col='Sales',
    horizon=10,
    order=(2, 1, 0),  # Example order (p,d,q)
    seasonal_order=(0, 0, 0, 0), # Example seasonal order (P,D,Q,s)
    winsorize=True
    )

Processing SARIMAX for P12...
Sales: Winsorized Bounds -> Lower = 2874.93, Upper = 446478.43


In [112]:
test_12['Sales'] = predictions12.values

In [113]:
final_12 = pd.concat([df_12, test_12])

##### <span style="background-color:#000027; padding:5px; border-radius:5px;font-size:13pt">**5.11. P13: XGBoost  ['P13_lag_8', 'P13_ma_17', 'P13_ma_5', 'P13_ma_8']**</span> <a id='p13'></a> 

In [114]:
test_13 = pd.DataFrame(index=test[test['Mapped_GCK'] == '13']['Month Year'], columns=lagged_product_dfs['P13'][['Sales','P13_lag_8', 'P13_ma_17', 'P13_ma_5', 'P13_ma_8']].columns) #we need the lags
test_13

,Sales,P13_lag_8,P13_ma_17,P13_ma_5,P13_ma_8
Month Year,,,,,
2022-05-01,NaN,NaN,NaN,NaN,NaN
2022-06-01,NaN,NaN,NaN,NaN,NaN
2022-07-01,NaN,NaN,NaN,NaN,NaN
2022-08-01,NaN,NaN,NaN,NaN,NaN
2022-09-01,NaN,NaN,NaN,NaN,NaN
2022-10-01,NaN,NaN,NaN,NaN,NaN
2022-11-01,NaN,NaN,NaN,NaN,NaN
2022-12-01,NaN,NaN,NaN,NaN,NaN
2023-01-01,NaN,NaN,NaN,NaN,NaN


In [115]:
df_13 = lagged_product_dfs['P13'][['Sales','P13_lag_8', 'P13_ma_17', 'P13_ma_5', 'P13_ma_8']].copy()

In [116]:
test_index = pd.date_range(start="2022-05-01", periods=10, freq="MS")
df_extended = pd.concat([df_13, test_13])

WARNING - (py.warnings._showwarnmsg) - C:\Users\catar\AppData\Local\Temp\ipykernel_524\2489420279.py:2: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.




In [117]:
df_extended['P13_lag_8'] = df_extended['Sales'].shift(8) # we have these lags

In [118]:
test_13 = df_extended.loc[test_index, ['Sales','P13_lag_8', 'P13_ma_17', 'P13_ma_5', 'P13_ma_8']]

In [119]:
forecast_results_ma_13 = forecast_ma_columns(df_13, forecast_horizon=10)

INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


Forecasting P13_ma_17...


INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


Forecasting P13_ma_5...


INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


Forecasting P13_ma_8...


In [120]:
for col in forecast_results_ma_13.columns:
    if col in test_13.columns:
        test_13.loc[test_13.index.isin(forecast_results_ma_13.index), col] = forecast_results_ma_13[col]

In [121]:
test_13

,Sales,P13_lag_8,P13_ma_17,P13_ma_5,P13_ma_8
2022-05-01,NaN,29772.5284,23607.3960,25855.5120,25483.4500
2022-06-01,NaN,31817.9145,23606.6430,23966.9920,23810.9650
2022-07-01,NaN,8503.0814,23588.7620,22390.6370,22041.3900
2022-08-01,NaN,26037.6964,23800.5500,19100.5300,21735.1170
2022-09-01,NaN,39947.3961,23780.2730,17636.3650,21821.0350
2022-10-01,NaN,35456.2050,24448.2810,19400.8160,21621.8550
2022-11-01,NaN,13133.5014,24439.4770,20561.5270,19483.6370
2022-12-01,NaN,18517.8035,24751.5100,23103.3770,18627.5250
2023-01-01,NaN,NaN,24662.5000,23571.3200,20369.6400
2023-02-01,NaN,NaN,24719.6400,23465.6000,22117.1050


In [122]:
forecast_results_p13 = forecast_xgboost(
    df=df_13,  # Full dataset with target (Sales)
    test_dataset=test_13,  # Future exogenous variables only
    product_id='P13',
    feature_set=['P13_lag_8', 'P13_ma_17', 'P13_ma_5', 'P13_ma_8'],  # Features available in test
    best_params={'colsample_bytree': 1.0, 'learning_rate': 0.01, 'max_depth': 2, 'min_child_weight': 3, 'n_estimators': 100, 'subsample': 1.0},
    target_col='Sales',
    horizon=8,
    winsorize=False)

Processing XGBoost for P13 with params: {'objective': 'reg:squarederror', 'random_state': 42, 'n_jobs': 1, 'tree_method': 'hist', 'enable_categorical': False, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1, 'colsample_bytree': 1.0, 'learning_rate': 0.01, 'max_depth': 2, 'min_child_weight': 3, 'n_estimators': 100, 'subsample': 1.0}
Iteration 1: Prediction=19087.556640625, Inputs=[[29772.52842 23607.396   25855.512   25483.45   ]]
Iteration 2: Prediction=19087.556640625, Inputs=[[31817.91445 23606.643   23966.992   23810.965  ]]
Iteration 3: Prediction=17243.328125, Inputs=[[ 8503.0814 23588.762  22390.637  22041.39  ]]
Iteration 4: Prediction=19087.556640625, Inputs=[[26037.69642 23800.55    19100.53    21735.117  ]]
Iteration 5: Prediction=19087.556640625, Inputs=[[39947.39612 23780.273   17636.365   21821.035  ]]
Iteration 6: Prediction=19087.556640625, Inputs=[[35456.205 24448.281 19400.816 21621.855]]
Iteration 7: Prediction=17569.560546875, Inputs=[[13133.5014 24439.477  20561.527  19

In [123]:
test_13.loc[test_13.index[:len(forecast_results_p13)], "Sales"] = forecast_results_p13

In [124]:
test_13

,Sales,P13_lag_8,P13_ma_17,P13_ma_5,P13_ma_8
2022-05-01,19087.5566,29772.5284,23607.3960,25855.5120,25483.4500
2022-06-01,19087.5566,31817.9145,23606.6430,23966.9920,23810.9650
2022-07-01,17243.3281,8503.0814,23588.7620,22390.6370,22041.3900
2022-08-01,19087.5566,26037.6964,23800.5500,19100.5300,21735.1170
2022-09-01,19087.5566,39947.3961,23780.2730,17636.3650,21821.0350
2022-10-01,19087.5566,35456.2050,24448.2810,19400.8160,21621.8550
2022-11-01,17569.5605,13133.5014,24439.4770,20561.5270,19483.6370
2022-12-01,17569.5605,18517.8035,24751.5100,23103.3770,18627.5250
2023-01-01,NaN,NaN,24662.5000,23571.3200,20369.6400
2023-02-01,NaN,NaN,24719.6400,23465.6000,22117.1050


In [125]:
df_extended = pd.concat([df_13, test_13])

In [126]:
# Create new shifted columns without replacing existing ones
df_extended["P13_lag_8_new"] = df_extended["Sales"].shift(8)

# Fill NaN values in the original lag columns with the new calculated ones
df_extended["P13_lag_8"] = df_extended["P13_lag_8"].fillna(df_extended["P13_lag_8_new"])

# Drop the temporary columns
df_extended.drop(columns=['P13_lag_8_new'], inplace=True)

In [127]:
test_13 = df_extended.loc[test_index, ['Sales','P13_lag_8', 'P13_ma_17', 'P13_ma_5', 'P13_ma_8']]

In [128]:
test_13_2nd = df_extended.tail(2)  # Últimas 7 linhas
df_13_2nd = df_extended.iloc[:-2]  # Todas menos as últimas 7 linhas

In [129]:
df_13_2nd

,Sales,P13_lag_8,P13_ma_17,P13_ma_5,P13_ma_8
2020-03-01,23584.1308,43137.3751,27965.5913,25306.0330,29324.8839
2020-04-01,30838.9744,22542.5952,25829.2192,22878.8666,30361.9313
2020-05-01,3777.8713,19640.2521,25144.3674,18288.4889,28379.1337
2020-06-01,9660.8640,65886.0591,24149.5518,16324.5515,21350.9843
2020-07-01,6724.9644,42974.8062,22661.6970,14917.3610,16819.7541
2020-08-01,25612.5711,26729.7600,23479.3982,15323.0490,16680.1055
2020-09-01,15268.9154,19480.5508,23939.7104,12209.0372,16153.6511
2020-10-01,9515.3551,13760.9172,24350.0190,13356.5340,15622.9558
2020-11-01,20601.6491,23584.1308,23513.9771,15544.6910,15250.1456
2020-12-01,31154.2766,30838.9744,22809.0890,20430.5535,15289.5584


In [130]:
forecast_results_p13 = forecast_xgboost(
    df=df_13_2nd,  # Full dataset with target (Sales)
    test_dataset=test_13_2nd,  # Future exogenous variables only
    product_id='P13',
    feature_set=['P13_lag_8', 'P13_ma_17', 'P13_ma_5', 'P13_ma_8'],  # Features available in test
    best_params={'colsample_bytree': 1.0, 'learning_rate': 0.01, 'max_depth': 2, 'min_child_weight': 3, 'n_estimators': 100, 'subsample': 1.0},
    target_col='Sales',
    horizon=2,
    winsorize=False)

Processing XGBoost for P13 with params: {'objective': 'reg:squarederror', 'random_state': 42, 'n_jobs': 1, 'tree_method': 'hist', 'enable_categorical': False, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1, 'colsample_bytree': 1.0, 'learning_rate': 0.01, 'max_depth': 2, 'min_child_weight': 3, 'n_estimators': 100, 'subsample': 1.0}
Iteration 1: Prediction=17853.07421875, Inputs=[[19087.55664062 24662.5        23571.32       20369.64      ]]
Iteration 2: Prediction=17853.07421875, Inputs=[[19087.55664062 24719.64       23465.6        22117.105     ]]


In [131]:
test_13.loc[test_13.index[8:8+len(forecast_results_p13)], "Sales"] = forecast_results_p13
final_13 = pd.concat([df_13, test_13])

In [132]:
test_13

,Sales,P13_lag_8,P13_ma_17,P13_ma_5,P13_ma_8
2022-05-01,19087.5566,29772.5284,23607.3960,25855.5120,25483.4500
2022-06-01,19087.5566,31817.9145,23606.6430,23966.9920,23810.9650
2022-07-01,17243.3281,8503.0814,23588.7620,22390.6370,22041.3900
2022-08-01,19087.5566,26037.6964,23800.5500,19100.5300,21735.1170
2022-09-01,19087.5566,39947.3961,23780.2730,17636.3650,21821.0350
2022-10-01,19087.5566,35456.2050,24448.2810,19400.8160,21621.8550
2022-11-01,17569.5605,13133.5014,24439.4770,20561.5270,19483.6370
2022-12-01,17569.5605,18517.8035,24751.5100,23103.3770,18627.5250
2023-01-01,17853.0742,19087.5566,24662.5000,23571.3200,20369.6400
2023-02-01,17853.0742,19087.5566,24719.6400,23465.6000,22117.1050


##### <span style="background-color:#000027; padding:5px; border-radius:5px;font-size:13pt">**5.12. P14: Prophet ['PRO27392_org', 'PRO28380_org']**</span> <a id='p14'></a> 

In [133]:
test_14 = pd.DataFrame(index=test[test['Mapped_GCK'] == '14']['Month Year'], columns=product_dfs['P14'][['Sales','PRO27392_org', 'PRO28380_org']].columns)
test_14

,Sales,PRO27392_org,PRO28380_org
Month Year,,,
2022-05-01,NaN,NaN,NaN
2022-06-01,NaN,NaN,NaN
2022-07-01,NaN,NaN,NaN
2022-08-01,NaN,NaN,NaN
2022-09-01,NaN,NaN,NaN
2022-10-01,NaN,NaN,NaN
2022-11-01,NaN,NaN,NaN
2022-12-01,NaN,NaN,NaN
2023-01-01,NaN,NaN,NaN


In [134]:
macro_preds[['PRO27392_org', 'PRO28380_org']]

,PRO27392_org,PRO28380_org
Month Year,,
2022-05-01,99.8816,112.4203
2022-06-01,115.7264,121.0243
2022-07-01,110.5113,121.1212
2022-08-01,95.5346,75.0991
2022-09-01,112.9201,122.8645
2022-10-01,108.5995,122.8734
2022-11-01,110.3707,120.8869
2022-12-01,114.6863,110.1409
2023-01-01,103.1821,102.6021


In [135]:
test_14 = test_14.merge(macro_preds[['PRO27392_org', 'PRO28380_org']], 
                       left_index=True, 
                       right_index=True, 
                       how='left')
test_14 = test_14[['Sales', 'PRO27392_org_y', 'PRO28380_org_y']]
test_14.columns = ['Sales','PRO27392_org', 'PRO28380_org']

In [136]:
df_14 = product_dfs['P14'][['Sales', 'PRO27392_org', 'PRO28380_org']].copy()

In [137]:
predictions_14 = forecast_prophet(
    df=df_14,
    test_dataset=test_14,
    product_id='P14',
    feature_set=['PRO27392_org', 'PRO28380_org'],
    target_col='Sales',
    horizon=10)

DEBUG:cmdstanpy:cmd: where.exe tbb.dll
cwd: None
DEBUG:cmdstanpy:Adding TBB (c:\Users\catar\anaconda3\Lib\site-packages\prophet\stan_model\cmdstan-2.33.1\stan\lib\stan_math\lib\tbb) to PATH


Processing Prophet for P14...


DEBUG:cmdstanpy:input tempfile: C:\Users\catar\AppData\Local\Temp\tmpjabb7fwg\geulk_n9.json
DEBUG:cmdstanpy:input tempfile: C:\Users\catar\AppData\Local\Temp\tmpjabb7fwg\69mzay45.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['C:\\Users\\catar\\anaconda3\\Lib\\site-packages\\prophet\\stan_model\\prophet_model.bin', 'random', 'seed=74689', 'data', 'file=C:\\Users\\catar\\AppData\\Local\\Temp\\tmpjabb7fwg\\geulk_n9.json', 'init=C:\\Users\\catar\\AppData\\Local\\Temp\\tmpjabb7fwg\\69mzay45.json', 'output', 'file=C:\\Users\\catar\\AppData\\Local\\Temp\\tmpjabb7fwg\\prophet_modelwn41_nc6\\prophet_model-20250401000824.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
00:08:24 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
00:08:25 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [138]:
test_14.loc[test_14.index[:len(predictions_14)], "Sales"] = predictions_14

In [139]:
final_14 = pd.concat([df_14, test_14])

##### <span style="background-color:#000027; padding:5px; border-radius:5px;font-size:13pt">**5.13. P16: XGBoost ['P16_lag_3', 'P16_lag_8', 'P16_lag_10', 'P16_ma_3', 'P16_ma_7', 'P16_ma_8', 'P16_ma_9', 'P16_ma_10']**</span> <a id='p16'></a> 

In [140]:
test_16 = pd.DataFrame(index=test[test['Mapped_GCK'] == '16']['Month Year'], columns=lagged_product_dfs['P16'][['Sales','P16_lag_3', 'P16_lag_8', 'P16_lag_10', 'P16_ma_3', 'P16_ma_7', 'P16_ma_8', 'P16_ma_9', 'P16_ma_10']].columns) #we need the lags
test_16

,Sales,P16_lag_3,P16_lag_8,P16_lag_10,P16_ma_3,P16_ma_7,P16_ma_8,P16_ma_9,P16_ma_10
Month Year,,,,,,,,,
2022-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-07-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-08-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-09-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [141]:
df_16 = lagged_product_dfs['P16'][['Sales','P16_lag_3', 'P16_lag_8', 'P16_lag_10', 'P16_ma_3', 'P16_ma_7', 'P16_ma_8', 'P16_ma_9', 'P16_ma_10']].copy()

In [142]:
test_index = pd.date_range(start="2022-05-01", periods=10, freq="MS")
df_extended = pd.concat([df_16, test_16])

WARNING - (py.warnings._showwarnmsg) - C:\Users\catar\AppData\Local\Temp\ipykernel_524\1993590299.py:2: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.




In [143]:
df_extended['P16_lag_10'] = df_extended['Sales'].shift(10) # we have these lags
df_extended['P16_lag_3'] = df_extended['Sales'].shift(3) # we will need to divide the forecast
df_extended['P16_lag_8'] = df_extended['Sales'].shift(8) # we will need to divide the forecast

In [144]:
test_16 = df_extended.loc[test_index, ['Sales','P16_lag_3', 'P16_lag_8', 'P16_lag_10', 'P16_ma_3', 'P16_ma_7', 'P16_ma_8', 'P16_ma_9', 'P16_ma_10']]

In [145]:
forecast_results_ma_16 = forecast_ma_columns(df_16, forecast_horizon=10)

INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


Forecasting P16_ma_3...


INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


Forecasting P16_ma_7...


INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


Forecasting P16_ma_8...


INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


Forecasting P16_ma_9...


INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Restricting input...
INFO:nixtla.nixtla_client:Calling Forecast Endpoint...


Forecasting P16_ma_10...


In [146]:
for col in forecast_results_ma_16.columns:
    if col in test_16.columns:
        test_16.loc[test_16.index.isin(forecast_results_ma_16.index), col] = forecast_results_ma_16[col]

In [147]:
forecast_results_p16 = forecast_xgboost(
    df=df_16,  # Full dataset with target (Sales)
    test_dataset=test_16,  # Future exogenous variables only
    product_id='P16',
    feature_set=['P16_lag_3', 'P16_lag_8', 'P16_lag_10', 'P16_ma_3', 'P16_ma_7', 'P16_ma_8', 'P16_ma_9', 'P16_ma_10'],  # Features available in test
    best_params={'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 2, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 1.0}	,
    target_col='Sales',
    horizon=3,
    winsorize=False)

Processing XGBoost for P16 with params: {'objective': 'reg:squarederror', 'random_state': 42, 'n_jobs': 1, 'tree_method': 'hist', 'enable_categorical': False, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1, 'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 2, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 1.0}
Iteration 1: Prediction=119503.1171875, Inputs=[[ 48438.5656  193676.51712 107469.65856 117372.016   165369.48
  164900.69    182931.17    171982.52   ]]
Iteration 2: Prediction=85575.2578125, Inputs=[[106727.03271 147101.65358 334872.21105 123668.484   169765.7
  170026.11    181152.81    175625.86   ]]
Iteration 3: Prediction=119824.796875, Inputs=[[145869.50912 240484.8886  193676.51712 129064.      174122.1
  172538.34    184264.16    174926.27   ]]


In [148]:
test_16.loc[test_16.index[:len(forecast_results_p16)], "Sales"] = forecast_results_p16

In [149]:
df_extended = pd.concat([df_16, test_16])

In [150]:
# Create new shifted columns without replacing existing ones
df_extended["P16_lag_10_new"] = df_extended["Sales"].shift(10)
df_extended["P16_lag_3_new"] = df_extended["Sales"].shift(3)
df_extended["P16_lag_8_new"] = df_extended["Sales"].shift(8)

# Fill NaN values in the original lag columns with the new calculated ones
df_extended["P16_lag_10"] = df_extended["P16_lag_10"].fillna(df_extended["P16_lag_10_new"])
df_extended["P16_lag_3"] = df_extended["P16_lag_3"].fillna(df_extended["P16_lag_3_new"])
df_extended["P16_lag_8"] = df_extended["P16_lag_8"].fillna(df_extended["P16_lag_8_new"])

# Drop the temporary columns
df_extended.drop(columns=["P16_lag_10_new", "P16_lag_3_new", "P16_lag_8_new"], inplace=True)

In [151]:
test_16 = df_extended.loc[test_index, ['Sales','P16_lag_3', 'P16_lag_8', 'P16_lag_10', 'P16_ma_3', 'P16_ma_7', 'P16_ma_8', 'P16_ma_9', 'P16_ma_10']]

In [152]:
test_16_2nd = df_extended.tail(7)  # Últimas 7 linhas
df_16_2nd = df_extended.iloc[:-7]  # Todas menos as últimas 7 linhas

In [153]:
forecast_results_p16 = forecast_xgboost(
    df=df_16_2nd,  # Full dataset with target (Sales)
    test_dataset=test_16_2nd,  # Future exogenous variables only
    product_id='P16',
    feature_set=['P16_lag_3', 'P16_lag_8', 'P16_lag_10', 'P16_ma_3', 'P16_ma_7', 'P16_ma_8', 'P16_ma_9', 'P16_ma_10'],  # Features available in test
    best_params={'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 2, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 1.0}	,
    target_col='Sales',
    horizon=3,
    winsorize=False)

Processing XGBoost for P16 with params: {'objective': 'reg:squarederror', 'random_state': 42, 'n_jobs': 1, 'tree_method': 'hist', 'enable_categorical': False, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1, 'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 2, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 1.0}
Iteration 1: Prediction=162708.71875, Inputs=[[119503.1171875 283853.62533   147101.65358   133211.28
  177053.25      177116.84      184352.73      176357.33     ]]
Iteration 2: Prediction=97059.8828125, Inputs=[[ 85575.2578125  95755.02828   240484.8886    134001.75
  181310.84      176234.23      184370.36      177775.64     ]]
Iteration 3: Prediction=89499.6953125, Inputs=[[119824.796875  48438.5656   283853.62533  127593.18     179583.6
  179786.7      183721.92     177462.14    ]]


In [154]:
test_16.loc[test_16.index[3:3+len(forecast_results_p16)], "Sales"] = forecast_results_p16

In [155]:
df_extended = pd.concat([df_16, test_16])

In [156]:
# Create new shifted columns without replacing existing ones
df_extended["P16_lag_10_new"] = df_extended["Sales"].shift(10)
df_extended["P16_lag_3_new"] = df_extended["Sales"].shift(3)
df_extended["P16_lag_8_new"] = df_extended["Sales"].shift(8)

# Fill NaN values in the original lag columns with the new calculated ones
df_extended["P16_lag_10"] = df_extended["P16_lag_10"].fillna(df_extended["P16_lag_10_new"])
df_extended["P16_lag_3"] = df_extended["P16_lag_3"].fillna(df_extended["P16_lag_3_new"])
df_extended["P16_lag_8"] = df_extended["P16_lag_8"].fillna(df_extended["P16_lag_8_new"])

# Drop the temporary columns
df_extended.drop(columns=["P16_lag_10_new", "P16_lag_3_new", "P16_lag_8_new"], inplace=True)

In [157]:
test_16 = df_extended.loc[test_index, ['Sales','P16_lag_3', 'P16_lag_8', 'P16_lag_10', 'P16_ma_3', 'P16_ma_7', 'P16_ma_8', 'P16_ma_9', 'P16_ma_10']]
test_16_3rd = df_extended.tail(4)  # Últimas 7 linhas
df_16_3rd = df_extended.iloc[:-4]  # Todas menos as últimas 7 linhas

In [158]:
forecast_results_p16 = forecast_xgboost(
    df=df_16_3rd,  # Full dataset with target (Sales)
    test_dataset=test_16_3rd,  # Future exogenous variables only
    product_id='P16',
    feature_set=['P16_lag_3', 'P16_lag_8', 'P16_lag_10', 'P16_ma_3', 'P16_ma_7', 'P16_ma_8', 'P16_ma_9', 'P16_ma_10'],  # Features available in test
    best_params={'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 2, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 1.0}	,
    target_col='Sales',
    horizon=3,
    winsorize=False)

Processing XGBoost for P16 with params: {'objective': 'reg:squarederror', 'random_state': 42, 'n_jobs': 1, 'tree_method': 'hist', 'enable_categorical': False, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1, 'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 2, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 1.0}
Iteration 1: Prediction=144888.296875, Inputs=[[162708.71875 106727.03271  95755.02828 126202.23    184488.86
  179345.83    186654.53    176063.     ]]
Iteration 2: Prediction=126547.484375, Inputs=[[ 97059.8828125 145869.50912    48438.5656    126565.75
  184813.44      185781.78      187981.39      178657.25     ]]
Iteration 3: Prediction=142219.28125, Inputs=[[ 89499.6953125 119503.1171875 106727.03271   120240.69
  175722.39      179691.31      185937.67      175710.53     ]]


In [159]:
test_16.loc[test_16.index[6:6+len(forecast_results_p16)], "Sales"] = forecast_results_p16

In [160]:
df_extended = pd.concat([df_16, test_16])

In [161]:
# Create new shifted columns without replacing existing ones
df_extended["P16_lag_10_new"] = df_extended["Sales"].shift(10)
df_extended["P16_lag_3_new"] = df_extended["Sales"].shift(3)
df_extended["P16_lag_8_new"] = df_extended["Sales"].shift(8)

# Fill NaN values in the original lag columns with the new calculated ones
df_extended["P16_lag_10"] = df_extended["P16_lag_10"].fillna(df_extended["P16_lag_10_new"])
df_extended["P16_lag_3"] = df_extended["P16_lag_3"].fillna(df_extended["P16_lag_3_new"])
df_extended["P16_lag_8"] = df_extended["P16_lag_8"].fillna(df_extended["P16_lag_8_new"])

# Drop the temporary columns
df_extended.drop(columns=["P16_lag_10_new", "P16_lag_3_new", "P16_lag_8_new"], inplace=True)

In [162]:
test_16 = df_extended.loc[test_index, ['Sales','P16_lag_3', 'P16_lag_8', 'P16_lag_10', 'P16_ma_3', 'P16_ma_7', 'P16_ma_8', 'P16_ma_9', 'P16_ma_10']]
test_16_4th = df_extended.tail(1)  # Últimas 7 linhas
df_16_4th = df_extended.iloc[:-1]  # Todas menos as últimas 7 linhas

In [163]:
forecast_results_p16 = forecast_xgboost(
    df=df_16_4th,  # Full dataset with target (Sales)
    test_dataset=test_16_4th,  # Future exogenous variables only
    product_id='P16',
    feature_set=['P16_lag_3', 'P16_lag_8', 'P16_lag_10', 'P16_ma_3', 'P16_ma_7', 'P16_ma_8', 'P16_ma_9', 'P16_ma_10'],  # Features available in test
    best_params={'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 2, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 1.0}	,
    target_col='Sales',
    horizon=1,
    winsorize=False)

Processing XGBoost for P16 with params: {'objective': 'reg:squarederror', 'random_state': 42, 'n_jobs': 1, 'tree_method': 'hist', 'enable_categorical': False, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1, 'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 2, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 1.0}
Iteration 1: Prediction=120626.7734375, Inputs=[[144888.296875   85575.2578125 145869.50912   110355.66
  166537.47      172915.31      184310.39      176569.2      ]]


In [164]:
test_16.loc[test_16.index[9:9+len(forecast_results_p16)], "Sales"] = forecast_results_p16

In [165]:
test_16

,Sales,P16_lag_3,P16_lag_8,P16_lag_10,P16_ma_3,P16_ma_7,P16_ma_8,P16_ma_9,P16_ma_10
2022-05-01,119503.1172,48438.5656,193676.5171,107469.6586,117372.0160,165369.4800,164900.6900,182931.1700,171982.5200
2022-06-01,85575.2578,106727.0327,147101.6536,334872.2110,123668.4840,169765.7000,170026.1100,181152.8100,175625.8600
2022-07-01,119824.7969,145869.5091,240484.8886,193676.5171,129064.0000,174122.1000,172538.3400,184264.1600,174926.2700
2022-08-01,162708.7188,119503.1172,283853.6253,147101.6536,133211.2800,177053.2500,177116.8400,184352.7300,176357.3300
2022-09-01,97059.8828,85575.2578,95755.0283,240484.8886,134001.7500,181310.8400,176234.2300,184370.3600,177775.6400
2022-10-01,89499.6953,119824.7969,48438.5656,283853.6253,127593.1800,179583.6000,179786.7000,183721.9200,177462.1400
2022-11-01,144888.2969,162708.7188,106727.0327,95755.0283,126202.2300,184488.8600,179345.8300,186654.5300,176063.0000
2022-12-01,126547.4844,97059.8828,145869.5091,48438.5656,126565.7500,184813.4400,185781.7800,187981.3900,178657.2500
2023-01-01,142219.2812,89499.6953,119503.1172,106727.0327,120240.6900,175722.3900,179691.3100,185937.6700,175710.5300
2023-02-01,120626.7734,144888.2969,85575.2578,145869.5091,110355.6600,166537.4700,172915.3100,184310.3900,176569.2000


In [166]:
final_16 = pd.concat([df_16, test_16])

##### <span style="background-color:#000027; padding:5px; border-radius:5px;font-size:13pt">**5.14. P20: XGBoost  ['PRO27250_org', 'PRO28392_org']**</span> <a id='p20'></a> 

In [167]:
test_20 = pd.DataFrame(index=test[test['Mapped_GCK'] == '20']['Month Year'], columns=product_dfs['P20'][['Sales','PRO27250_org', 'PRO28392_org']].columns)
test_20


,Sales,PRO27250_org,PRO28392_org
Month Year,,,
2022-05-01,NaN,NaN,NaN
2022-06-01,NaN,NaN,NaN
2022-07-01,NaN,NaN,NaN
2022-08-01,NaN,NaN,NaN
2022-09-01,NaN,NaN,NaN
2022-10-01,NaN,NaN,NaN
2022-11-01,NaN,NaN,NaN
2022-12-01,NaN,NaN,NaN
2023-01-01,NaN,NaN,NaN


In [168]:
macro_preds[['PRO27250_org', 'PRO28392_org']]

,PRO27250_org,PRO28392_org
Month Year,,
2022-05-01,77.7203,118.6531
2022-06-01,91.9900,136.6734
2022-07-01,84.7953,130.1510
2022-08-01,66.3808,120.9310
2022-09-01,91.9670,136.9477
2022-10-01,91.7211,133.1175
2022-11-01,89.9914,138.3113
2022-12-01,84.2375,142.4281
2023-01-01,81.0084,127.9608


In [169]:
test_20 = test_20.merge(macro_preds[['PRO27250_org', 'PRO28392_org']], 
                       left_index=True, 
                       right_index=True, 
                       how='left')
test_20 = test_20[['Sales', 'PRO27250_org_y', 'PRO28392_org_y']]
test_20.columns = ['Sales','PRO27250_org', 'PRO28392_org'] 

In [170]:
df_20 = product_dfs['P20'][['Sales','PRO27250_org', 'PRO28392_org']].copy()

In [171]:
forecast_results_p20 = forecast_xgboost(
    df=df_20,  # Full dataset with target (Sales)
    test_dataset=test_20,  # Future exogenous variables only
    product_id='P20',
    feature_set=['PRO27250_org', 'PRO28392_org'],  # Features available in test
    best_params={'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 50, 'subsample': 0.9},
    target_col='Sales',
    horizon=10,
    winsorize=False)

Processing XGBoost for P20 with params: {'objective': 'reg:squarederror', 'random_state': 42, 'n_jobs': 1, 'tree_method': 'hist', 'enable_categorical': False, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1, 'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 3, 'min_child_weight': 3, 'n_estimators': 50, 'subsample': 0.9}
Iteration 1: Prediction=968.6133422851562, Inputs=[[ 77.72031 118.6531 ]]
Iteration 2: Prediction=3700.111083984375, Inputs=[[ 91.990036 136.67337 ]]
Iteration 3: Prediction=720.9829711914062, Inputs=[[ 84.7953 130.151 ]]
Iteration 4: Prediction=1201.578857421875, Inputs=[[ 66.38075  120.930984]]
Iteration 5: Prediction=3700.111083984375, Inputs=[[ 91.96703 136.94765]]
Iteration 6: Prediction=3700.111083984375, Inputs=[[ 91.7211  133.11746]]
Iteration 7: Prediction=3700.111083984375, Inputs=[[ 89.991394 138.31134 ]]
Iteration 8: Prediction=509.14349365234375, Inputs=[[ 84.23749 142.4281 ]]
Iteration 9: Prediction=658.9638671875, Inputs=[[ 81.00841 127.96083]]
Iter

In [172]:
test_20.loc[test_20.index[:len(forecast_results_p20)], "Sales"] = forecast_results_p20

In [173]:
final_20 = pd.concat([df_20, test_20])

##### <span style="background-color:#000027; padding:5px; border-radius:5px;font-size:13pt">**5.15. P36: XGBoost  ['PRO27756_org']**</span> <a id='p26'></a> 

In [174]:
test_36 = pd.DataFrame(index=test[test['Mapped_GCK'] == '36']['Month Year'], columns=product_dfs['P36'][['Sales','PRO27756_org']].columns)
test_36

,Sales,PRO27756_org
Month Year,,
2022-05-01,NaN,NaN
2022-06-01,NaN,NaN
2022-07-01,NaN,NaN
2022-08-01,NaN,NaN
2022-09-01,NaN,NaN
2022-10-01,NaN,NaN
2022-11-01,NaN,NaN
2022-12-01,NaN,NaN
2023-01-01,NaN,NaN


In [175]:
macro_preds[['PRO27756_org']]

,PRO27756_org
Month Year,
2022-05-01,93.8987
2022-06-01,90.7787
2022-07-01,95.5829
2022-08-01,101.4852
2022-09-01,106.8750
2022-10-01,111.6346
2022-11-01,114.7831
2022-12-01,118.0444
2023-01-01,113.0140


In [176]:
test_36 = test_36.merge(macro_preds[['PRO27756_org']], 
                       left_index=True, 
                       right_index=True, 
                       how='left')
test_36 = test_36[['Sales', 'PRO27756_org_y']]
test_36.columns = ['Sales','PRO27756_org']

In [177]:
df_36 = product_dfs['P36'][['Sales', 'PRO27756_org']].copy()

In [178]:
forecast_results_p36 = forecast_xgboost(
    df=df_36,  # Full dataset with target (Sales)
    test_dataset=test_36,  # Future exogenous variables only
    product_id='P36',
    feature_set=['PRO27756_org'],  # Features available in test
    best_params={'colsample_bytree': 0.9, 'learning_rate': 0.01, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 1.0},
    target_col='Sales',
    horizon=10,
    winsorize=True)

Processing XGBoost for P36 with params: {'objective': 'reg:squarederror', 'random_state': 42, 'n_jobs': 1, 'tree_method': 'hist', 'enable_categorical': False, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1, 'colsample_bytree': 0.9, 'learning_rate': 0.01, 'max_depth': 3, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 1.0}
Sales: Winsorized Bounds -> Lower = 957.35, Upper = 212525.09
Iteration 1: Prediction=19324.392578125, Inputs=[[93.89866]]
Iteration 2: Prediction=18429.0546875, Inputs=[[90.778725]]
Iteration 3: Prediction=19324.392578125, Inputs=[[95.58292]]
Iteration 4: Prediction=19324.392578125, Inputs=[[101.485214]]
Iteration 5: Prediction=15609.1611328125, Inputs=[[106.87502]]
Iteration 6: Prediction=15609.1611328125, Inputs=[[111.63457]]
Iteration 7: Prediction=18464.7421875, Inputs=[[114.7831]]
Iteration 8: Prediction=18464.7421875, Inputs=[[118.0444]]
Iteration 9: Prediction=15609.1611328125, Inputs=[[113.01401]]
Iteration 10: Prediction=15609.1611328125, Inputs=[[108.

In [179]:
test_36.loc[test_36.index[:len(forecast_results_p36)], "Sales"] = forecast_results_p36

In [180]:
final_36 = pd.concat([df_36, test_36])

#### <span style="background-color:#000027; padding:5px; border-radius:5px;">**6. Evaluation**</span> <a id='Evaluation'></a>  

Click [here](#table-of-contents) ⬆️ to return to the Index.


In [181]:
def analyze_sales_forecast(df):
    # Ensure the index is datetime
    df.index = pd.to_datetime(df.index)
    
    # Last 10 months (predictions) - next 10 months forecast
    forecast_10_months = df.iloc[-10:]
    avg_forecast_sales = forecast_10_months["Sales"].mean()
    
    # Calculate forecasted growth for next 10 months vs same period last year
    forecast_period_start = forecast_10_months.index[0]
    same_period_last_year = df[
        (df.index.year == forecast_period_start.year - 1) & 
        (df.index.month >= forecast_period_start.month)
    ]
    
    if len(same_period_last_year) > 0:
        forecast_growth = ((forecast_10_months["Sales"].sum() - same_period_last_year["Sales"].sum()) / 
                          same_period_last_year["Sales"].sum()) * 100
    else:
        forecast_growth = None
    
    # 2022 vs 2021 full year growth
    sales_2022 = df[df.index.year == 2022]["Sales"].sum()
    sales_2021 = df[df.index.year == 2021]["Sales"].sum()
    yoy_growth = ((sales_2022 - sales_2021) / sales_2021) * 100
    
    # CORRECTED: 2023 YTD growth (Jan-Feb 2023 vs Jan-Feb 2022)
    ytd_2023 = df[(df.index.year == 2023) & (df.index.month <= 2)]["Sales"].sum()
    ytd_2022 = df[(df.index.year == 2022) & (df.index.month <= 2)]["Sales"].sum()
    ytd_growth = ((ytd_2023 - ytd_2022) / ytd_2022) * 100 if ytd_2022 != 0 else None
    
    # Plot actual vs predicted sales
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=df.index[:-10], 
        y=df["Sales"][:-10], 
        mode='lines', 
        name='Actual Sales'
    ))
    fig.add_trace(go.Scatter(
        x=df.index[-10:], 
        y=df["Sales"][-10:], 
        mode='lines', 
        name='Forecasted Sales', 
        line=dict(dash='dash')
    ))
    
    fig.update_layout(
        title="Sales Analysis: Actual vs Forecasted",
        xaxis_title="Date",
        yaxis_title="Sales",
        template="plotly_dark"
    )
    
    fig.show()
    
    return {
        "avg_forecast_sales": avg_forecast_sales,
        "forecast_growth_next_10m": forecast_growth,
        "yoy_growth_2022_vs_2021": yoy_growth,
        "ytd_growth_2023_vs_2022": ytd_growth  # Renamed to be more explicit
    }


def format_results(results_dict, name):
    formatted = f"""
ANALYSIS RESULTS - {name}
--------------------------------------------
Metric                               | Value
--------------------------------------------
Avg Forecast Sales (next 10m):       {('€' + f"{results_dict['avg_forecast_sales']:,.2f}")}
Forecast Growth (next 10m):          {f"{results_dict['forecast_growth_next_10m']:.2f}%"}
YoY Growth (2022 vs 2021):           {f"{results_dict['yoy_growth_2022_vs_2021']:.2f}%"}
YTD Growth (2023 vs 2022):           {f"{results_dict['ytd_growth_2023_vs_2022']:.2f}%"}
--------------------------------------------
"""
    return formatted


> P1 evaluation

In [182]:
# First run the analysis to get the results dictionary
results_p1 = analyze_sales_forecast(final_1)

# Then format the results
print(format_results(results_p1, "P1"))


ANALYSIS RESULTS - P1
--------------------------------------------
Metric                               | Value
--------------------------------------------
Avg Forecast Sales (next 10m):       €43,019,464.50
Forecast Growth (next 10m):          40.18%
YoY Growth (2022 vs 2021):           14.11%
YTD Growth (2023 vs 2022):           12.07%
--------------------------------------------



> P3 evaluation

In [183]:
# First run the analysis to get the results dictionary
results_p3 = analyze_sales_forecast(final_3)

# Then format the results
print(format_results(results_p3, "P3"))


ANALYSIS RESULTS - P3
--------------------------------------------
Metric                               | Value
--------------------------------------------
Avg Forecast Sales (next 10m):       €14,858,782.50
Forecast Growth (next 10m):          29.74%
YoY Growth (2022 vs 2021):           7.99%
YTD Growth (2023 vs 2022):           3.62%
--------------------------------------------



> P4 evaluation

In [184]:
# First run the analysis to get the results dictionary
results = analyze_sales_forecast(final_4)

# Then format the results
print(format_results(results, "P4"))


ANALYSIS RESULTS - P4
--------------------------------------------
Metric                               | Value
--------------------------------------------
Avg Forecast Sales (next 10m):       €339,579.98
Forecast Growth (next 10m):          0.96%
YoY Growth (2022 vs 2021):           -21.14%
YTD Growth (2023 vs 2022):           23.17%
--------------------------------------------



> P5 evaluation

In [185]:
# First run the analysis to get the results dictionary
results = analyze_sales_forecast(final_5)

# Then format the results
print(format_results(results, "P5"))


ANALYSIS RESULTS - P5
--------------------------------------------
Metric                               | Value
--------------------------------------------
Avg Forecast Sales (next 10m):       €10,834,543.20
Forecast Growth (next 10m):          11.55%
YoY Growth (2022 vs 2021):           -3.32%
YTD Growth (2023 vs 2022):           26.65%
--------------------------------------------



> P6 evaluation

In [186]:
# First run the analysis to get the results dictionary
results = analyze_sales_forecast(final_6)

# Then format the results
print(format_results(results, "P6"))


ANALYSIS RESULTS - P6
--------------------------------------------
Metric                               | Value
--------------------------------------------
Avg Forecast Sales (next 10m):       €259,748.48
Forecast Growth (next 10m):          -34.11%
YoY Growth (2022 vs 2021):           -40.28%
YTD Growth (2023 vs 2022):           -21.54%
--------------------------------------------



> P8 evaluation

In [187]:
# First run the analysis to get the results dictionary
results = analyze_sales_forecast(final_8)

# Then format the results
print(format_results(results, "P8"))


ANALYSIS RESULTS - P8
--------------------------------------------
Metric                               | Value
--------------------------------------------
Avg Forecast Sales (next 10m):       €1,532,561.77
Forecast Growth (next 10m):          41.97%
YoY Growth (2022 vs 2021):           24.02%
YTD Growth (2023 vs 2022):           19.31%
--------------------------------------------



> P9 evaluation

In [188]:
# First run the analysis to get the results dictionary
results = analyze_sales_forecast(final_9)

# Then format the results
print(format_results(results, "P9"))


ANALYSIS RESULTS - P9
--------------------------------------------
Metric                               | Value
--------------------------------------------
Avg Forecast Sales (next 10m):       €10,777.15
Forecast Growth (next 10m):          56.86%
YoY Growth (2022 vs 2021):           -22.17%
YTD Growth (2023 vs 2022):           595.21%
--------------------------------------------



> P11 evaluation

In [189]:
# First run the analysis to get the results dictionary
results = analyze_sales_forecast(final_11)

# Then format the results
print(format_results(results, "P11"))


ANALYSIS RESULTS - P11
--------------------------------------------
Metric                               | Value
--------------------------------------------
Avg Forecast Sales (next 10m):       €1,722,006.20
Forecast Growth (next 10m):          1.22%
YoY Growth (2022 vs 2021):           -8.10%
YTD Growth (2023 vs 2022):           33.95%
--------------------------------------------



> P12 evaluation

In [190]:
# First run the analysis to get the results dictionary
results = analyze_sales_forecast(final_12)

# Then format the results
print(format_results(results, "P12"))


ANALYSIS RESULTS - P12
--------------------------------------------
Metric                               | Value
--------------------------------------------
Avg Forecast Sales (next 10m):       €281,314.06
Forecast Growth (next 10m):          4.08%
YoY Growth (2022 vs 2021):           -11.25%
YTD Growth (2023 vs 2022):           6.31%
--------------------------------------------



> P13 evaluation

In [191]:
# First run the analysis to get the results dictionary
results = analyze_sales_forecast(final_13)

# Then format the results
print(format_results(results, "P13"))


ANALYSIS RESULTS - P13
--------------------------------------------
Metric                               | Value
--------------------------------------------
Avg Forecast Sales (next 10m):       €18,352.64
Forecast Growth (next 10m):          45.13%
YoY Growth (2022 vs 2021):           -0.88%
YTD Growth (2023 vs 2022):           -52.65%
--------------------------------------------



> P14 evaluation

In [192]:
# First run the analysis to get the results dictionary
results = analyze_sales_forecast(final_14)

# Then format the results
print(format_results(results, "P14"))


ANALYSIS RESULTS - P14
--------------------------------------------
Metric                               | Value
--------------------------------------------
Avg Forecast Sales (next 10m):       €19,182.37
Forecast Growth (next 10m):          46.85%
YoY Growth (2022 vs 2021):           42.21%
YTD Growth (2023 vs 2022):           236.87%
--------------------------------------------



> P16 evaluation

In [193]:
# First run the analysis to get the results dictionary
results = analyze_sales_forecast(final_16)

# Then format the results
print(format_results(results, "P16"))


ANALYSIS RESULTS - P16
--------------------------------------------
Metric                               | Value
--------------------------------------------
Avg Forecast Sales (next 10m):       €120,845.33
Forecast Growth (next 10m):          -29.90%
YoY Growth (2022 vs 2021):           -42.34%
YTD Growth (2023 vs 2022):           82.29%
--------------------------------------------



> P20 evaluation

In [194]:
# First run the analysis to get the results dictionary
results = analyze_sales_forecast(final_20)

# Then format the results
print(format_results(results, "P20"))


ANALYSIS RESULTS - P20
--------------------------------------------
Metric                               | Value
--------------------------------------------
Avg Forecast Sales (next 10m):       €1,919.62
Forecast Growth (next 10m):          299.76%
YoY Growth (2022 vs 2021):           80.69%
YTD Growth (2023 vs 2022):           59.31%
--------------------------------------------



> P36 evaluation

In [195]:
# First run the analysis to get the results dictionary
results = analyze_sales_forecast(final_36)

# Then format the results
print(format_results(results, "P36"))


ANALYSIS RESULTS - P36
--------------------------------------------
Metric                               | Value
--------------------------------------------
Avg Forecast Sales (next 10m):       €17,576.83
Forecast Growth (next 10m):          16.47%
YoY Growth (2022 vs 2021):           -53.33%
YTD Growth (2023 vs 2022):           -17.82%
--------------------------------------------

